# Introduzione

Se non già presenti, installiamo i pacchetti python relativi all'autenticazione e all'utilizzo dei servizi di Google. 

In [1]:
!pip install google-auth google-auth-oauthlib googleapiclient

ERROR: Could not find a version that satisfies the requirement googleapiclient (from versions: none)
ERROR: No matching distribution found for googleapiclient


Importiamo tutti i metodi e le librerie che ci serviranno nel corso dei test.

In [ ]:
from vertexai.generative_models import GenerativeModel, Part
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai
import vertexai.preview.generative_models as generative_models

Creiamo un'istanza del modello, ossia Gemini 1.5 Pro. 

In [ ]:
vertexai.init(project="<PROJECT_ID>", location="<REGION>")
model = GenerativeModel("gemini-1.5-pro-001")

Settiamo due configurazioni diverse per quel che riguarda la scelta dei token e i filtri di sicurezza. Questi andranno inseriti in ogni  prompt. 

In [4]:
generation_config1 = {
    "max_output_tokens": 8192,
    "temperature": 0.5,
    "top_p": 0.95,
    "top_k": 32,
    "candidate_count":1
}

generation_config2 = {
    "max_output_tokens": 8192,
    "temperature": 0.9,
    "top_p": 1.0,
    "top_k": 32,
    "candidate_count":1
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

Creaimo il Rag corpus. Il seguente passaggio crea la base di dati personalizzata a cui si attingerà con le retrieval query partendo dai libri e gli esercizi inerenti al corso presenti su Google Drive. 

In [ ]:
display_name = "reti_logiche"

textbook_path_en = ["<TEXTBOOK_PATH_EN>"]

other_sources_paths_en = ["<OTHER_SOURCES_PATH_EN>"]

rag_corpus = rag.create_corpus(display_name=display_name)

all_file_paths = textbook_path_en + other_sources_paths_en

#il massimo che si può importare per volta è 25 files
file_path_groups = [all_file_paths[i:i+25] for i in range(0, len(all_file_paths), 25)]

for group in file_path_groups:
    response = rag.import_files(
        rag_corpus.name,
        group,
        chunk_size=512,  # Optional
        chunk_overlap=100,  # Optional
    )

Creiamo infine le funzioni con cui interrogheremo Gemini, sia in modalità multimodale che di solo testo, e con cui recuperemo le informazioni di contesto mediante RAG. 

In [ ]:
def askMultimodalGemini(prompt_text, generation_config, *args):
    contents = [prompt_text]
    for arg in args:
        contents.append(arg)
    responses = model.generate_content(
      contents,
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
      )
    
    print("--------Response--------")
    for response in responses:
        print(response.text, end="")
        
def askGemini(prompt_text, generation_config):
    responses = model.generate_content(
      prompt_text,
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
      )
    
    print("--------Response--------")
    for response in responses:
        print(response.text, end="")
          
def retrieveInformationFromCustomDataSource(retrieval_text):
    retrieved_content = rag.retrieval_query(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,
                )
            ],
            text=retrieval_text,
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        )

    text_retrieved = ""
    if retrieved_content.contexts:

        for context in retrieved_content.contexts.contexts:
            text_retrieved += context.text + ". \n\n"
        print("--------Text Retrieved--------")
        print("\n\n" + text_retrieved)
        return text_retrieved
    else:
        print("No content retrieved.")
        return ""

We instantiate the model enhanced with retrieval tools.

In [ ]:
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                )
            ],
            similarity_top_k=10,  # Optional
            vector_distance_threshold=0.5,  # Optional
        ),
    )
)

rag_model = GenerativeModel(
    model_name="gemini-1.5-pro-001", tools=[rag_retrieval_tool])

# Esempi e contesto per la categoria 1: Reti combinatorie

In [ ]:
# Immagini degli esempi per il few-shot prompting
example_image1 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image1-cartegory1>.png")
example_image2 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image2-cartegory1>.png")
example_image3 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image3-cartegory1>.png")

# Descrizione delle immagini degli esempi per il few-shot prompting
example_image1_description = """The combinational circuit has two inputs (A, B) and six outputs (NOT, AND, NAND, OR, NOR, EXOR).
NOT is generated by a NOT gate with input A.
AND is generated by a two-input AND gate: A and B.
NAND is generated by a two-input NAND gate: A and B.
OR is generated by an two-input OR gate: A and B.
NOR is generated by a two-input NOR gate: A and B.
EXOR is generated by a two-input EXOR gate: A and B.
"""
example_image2_description = """The combinational circuit has two inputs (X, Y) and one output (F).
A is an intermediate variable generated by a two-input AND gate: Y and X negated.
B is an intermediate variable generated by a two-input AND gate: X and Y negated.
F is generated by a two-input OR gate: A and B.
"""
example_image3_description ="""The combinational circuit has three inputs (X, Y, Z) and two outputs (F, G).
A is an intermediate variable generated by a two-input OR gate: X and Y.
F is generated by a two-input EXOR gate: A and Z negated.
G is generated by a three-input AND gate: X, Y, and Z negated.
"""

# Soluzioni degli esempi per il few-shot prompting
solution1 = """ A | B | NOT | AND | NAND | OR | NOR | EXOR
---|---|-----|-----|------|----|-----|-----
 0 | 0 |  1  |  0  |  1   |  0 |  1  |  0
 0 | 1 |  1  |  0  |  1   |  1 |  0  |  1
 1 | 0 |  0  |  0  |  1   |  1 |  0  |  1
 1 | 1 |  0  |  1  |  0   |  1 |  0  |  0
"""
solution2 = """ Y | X | F
---|---|---
 0 | 0 | 0
 0 | 1 | 1
 1 | 0 | 1
 1 | 1 | 0
"""
solution3 = """ Z | Y | X | F | G
---|---|---|---|---
 0 | 0 | 0 | 1 | 0
 0 | 0 | 1 | 0 | 0
 0 | 1 | 0 | 0 | 0
 0 | 1 | 1 | 0 | 1
 1 | 0 | 0 | 0 | 0
 1 | 0 | 1 | 1 | 0
 1 | 1 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0
"""

# Costruzione dei prompt degli esempi per il few-shot prompting
input1 = f'{{"Combinational network": "{example_image1}", "Network description": "{example_image1_description}"}}'
input2 = f'{{"Combinational network": "{example_image2}", "Network description": "{example_image2_description}"}}'
input3 = f'{{"Combinational network": "{example_image3}", "Network description": "{example_image3_description}"}}'

# Recupero esplicito delle informazioni sull'argomento (direct retrieval)
retrieval_text = """What is a combinational network?
How do you analyze a combinational network?
What is a truth table of a combinational network?
How do you construct the truth table of a combinational network?"""

text_retrieved = retrieveInformationFromCustomDataSource(retrieval_text)

# Libro di testo sull'argomento
textbook = Part.from_uri(mime_type="application/pdf", uri="gs://<bucket-name-en>/<textbook-file>.pdf")

--------Text Retrieved--------


■ Describe the fundamental idea of programmable logic devices (PLDs).
■ Describe the steps involved in programming a PLD to perform a simple 
combinational logic function.
■ Describe hierarchical design methods.
■ Identify proper data types for single-bit, bit array, and numeric value 
variables.
■ Describe logic circuits using HDL control structures IF/ELSE, IF/
ELSIF, and CASE.
■ Select the appropriate HDL control structure for a given problem.
■ Introduction 
In Chapter 3, we studied the operation of all the basic logic gates, and we 
used Boolean algebra to describe and analyze circuits that were made up 
of combinations of logic gates. These circuits can be classified as combi￾national logic circuits because, at any time, the logic level at the output 
depends on the combination of logic levels present at the inputs. A combi￾national circuit has no memory characteristic, so its output depends only on 
the current value of its inputs.
In this chapte

# Exercise 1

## 1.0 Definition of the Prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image1>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video1>.mp4")

image_description = """The combinational network has three inputs (S0, S1, EN) and four outputs (U0, U1, U2, U3).
U0 is generated by a three-input AND gate: negated S0, negated S1, and EN.
U1 is generated by a three-input AND gate: S0, negated S1, and EN.
U2 is generated by a three-input AND gate: negated S0, S1, and EN.
U3 is generated by a three-input AND gate: S0, S1, and EN.
"""

prompt_text = f""" A combinational network is a circuit in which inputs and outputs can only assume two states corresponding to high or low levels, and the outputs are solely a function of the inputs.
A truth table is a tool used to represent the logical behavior of a combinational network in a tabular format. The truth table lists all possible combinations of the circuit’s input values and shows the corresponding output values for each combination.

Given {image} as an image containing the graphical representation of a combinational network, complete the truth table of the circuit.

The description of the circuit is provided below:
{image_description}
"""

## 1.1 Exercise without Additional Context

In [77]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| S0 | S1 | EN | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

**Explanation:**

* **EN = 0:** When EN is 0, all outputs (U0-U3) are 0 regardless of the S0 and S1 inputs. This is because an AND gate requires all inputs to be 1 for the output to be 1.
* **EN = 1:** When EN is 1, the outputs are determined by the combination of S0 and S1:
    * **U0 = 1** only when S0 = 0 and S1 = 0 (negated S0 and negated S1 are both 1).
    * **U1 = 1** only when S0 = 1 and S1 = 0.
    * **U2 = 1** only when S0 = 0 and S1 = 1.
    * **U3 = 1** only when S0 = 1 and S1 = 1.

This circuit acts as a 2-to-4 decoder with an enable input (EN). When enabled, it selects one of the four outputs (U0-U3) based on the 2-bit input (S0S1). 


## 1.2 Exercise with Direct Retrieval

In [78]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| S0 | S1 | EN | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

**Explanation:**

* **EN = 0:** When EN is 0, all outputs (U0-U3) are 0 regardless of the inputs S0 and S1. This is because an AND gate requires all inputs to be 1 for the output to be 1.
* **EN = 1:** When EN is 1, the circuit acts as a decoder based on the values of S0 and S1:
    * **S0 = 0, S1 = 0:** Only U0 is 1.
    * **S0 = 1, S1 = 0:** Only U1 is 1.
    * **S0 = 0, S1 = 1:** Only U2 is 1.
    * **S0 = 1, S1 = 1:** Only U3 is 1.

This truth table represents the behavior of a 2-to-4 line decoder with an enable input (EN). 


## 1.3 Exercise with Direct Retrieval and Solution Video

In [79]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is requesting the completion of a truth table for a combinational network. The description of the circuit, the original image, and a video solution are provided.

Based on the provided information, here is the completed truth table:

| EN | S1 | S0 | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 | 


## 1.4 Exercise with Solution Video

In [80]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user filled the truth table with the following values:

| EN | S1 | S0 | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 |
| 1 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 | 


## 1.5 Exercise with Textbook in the Prompt

In [81]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
| S0 | S1 | EN | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

**Explanation:**

The truth table is constructed by considering all possible combinations of the three inputs (S0, S1, EN). For each combination, the outputs (U0, U1, U2, U3) are determined based on the logic gates described:

* **U0:**  1 only when S0=0, S1=0, and EN=1 (negated S0, negated S1, and EN).
* **U1:** 1 only when S0=1, S1=0, and EN=1 (S0, negated S1, and EN).
* **U2:** 1 only when S0=0, S1=1, and EN=1 (negated S0, S1, and EN).
* **U3:** 1 only when S0=1, S1=1, and EN=1 (S0, S1, and EN).

Since all outputs are generated by AND gates, they will be 0 for any input combination that doesn't meet the specific conditions listed above. 


## 1.6 Exercise with Textbook in the Prompt and Solution Video

In [82]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
The completed truth table is:

| EN | S1 | S0 | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 | 


## 1.7 Exercise with Retrieval via Tool

In [85]:
prompt_text = f"""A combinational circuit is a circuit in which the inputs and outputs can only assume two states corresponding to high or low levels, and the outputs are solely a function of the inputs.
A truth table is a tool used to represent the logical behavior of a combinational circuit in a tabular format. The truth table lists all possible combinations of the circuit’s input values and shows the corresponding output values for each combination.
A combinational circuit is structured as follows:
{image_description}
Complete the truth table of the circuit.
"""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

| S0 | S1 | EN | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 | 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "3 input AND gate truth table"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "3 input AND gate truth table"
        ]
      }
    }
  ]
}

## 1.8 Exercise with Few-Shot Prompting

In [27]:
input = f'{{"Combinational network": "{image}", "Network description": "{image_description}"}}' 

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
 EN | S1 | S0 | U0 | U1 | U2 | U3
---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 0 | 0 | 0
 0 | 0 | 1 | 0 | 0 | 0 | 0
 0 | 1 | 0 | 0 | 0 | 0 | 0
 0 | 1 | 1 | 0 | 0 | 0 | 0
 1 | 0 | 0 | 1 | 0 | 0 | 0
 1 | 0 | 1 | 0 | 1 | 0 | 0
 1 | 1 | 0 | 0 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0 | 0 | 1


## 1.9 Exercise with Few-Shot Prompting, Textbook in the Prompt, and Solution Video

In [84]:
input = f'{{"Combinational network": "{image}", "Network description": "{image_description}"}}' 

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
 EN | S1 | S0 | U0 | U1 | U2 | U3
---|---|-----|-----|------|----|-----
 0 | 0 | 0 |  1  |  0  |  0   |  0
 0 | 0 | 1 |  1  |  0  |  0   |  0
 0 | 1 | 0 |  0  |  1  |  0   |  0
 0 | 1 | 1 |  0  |  0  |  0   |  0
 1 | 0 | 0 |  1  |  0  |  0   |  0
 1 | 0 | 1 |  0  |  1  |  0   |  0
 1 | 1 | 0 |  0  |  0  |  1   |  0
 1 | 1 | 1 |  0  |  0  |  0   |  1 


# Exercise 2

## 1.0 Definition of the Prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image2>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video2>.mp4")

image_description = """The combinational network has four inputs (RPar, D2, D1, D0) and three outputs (Check1, Check2, ERR).
A is an intermediate variable generated by a two-input OR gate: RPar and D2.
B is an intermediate variable generated by a two-input NAND gate: RPar and D2.
C is an intermediate variable generated by a two-input NAND gate: D1 and D0.
D is an intermediate variable generated by a two-input OR gate: D1 and D0.
Check1 is generated by a two-input AND gate: A and B.
Check2 is generated by a two-input AND gate: C and D.
F is an intermediate variable generated by a two-input NAND gate: Check1 and Check2.
G is an intermediate variable generated by a two-input OR gate: Check1 and Check2.
ERR is generated by a two-input AND gate: F and G.
"""

prompt_text = f""" A combinational network is a circuit in which inputs and outputs can only assume two states corresponding to high or low levels, and the outputs are solely a function of the inputs.
A truth table is a tool used to represent the logical behavior of a combinational network in a tabular format. The truth table lists all possible combinations of the circuit’s input values and shows the corresponding output values for each combination.

Given {image} as an image containing the graphical representation of a combinational network, complete the truth table of the circuit.

The description of the circuit is provided below:
{image_description}
"""

## 1.1 Exercise without Additional Context

In [15]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| **RPar** | **D2** | **D1** | **D0** | **A** | **B** | **C** | **D** | **Check1** | **Check2** | **F** | **G** | **ERR** |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 |

## 1.2 Exercise with Direct Retrieval

In [16]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| RPar | D2 | D1 | D0 | A | B | C | D | Check1 | Check2 | F | G | ERR |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 0

## 1.3 Exercise with Direct Retrieval and Solution Video

In [18]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The provided context does not contain information that is relevant to the completion of the truth table. The text describes general concepts related to combinational logic circuits, Boolean algebra, Karnaugh maps, and Hardware Description Languages (HDL), but it does not provide specific details about the circuit in the image.

To complete the truth table, we need to analyze the circuit's logic based on the provided description and the image. Here's the completed truth table:

| D0 | D1 | D2 | RPar | Check1 | Check2 | ERR |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 0 | 1 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 | 1 | 1 |
| 0 | 1 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 1 | 0 | 0 |

## 1.4 Exercise with Solution Video

In [25]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user fills the "Check1" column with "0" for all rows except for the rows "0101" and "1010", which are filled with "1".
The user fills the "Check2" column with "0" for all rows except for the rows "0110" and "1001", which are filled with "1".
The user fills the "ERR" column with "0" for all rows except for the rows "0101", "0110", "1001", and "1010", which are filled with "1". 


## 1.5 Exercise with Textbook in the Prompt

In [20]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
| RPar | D2 | D1 | D0 | A | B | C | D | Check1 | Check2 | F | G | ERR |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0

## 1.6 Exercise with Textbook in the Prompt and Solution Video

In [21]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
```
D0 | D1 | D2 | RPar | Check1 | Check2 | ERR
------- | -------- | -------- | -------- | -------- | -------- | --------
0 | 0 | 0 | 0 | 0 | 0 | 0
0 | 0 | 0 | 1 | 1 | 0 | 0
0 | 0 | 1 | 0 | 0 | 0 | 0
0 | 0 | 1 | 1 | 0 | 0 | 0
0 | 1 | 0 | 0 | 0 | 1 | 0
0 | 1 | 0 | 1 | 1 | 1 | 1
0 | 1 | 1 | 0 | 0 | 1 | 0
0 | 1 | 1 | 1 | 0 | 1 | 0
1 | 0 | 0 | 0 | 0 | 1 | 0
1 | 0 | 0 | 1 | 1 | 1 | 1
1 | 0 | 1 | 0 | 0 | 1 | 0
1 | 0 | 1 | 1 | 0 | 1 | 0
1 | 1 | 0 | 0 | 0 | 0 | 0
1 | 1 | 0 | 1 | 1 | 0 | 0
1 | 1 | 1 | 0 | 0 | 0 | 0
1 | 1 | 1 | 1 | 0 | 0 | 0
```

## 1.7 Exercise with Retrieval via Tool

In [24]:
prompt_text = f"""A combinational circuit is a circuit in which the inputs and outputs can only assume two states corresponding to high or low levels, and the outputs are solely a function of the inputs.
A truth table is a tool used to represent the logical behavior of a combinational circuit in a tabular format. The truth table lists all possible combinations of the circuit’s input values and shows the corresponding output values for each combination.
A combinational circuit is structured as follows:
{image_description}
Complete the truth table of the circuit.
"""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

Based on your provided description of the combinational circuit, its truth table can be completed as follows:

| RPar | D2 | D1 | D0 | A   | B   | C   | D   | Check1 | Check2 | F   | G   | ERR |
|------|----|----|----|-----|-----|-----|-----|--------|--------|-----|-----|-----|
| 0    | 0  | 0  | 0  | 0   | 1   | 1   | 0   | 0      | 0      | 1   | 0   | 0   |
| 0    | 0  | 0  | 1  | 0   | 1   | 1   | 1   | 0      | 1      | 1   | 1   | 1   |
| 0    | 0  | 1  | 0  | 0   | 1   | 1   | 1   | 0      | 1      | 1   | 1   | 1   |
| 0    | 0  | 1  | 1  | 0   | 1   | 0   | 1   | 0      | 0      | 1   | 0   | 0   |
| 0    | 1  | 0  | 0  | 1   | 1   | 1   | 0   | 1      | 0      | 1   | 1   | 1   |
| 0    | 1  | 0  | 1  | 1   | 1   | 1   | 1   | 1      | 1      | 0   | 1   | 0   |
| 0    | 1  | 1  | 0  | 1   | 1   | 1   | 1   | 1      | 1      | 0   | 1   | 0   |
| 0    | 1  | 1  | 1  | 1   | 1   | 0   | 1   | 1      | 0      | 1   | 1   | 1   |
| 1    | 0  | 0  | 0  | 1   | 0   | 1   | 0   | 0

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "4 input 3 output combinational circuit truth table"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "4 input 3 output combinational circuit truth table"
        ]
      }
    }
  ]
}

## 1.8 Exercise with Few-Shot Prompting

In [29]:
input = f'{{"Combinational network": "{image}", "Network description": "{image_description}"}}' 

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
RPar | D2 | D1 | D0 | Check1 | Check2 | ERR
---|---|---|---|---|---|---|
 0 | 0 | 0 | 0 | 0 | 0 | 0
 0 | 0 | 0 | 1 | 0 | 1 | 0
 0 | 0 | 1 | 0 | 0 | 1 | 0
 0 | 0 | 1 | 1 | 0 | 1 | 0
 0 | 1 | 0 | 0 | 1 | 0 | 0
 0 | 1 | 0 | 1 | 1 | 1 | 0
 0 | 1 | 1 | 0 | 1 | 1 | 0
 0 | 1 | 1 | 1 | 1 | 1 | 0
 1 | 0 | 0 | 0 | 1 | 0 | 0
 1 | 0 | 0 | 1 | 1 | 1 | 0
 1 | 0 | 1 | 0 | 1 | 1 | 0
 1 | 0 | 1 | 1 | 1 | 1 | 0
 1 | 1 | 0 | 0 | 1 | 0 | 0
 1 | 1 | 0 | 1 | 1 | 1 | 0
 1 | 1 | 1 | 0 | 1 | 1 | 0
 1 | 1 | 1 | 1 | 1 | 1 | 0


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [23]:
input = f'{{"Combinational network": "{image}", "Network description": "{image_description}"}}' 

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
RPar | D2 | D1 | D0 | Check1 | Check2 | ERR
---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 0 | 0 | 0
 0 | 0 | 0 | 1 | 0 | 1 | 0
 0 | 0 | 1 | 0 | 0 | 1 | 0
 0 | 0 | 1 | 1 | 0 | 1 | 0
 0 | 1 | 0 | 0 | 0 | 0 | 0
 0 | 1 | 0 | 1 | 0 | 1 | 0
 0 | 1 | 1 | 0 | 0 | 1 | 0
 0 | 1 | 1 | 1 | 0 | 1 | 0
 1 | 0 | 0 | 0 | 1 | 0 | 0
 1 | 0 | 0 | 1 | 1 | 1 | 1
 1 | 0 | 1 | 0 | 1 | 1 | 1
 1 | 0 | 1 | 1 | 1 | 1 | 1
 1 | 1 | 0 | 0 | 0 | 0 | 0
 1 | 1 | 0 | 1 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0 | 1 | 0
 1 | 1 | 1 | 1 | 0 | 1 | 0 


# Esercizio 3

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image3>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video3>.mp4")

image_description = """The combinational network has four inputs (A, B, C, D) and three outputs (O1, O2, O3).
X is an intermediate variable generated by a three-input EXOR gate: negated A, B and C.
Y is an intermediate variable generated by a three-input NOR gate: B, negated C and D.
O1 is generated by a four-input AND gate: A, X, C and D.
O2 is generated by a two-input OR gate: C and Y.
O3 is generated by a two-input NAND gate: A and Y.
"""

prompt_text = f""" A combinational network is a circuit in which inputs and outputs can only assume two states corresponding to high or low levels, and the outputs are solely a function of the inputs.
A truth table is a tool used to represent the logical behavior of a combinational network in a tabular format. The truth table lists all possible combinations of the circuit’s input values and shows the corresponding output values for each combination.

Given {image} as an image containing the graphical representation of a combinational network, complete the truth table of the circuit.

The description of the circuit is provided below:
{image_description}
"""

## 1.1 Esercizio senza ulteriore contesto

In [31]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| A | B | C | D | X | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |

**Explanation:**

1. **Intermediate Variables:**
    * **X:** Calculated using the EXOR truth table for negated A, B, and C. 
    * **Y:** Calculated using the NOR truth table for B, negated C, and D.

2. **Outputs:**
    * **O1:** Calculated using the AND truth table for A, X, C, and D

## 1.2 Esercizio con recupero diretto

In [32]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| A | B | C | D | X | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 | 1 | 1 | 1 | 0 |
| 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |

**Explanation of the Truth Table:**

The truth table represents all possible input combinations for the four input variables (A, B, C, and D). For each input combination, the outputs of the intermediate variables (X and Y) and the final outputs (O1, O2, and O3) are determined based on t

## 1.3 Esercizio con recupero diretto e video soluzione

In [33]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
```
A | B | C | D | EXOR | NOR | O1 | O2 | O3
---|---|---|---|---|---|---|---|---
0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1
0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1
0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1
0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 1
0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1
0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1
0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1
0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 1
1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0
1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1
1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0
1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1
1 | 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0
1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1
1 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0
1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 1
```

**Programmable Logic Devices (PLDs)**

PLDs are integrated circuits that can be programmed to implement custom logic functions. They contain an array of logic gates and interconnections that can be configured by the user. This allows for the creation of complex combinational and sequential logic circuits within a single chip.

**Programming a

## 1.4 Esercizio con video soluzione

In [34]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user writes the value "1" in the "EXOR" column of the table in the following rows: 1, 2, 4, 8, 11, 12, 14, 15. Then, they write "0" in the "NOR" column of the table in the following rows: 1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15. Then, they write "1" in the "NOR" column of the table in row 4 and row 8. Then, they write "0" in the "O3" column of the table in the following rows: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15. Then, they write "1" in the "O3" column of the table in row 12. Then, they write "0" in the "O2" column of the table in the following rows: 1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15. Then, they write "1" in the "O2" column of the table in the following rows: 4, 8, 12. 


## 1.5 Esercizio con libro di testo nel prompt

In [35]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
| A | B | C | D | X | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 1 |

**Explanation:**

1. **Intermediate Variables:**
    - **X:**  Calculated using the EXOR truth table for inputs (not A, B, C).
    - **Y:** Calculated using the NOR truth table for inputs (B, not C, D).

2. **Outputs:**
    - **O1:** Calculated using the AND truth table for inputs (A, X

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [36]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
```
A B C D | X Y | O1 O2 O3
------- | -------- | --------
0 0 0 0 | 1 1 | 0 1 0
0 0 0 1 | 1 0 | 0 0 1
0 0 1 0 | 1 1 | 0 1 0
0 0 1 1 | 1 0 | 0 0 1
0 1 0 0 | 0 1 | 0 1 0
0 1 0 1 | 0 1 | 0 1 0
0 1 1 0 | 0 1 | 0 1 0
0 1 1 1 | 0 0 | 0 0 1
1 0 0 0 | 0 1 | 0 1 1
1 0 0 1 | 0 0 | 0 0 1
1 0 1 0 | 0 1 | 1 1 1
1 0 1 1 | 0 0 | 1 0 1
1 1 0 0 | 1 1 | 0 1 0
1 1 0 1 | 1 0 | 0 0 1
1 1 1 0 | 1 1 | 0 1 0
1 1 1 1 | 1 0 | 0 0 1
```

## 1.7 Esercizio con recupero mediante tool

In [39]:
prompt_text = f"""A combinational circuit is a circuit in which the inputs and outputs can only assume two states corresponding to high or low levels, and the outputs are solely a function of the inputs.
A truth table is a tool used to represent the logical behavior of a combinational circuit in a tabular format. The truth table lists all possible combinations of the circuit’s input values and shows the corresponding output values for each combination.
A combinational circuit is structured as follows:
{image_description}
Complete the truth table of the circuit.
"""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

| A | B | C | D | X | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 0 |
| 0 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 | 1 | 1 | 1 | 0 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "3 input exor gate truth table"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "3 input exor gate truth table"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [40]:
input = f'{{"Combinational network": "{image}", "Network description": "{image_description}"}}' 

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
 D | C | B | A | X | Y | O1 | O2 | O3
---|---|---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0
 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0
 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1
 0 | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 1
 0 | 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0
 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 0
 0 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 1
 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1
 1 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 0
 1 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0
 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1
 1 | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 1
 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0
 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 1
 1 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 1 


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [38]:
input = f'{{"Combinational network": "{image}", "Network description": "{image_description}"}}' 

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
D | C | B | A | O1 | O2 | O3
---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 0 | 0 | 1
 0 | 0 | 0 | 1 | 0 | 0 | 1
 0 | 0 | 1 | 0 | 0 | 1 | 1
 0 | 0 | 1 | 1 | 0 | 1 | 0
 0 | 1 | 0 | 0 | 0 | 1 | 1
 0 | 1 | 0 | 1 | 1 | 1 | 1
 0 | 1 | 1 | 0 | 0 | 1 | 1
 0 | 1 | 1 | 1 | 0 | 1 | 0
 1 | 0 | 0 | 0 | 0 | 0 | 1
 1 | 0 | 0 | 1 | 0 | 0 | 0
 1 | 0 | 1 | 0 | 0 | 0 | 1
 1 | 0 | 1 | 1 | 0 | 0 | 0
 1 | 1 | 0 | 0 | 0 | 1 | 1
 1 | 1 | 0 | 1 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0 | 0 | 1
 1 | 1 | 1 | 1 | 0 | 0 | 0


# Esercizio 4

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image4>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video4>.mp4")

image_description = """The combinatorial network has four inputs (A, B, C, D) and two outputs (F, G).
X is an intermediate variable generated by a two-input AND gate: negated A and B.
Y is an intermediate variable generated by a two-input AND gate: A and negated B.
W is an intermediate variable generated by a two-input AND gate: A and B.
Z is an intermediate variable generated by a two-input AND gate: negated A and negated B.
J is an intermediate variable generated by a two-input OR gate: X and Y.
K is an intermediate variable generated by a two-input OR gate: W and Z.
M is an intermediate variable generated by a two-input AND gate: J and negated C.
N is an intermediate variable generated by a two-input AND gate: negated C and negated D.
O is an intermediate variable generated by a two-input AND gate: K and negated D.
F is generated by a three-input OR gate: M, N, and O.
G is generated by a four-input EXOR gate: C, D, M, and O.
"""

prompt_text = f""" A combinational network is a circuit in which inputs and outputs can only assume two states corresponding to high or low levels, and the outputs are solely a function of the inputs.
A truth table is a tool used to represent the logical behavior of a combinational network in a tabular format. The truth table lists all possible combinations of the circuit’s input values and shows the corresponding output values for each combination.

Given {image} as an image containing the graphical representation of a combinational network, complete the truth table of the circuit.

The description of the circuit is provided below:
{image_description}
"""

## 1.1 Esercizio senza ulteriore contesto

In [24]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| A | B | C | D | X | Y | W | Z | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 |

## 1.2 Esercizio con recupero diretto

In [25]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| A | B | C | D | X | Y | W | Z | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 |

## 1.3 Esercizio con recupero diretto e video soluzione

In [26]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The provided context does not contain the information required to complete the truth table. However, the video you provided contains the solution.

Here is the completed truth table:

| A | B | C | D | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 1 | 1 |

## 1.4 Esercizio con video soluzione

In [27]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user fills in the "J" column of the truth table with all 0's. Then she fills in the "K" column with "1111000000001111". Then she fills in the "M" column with "0000110011110000". Then she fills in the "N" column with "1000100010001000". Then she fills in the "O" column with "1010000010100000". Finally, she fills in the "F" column with "1010111011111010" and the "G" column with "1110010110110100". 


## 1.5 Esercizio con libro di testo nel prompt

In [28]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
| A | B | C | D | X | Y | W | Z | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 |

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [29]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
```
A B C D | X Y W Z J K M N O | F G
------- | -------- | --------
0 0 0 0 | 1 0 0 1 1 1 0 1 1 | 1 0
0 0 0 1 | 1 0 0 1 1 1 0 1 0 | 1 1
0 0 1 0 | 1 0 0 1 1 1 0 0 0 | 0 1
0 0 1 1 | 1 0 0 1 1 1 0 0 0 | 0 0
0 1 0 0 | 0 1 0 0 1 0 1 1 0 | 1 0
0 1 0 1 | 0 1 0 0 1 0 1 1 0 | 1 1
0 1 1 0 | 0 1 0 0 1 0 0 0 0 | 0 1
0 1 1 1 | 0 1 0 0 1 0 0 0 0 | 0 0
1 0 0 0 | 0 0 1 0 0 1 0 1 1 | 1 1
1 0 0 1 | 0 0 1 0 0 1 0 1 0 | 1 0
1 0 1 0 | 0 0 1 0 0 1 0 0 0 | 0 0
1 0 1 1 | 0 0 1 0 0 1 0 0 0 | 0 1
1 1 0 0 | 0 0 1 0 0 1 0 1 0 | 1 1
1 1 0 1 | 0 0 1 0 0 1 0 1 0 | 1 0
1 1 1 0 | 0 0 1 0 0 1 0 0 0 | 0 0
1 1 1 1 | 0 0 1 0 0 1 0 0 0 | 0 1 
``` 


## 1.7 Esercizio con recupero mediante tool

In [32]:
prompt_text = f"""A combinational circuit is a circuit in which the inputs and outputs can only assume two states corresponding to high or low levels, and the outputs are solely a function of the inputs.
A truth table is a tool used to represent the logical behavior of a combinational circuit in a tabular format. The truth table lists all possible combinations of the circuit’s input values and shows the corresponding output values for each combination.
A combinational circuit is structured as follows:
{image_description}
Complete the truth table of the circuit.
"""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

| **A** | **B** | **C** | **D** | **X** | **Y** | **W** | **Z** | **J** | **K** | **M** | **N** | **O** | **F** | **G** |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |
| 0 | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {}
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {}
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [30]:
input = f'{{"Combinational network": "{image}", "Network description": "{image_description}"}}' 

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
 D | C | B | A | F | G
---|---|---|---|---|---|
 0 | 0 | 0 | 0 | 1 | 0
 0 | 0 | 0 | 1 | 1 | 1
 0 | 0 | 1 | 0 | 1 | 1
 0 | 0 | 1 | 1 | 1 | 0
 0 | 1 | 0 | 0 | 0 | 1
 0 | 1 | 0 | 1 | 0 | 0
 0 | 1 | 1 | 0 | 0 | 0
 0 | 1 | 1 | 1 | 1 | 1
 1 | 0 | 0 | 0 | 1 | 1
 1 | 0 | 0 | 1 | 1 | 0
 1 | 0 | 1 | 0 | 1 | 0
 1 | 0 | 1 | 1 | 1 | 1
 1 | 1 | 0 | 0 | 0 | 0
 1 | 1 | 0 | 1 | 0 | 1
 1 | 1 | 1 | 0 | 0 | 1
 1 | 1 | 1 | 1 | 0 | 0


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [31]:
input = f'{{"Combinational network": "{image}", "Network description": "{image_description}"}}' 

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on combinational networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
D | C | B | A | J | K | M | N | O | F | G
---|---|---|---|---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1
 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 0
 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1
 0 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0
 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1
 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1
 0 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0
 1 | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 1
 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0
 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1
 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1
 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
 1 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1
 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0


# Esempi e contesto per la categoria 2: mappe di Karnaugh e sintesi minima

In [ ]:
# Richietse degli esempi per il few-shot prompting
input1 = """| X | Y | T | Z | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 | 0 |
"""
input2 = """| A1 | A0 | B1 | B0 | S1 |
|----|----|----|----|----|
|  0 |  0 |  0 |  0 |  0 |
|  0 |  0 |  0 |  1 |  0 |
|  0 |  0 |  1 |  0 |  1 |
|  0 |  0 |  1 |  1 |  1 |
|  0 |  1 |  0 |  0 |  0 |
|  0 |  1 |  0 |  1 |  1 |
|  0 |  1 |  1 |  0 |  1 |
|  0 |  1 |  1 |  1 |  0 |
|  1 |  0 |  0 |  0 |  1 |
|  1 |  0 |  0 |  1 |  1 |
|  1 |  0 |  1 |  0 |  0 |
|  1 |  0 |  1 |  1 |  0 |
|  1 |  1 |  0 |  0 |  1 |
|  1 |  1 |  0 |  1 |  0 |
|  1 |  1 |  1 |  0 |  0 |
|  1 |  1 |  1 |  1 |  1 |
"""
input3 = """| X | Y | T | Z | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 1 |
"""

# Soluzioni degli esempi per il few-shot prompting
solution1 = """{"Karnaugh Map":
"        TY
         00  01  11  10
        ----------------
ZX  00 |  0   0   1   0
    01 |  0   0   0   0
    11 |  0   1   0   0
    10 |  0   1   1   0 ",
 "Sum-of-products expression":
 "F = Y!TZ + !XYT",
 "Explanation":
 "We can group the cells containing ones into two prime implicants.
  The first group contains the two cells (1101, 0101). Since the cells contain a one when Y is one, T is zero and Z is one, the first term is equal to Y!TZ.
  The second group contains the two cells (0111, 0110). Since the cells contain a one when X is zero, Y is one and T is one, the third term is equal to !XYT."}"""
solution2 = """{"Karnaugh Map":
"          B1A0
           00  01  11  10
          ----------------
B0A1  00 |  0   0   1   1
      01 |  1   1   0   0
      11 |  1   0   1   0
      10 |  0   1   0   1 ",
 "Sum-of-products expression":
 "S1 = !A1A0!B1B0 + A1A0B1B0 + A1!A0!B1 + !A1!A0B1 + A1!B1!B0 + !A1B1!B0",
 "Explanation":
 "We can group the cells containing ones into six prime implicants.
  The first group contains the one cell (0101). Since the cells contain a one when A1 is zero, A0 is one, B1 is zero and B0 is one, the first term is equal to !A1A0!B1B0.
  The second group contains the one cell (1111). Since the cells contain a one when A1 is one, A0 is one, B1 is one and B0 is one, the second term is equal to A1A0B1B0.
  The third group contains the two cells (1000, 1001). Since the cells contain a one when A1 is one, A0 is zero and B1 is zero, the third term is equal to A1!A0!B1.
  The fourth group contains the two cells (0010, 0011). Since the cells contain a one when A1 is zero, A0 is zero and B1 is one, the fourth term is equal to !A1!A0B1.
  The fifth group contains the two cells (1000, 1100). Since the cells contain a one when A1 is one, B1 is zero and B0 is zero, the fifth term is equal to  A1!B1!B0.
  The sixth group contains the two cells (0010, 0110). Since the cells contain a one when A1 is zero, B1 is one and B0 is zero, the sixth term is equal to !A1B1!B0."}"""
solution3 = """{"Karnaugh Map":
"        TY
         00  01  11  10
        ----------------
ZX  00 |  0   0   0   0
    01 |  0   1   1   1
    11 |  0   1   1   1
    10 |  1   0   0   1 ",
 "Sum-of-products expression":
 "F = XY + XT + !X!YZ",
 "Explanation":
 "We can group the cells containing ones into three prime implicants.
  The first group contains the four cells (1100, 1101, 1110, 1111). Since the cells contain a one when X is one and Y is one, the first term is equal to XY.
  The second group contains the four cells (1110, 1111, 1010, 1011). Since the cells contain a one when X is one and T is one, the second term is equal to XT.
  The third group contains the two cells (0001, 0011). Since the cells contain a one when X is zero, Y is zero, and Z is one, the third term is equal to !X!YZ."}"""

# Recupero esplicito delle informazioni sull'argomento (direct retrieval)
retrieval_text = """What is a Karnaugh map?
How do you construct a K-map from a truth table?
How do you group the cells containing one (1) in a K-map?
How to perform minimal synthesis using K-Maps?"""

text_retrieved = retrieveInformationFromCustomDataSource(retrieval_text)

# Libro di testo sull'argomento
textbook = Part.from_uri(mime_type="application/pdf", uri="gs://<bucket-name-en>/<textbook-file>.pdf")

--------Text Retrieved--------


■ Use K maps to generate the simplest SOP form from a truth table.
The Karnaugh map (K map) is a graphical tool used to simplify a logic equa￾tion or to convert a truth table to its corresponding logic circuit in a simple, 
orderly process. Although a K map can be used for problems involving any 
number of input variables, its practical usefulness is limited to five or six 
variables. The following discussion will be limited to problems with up to 
four inputs because even five- and six-input problems are too involved and 
are best done by a computer program.
Karnaugh Map Format
The K map, like a truth table, is a means for showing the relationship 
between logic inputs and the desired output. Figure 4-11 shows three exam￾ples of K maps for two, three, and four variables, together with the cor￾responding truth tables. These examples illustrate the following important 
points:
1. The truth table gives the value of output X for each combination of 
input 

# Esercizio 5

## 1.0 Definizione del prompt

In [ ]:
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video5>.mp4")

input = """| A1 | A0 | B1 | B0 | P1 |
|----|----|----|----|----|
|  0 |  0 |  0 |  0 |  0 |
|  0 |  0 |  0 |  1 |  0 |
|  0 |  0 |  1 |  0 |  0 |
|  0 |  0 |  1 |  1 |  0 |
|  0 |  1 |  0 |  0 |  0 |
|  0 |  1 |  0 |  1 |  0 |
|  0 |  1 |  1 |  0 |  1 |
|  0 |  1 |  1 |  1 |  1 |
|  1 |  0 |  0 |  0 |  0 |
|  1 |  0 |  0 |  1 |  1 |
|  1 |  0 |  1 |  0 |  0 |
|  1 |  0 |  1 |  1 |  1 |
|  1 |  1 |  0 |  0 |  0 |
|  1 |  1 |  0 |  1 |  1 |
|  1 |  1 |  1 |  0 |  1 |
|  1 |  1 |  1 |  1 |  0 |
"""

prompt_text = f""" Karnaugh maps (K-maps) are used to simplify Boolean expressions. A K-map is a graphical representation of truth tables and helps determine the minimal form of a Boolean function.
Create a Karnaugh map using the truth table provided: {input}.
Determine the minimal form of the represented Boolean function."""

## 1.1 Esercizio senza ulteriore contesto

In [18]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askGemini(prompt, generation_config1)

--------Response--------
## Karnaugh Map and Minimal Form

Here's the Karnaugh map for the provided truth table:

```
             B1B0
       00  01  11  10
     +---+---+---+---+
 A1A0| 0 | 0 | 0 | 0 | 00
     +---+---+---+---+
      | 0 | 0 | 1 | 1 | 01
     +---+---+---+---+
      | 0 | 1 | 0 | 1 | 11
     +---+---+---+---+
      | 0 | 1 | 1 | 0 | 10
     +---+---+---+---+
```

**Grouping the 1s:**

We can group the 1s in the K-map as follows:

* **Group 1:** A rectangle covering the four corners (wraps around): This group represents the term **B0A0**.
* **Group 2:** Two adjacent 1s in the third row: This group represents the term **A1B1'B0**.

**Minimal Form:**

The minimal form of the Boolean function is the sum of the terms representing the groups:

**P1 = B0A0 + A1B1'B0** 


## 1.2 Esercizio con recupero diretto

In [19]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askGemini(prompt, generation_config1)

--------Response--------
```
A1A0\B1B0 | 00  | 01  | 11 | 10 
-------|----|----|----|----
    00  |  0 |  0 |  0 |  0 
    01  |  0 |  0 |  1 |  1 
    11  |  0 |  1 |  0 |  1 
    10  |  0 |  1 |  1 |  0 
```

**Looping and Simplification:**

1. **Octet:** There are no octets (groups of eight 1s).
2. **Quads:** There are no quads (groups of four 1s).
3. **Pairs:** 
   - We can form a pair of the two 1s in the top right corner:  `B1 * B0`
   - We can form a pair of the two 1s in the bottom right corner: `A1 * B0`
   - We can form a pair of the two 1s on the right edge in the second and third row: `A0 * B1`

**Minimal SOP Expression:**

The minimal Sum-of-Products (SOP) expression is the OR of the terms derived from the looping:

**P1 = B1B0 + A1B0 + A0B1** 


## 1.3 Esercizio con recupero diretto e video soluzione

In [20]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
```
A1A0
B1B0  00  01  11  10
------ -- -- -- --
00      0   0   0   0
01      0   0   1   1
11      0   1   0   1
10      0   1   1   0
```

**Minimal Form (SOP):**

P1 = A1'B1B0 + A1'B0A0 + A1B1'B0 + A1B0A0

**Explanation:**

1. **Construct the K-map:** The K-map is a 4x4 grid representing all possible combinations of the four input variables (A1, A0, B1, B0). The output P1 is filled in each cell based on the truth table.

2. **Identify Groups:** We look for groups of '1's that are powers of 2 (1, 2, 4, 8, etc.).  We can use:
    * **Pairs:** Two adjacent '1's (horizontally or vertically).
    * **Quads:** Four adjacent '1's in a square or rectangular shape.
    * **Octets:** Eight adjacent '1's.

   In this K-map, we have four pairs:
    * Green: A1'B1B0 
    * Yellow: A1'B0A0
    * Red: A1B1'B0
    * Blue: A1B0A0

3. **Write the Boolean Expression:** For each group, we write a product term where:
    * Variables that remain the same within the group are inc

## 1.4 Esercizio con video soluzione

In [21]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
The video provided shows how to create a Karnaugh map from the given truth table and how to derive the minimal form of the Boolean function.

**Karnaugh Map:**

```
     B1 B0
A1 A0 00 01 11 10
--------------------
00   0  0  0  0 
01   0  0  1  1 
11   0  1  0  1 
10   0  1  1  0 
```

**Grouping and Minimal Form:**

The video identifies four essential prime implicants, represented by the colored groups on the Karnaugh map:

* **Green:** A1 * B1' * B0
* **Yellow:** A0 * B1 * B0'
* **Pink:** A0 * B1' * B0
* **Light Blue:** A1' * A0 * B1

Combining these groups with OR operations gives the minimal form of the Boolean function:

**P1 = (A1 * B1' * B0) + (A0 * B1 * B0') + (A0 * B1' * B0) + (A1' * A0 * B1)** 


## 1.5 Esercizio con libro di testo nel prompt

In [22]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook)

--------Response--------
## Karnaugh Map and Minimal Form

Here's the Karnaugh map for the provided truth table:

```
B1B0
A1A0  00  01  11  10
---------------------
00    0   0   0   0
01    0   0   1   1
11    0   1   0   1
10    0   1   1   0
```

**Determining the Minimal Form:**

1. **Identify the largest groupings of 1s:** We can create three groupings:
    * A group of four 1s covering the center of the map (A0B1 + A0B0).
    * A group of two 1s in the top right corner (A1B0).
    * A group of two 1s in the bottom left corner (A1B1).

2. **Write the Boolean expression for each grouping:**
    *  A0B1 + A0B0
    *  A1B0
    *  A1B1

3. **Combine the expressions with OR:** The minimal form of the Boolean function is:

**P1 = A0B1 + A0B0 + A1B0 + A1B1** 


## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [23]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
The Karnaugh map for the provided truth table is:

```
B1B0  00  01  11  10
A1A0
---------
00   | 0   0   0   0
---------
01   | 0   0   1   1
---------
11   | 0   1   0   1
---------
10   | 0   1   1   0
---------
```

The minimal form of the Boolean function can be derived by grouping the "1"s in the K-map. We can identify the following essential prime implicants:

* **A1 * B0:** This group covers the "1"s in cells (10,01), (11,01).
* **A0 * B1:** This group covers the "1"s in cells (01,10), (01,11), (11,10).

Therefore, the minimal form of the Boolean function is:

**P1 = A1 * B0 + A0 * B1** 


## 1.7 Esercizio con recupero mediante tool

In [26]:
responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

The Karnaugh map for the given truth table is:

```
     B1B0
A1A0  00 01 11 10
------------------
 00 |  0  0  0  0
 01 |  0  0  1  1
 10 |  0  1  1  0
 11 |  0  1  0  1 
```

The minimal form of the Boolean function can be derived by grouping the adjacent 1s in the Karnaugh map. The largest possible groups should be made, and each group should contain a number of cells that is a power of 2 (1, 2, 4, 8, etc.).

From the Karnaugh map, we can make the following groups:

* **Group 1:** A1B0 (covers cells 1001 and 1101)
* **Group 2:** A0B1 (covers cells 0110 and 0111)

Therefore, the minimal form of the Boolean function is: **P1 = A1B0 + A0B1** 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "Karnaugh map for the truth table provided: | A1 | A0 | B1 | B0 | P1 |\n|----|----|----|----|----|\n|  0 |  0 |  0 |  0 |  0 |\n|  0 |  0 |  0 |  1 |  0 |\n|  0 |  0 |  1 |  0 |  0 |\n|  0 |  0 |  1 |  1 |  0 |\n|  0 |  1 |  0 |  0 |  0 |\n|  0 |  1 |  0 |  1 |  0 |\n|  0 |  1 |  1 |  0 |  1 |\n|  0 |  1 |  1 |  1 |  1 |\n|  1 |  0 |  0 |  0 |  0 |\n|  1 |  0 |  0 |  1 |  1 |\n|"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "Karnaugh map for the truth table provided: | A1 | A0 | B1 | B0 | P1 |\n|----|----|----|----|----|\n|  0 |  0 |  0 |  0 |  0 |\n|  0 |  0 |  0 |  1 |  0 |\n|  0 |  0 |  1 |  0 |  0 |\n|  0 |  0 |  1 |  1 |  0 |\n|  0 |  1 |  0

## 1.8 Esercizio con few-shot prompting

In [24]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askGemini(prompt, generation_config1)

--------Response--------
Input: | A | B | C | D | P |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 0 |

Output: {"Karnaugh Map":
"        CD
         00  01  11  10
        ----------------
AB  00 |  0   0   0   0
    01 |  0   0   1   1
    11 |  0   1   0   1
    10 |  0   1   1   0 ",
 "Sum-of-products expression":
 "f = BC!D + A!BD + ABD",
 "Explanation":
 "We can group the cells containing ones into three prime implicants.
  The first group contains the two cells (0110, 0111). Since the cells contain a one when B is one, C is one, and D is zero, the first term is equal to BC!D.
  The second group contains the two cells (1001, 1011). Since the cells contain a one

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [25]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
```json
{"Karnaugh Map":
"        B1B0
         00  01  11  10
        ----------------
A1A0  00 |  0   0   1   0
    01 |  0   0   0   0
    11 |  0   1   1   1
    10 |  0   1   0   1 ",
 "Sum-of-products expression":
 "P1 = A1!B1B0 + !A1B1!B0 + A0B1B0",
 "Explanation":
 "We can group the cells containing ones into three prime implicants.
  The first group contains the two cells (1101, 1111). Since the cells contain a one when A1 is one, B1 is one and B0 is one, the first term is equal to A1B1B0.
  The second group contains the two cells (1001, 0001). Since the cells contain a one when A1 is zero, B1 is one and B0 is zero, the second term is equal to !A1B1!B0.
  The third group contains the two cells (1111, 1011). Since the cells contain a one when A0 is one, B1 is one and B0 is one, the third term is equal to A0B1B0."}
```

# Esercizio 6

## 1.0 Definizione del prompt

In [ ]:
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video6>.mp4")

input = """| X | Y | Z | T | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 1 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 1 |
"""

prompt_text = f""" Karnaugh maps (K-maps) are used to simplify Boolean expressions. A K-map is a graphical representation of truth tables and helps determine the minimal form of a Boolean function.
Create a Karnaugh map using the truth table provided: {input}.
Determine the minimal form of the represented Boolean function."""

## 1.1 Esercizio senza ulteriore contesto

In [10]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askGemini(prompt, generation_config1)

--------Response--------
## Karnaugh Map and Minimal Form

Here's the Karnaugh map for the given truth table, followed by the simplification process:

**1. K-Map Construction:**

We'll use a 4-variable K-map with the following layout:

```
     YZ
  \  00  01  11  10 
 XT \----------------
 00  | 1   1   0   1 |
     |----------------|
 01  | 0   0   1   0 |
     |----------------|
 11  | 1   1   1   1 |
     |----------------|
 10  | 1   1   1   1 |
     -----------------
```

**2. Grouping the 1s:**

We group the 1s in the K-map following these rules:

* Groups must be rectangles and contain 1, 2, 4, 8, ... cells (powers of 2).
* Groups should be as large as possible.
* Groups can wrap around the edges of the K-map.

Following these rules, we get the following groups:

```
     YZ
  \  00  01  11  10 
 XT \----------------
 00  | 1   1   0   1 |
     |----------------|
 01  | 0   0   1   0 |
     |----------------|
 11  | 1   1   1   1 |
     |----------------|
 10  | 1   1   1   1 |

## 1.2 Esercizio con recupero diretto

In [11]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askGemini(prompt, generation_config1)

--------Response--------
```
XYZT | 00 | 01 | 11 | 10
-------|----|----|----|----
**00**  | 1  |  1 |  1 |  1 
**01**  | 0  |  0 |  1 |  0
**11**  | 1  |  1 |  1 |  1
**10**  | 1  |  1 |  1 |  1 

```

**Looping and Simplification:**

1. **Red Loop (Largest):** This loop covers all the 1s in the top and bottom rows, encompassing eight squares. This represents the term  `X'`. 
2. **Blue Loop:** This loop covers the two 1s on the right side of the second and third rows. This represents the term `YZ`.

**Minimal SOP Form:**

The minimal Sum-of-Products (SOP) form of the Boolean function is the OR of the terms represented by the loops:

**F = X' + YZ** 


## 1.3 Esercizio con recupero diretto e video soluzione

In [12]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
The Karnaugh map for the provided truth table is:

```
     YZ
XT  00 01 11 10
00   1  0  1  1 
01   1  0  1  0
11   1  1  1  1
10   1  1  1  1

```

**Looping and Simplification:**

1. **Largest Group:** We can create a group of eight (octet) that covers all the '1's in the bottom two rows (XT = 11 and XT = 10). This octet eliminates three variables (Y, Z, and T), leaving us with just **X**.

2. **Remaining '1's:** The remaining '1's in the top row can be grouped into two pairs:
    * A pair covering XT = 00, YZ = 00 and 01. This eliminates Z, leaving us with **¬Y¬T**.
    * A pair covering XT = 00, YZ = 11 and 10. This eliminates Y, leaving us with **¬Z¬T**.

**Minimal SOP Expression:**

Combining the terms from the loops, we get the minimal sum-of-products (SOP) expression:

**F = X + ¬Y¬T + ¬Z¬T** 


## 1.4 Esercizio con video soluzione

In [15]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
The video explains how to fill a Karnaugh map from a truth table and then how to extract the minimal form of the boolean function.

The Karnaugh map is filled by writing the value of the function in the cell corresponding to the row of the truth table. For example, the cell corresponding to the row where X=1, Y=1, Z=0, T=1 is filled with a 1, because the value of the function in that row is 1.

Once the Karnaugh map is filled, the minimal form of the boolean function is extracted by grouping the cells containing a 1. The groups must be as large as possible and must contain a number of cells that is a power of 2 (2, 4, 8, etc.).

In the example, the minimal form of the boolean function is:

```
F = X + ¬Y¬T + ¬YZ¬T + YZT
```

This is because the Karnaugh map contains four groups of 1s:

* A group of 8 cells where X=1.
* A group of 4 cells where Y=0 and T=0.
* A group of 4 cells where Y=0 and Z=0.
* A group of 2 cells where Y=1, Z=1 and T=1.

Each group correspon

## 1.5 Esercizio con libro di testo nel prompt

In [16]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook)

--------Response--------
## Karnaugh Map and Minimal Form

Here's the Karnaugh map for the provided truth table:

```
YZ
  \  00  01  11  10
XT \-----------------
00 |  1   0   1   1 
01 |  1   0   0   1
11 |  1   1   1   1
10 |  1   1   1   1
```

**Explanation of K-map construction:**

1. **Variables:** The K-map is arranged with input variables X and T on the vertical axis and Y and Z on the horizontal axis.
2. **Cell Values:** Each cell in the K-map corresponds to a unique combination of input variables. The cell value is taken directly from the 'F' column of the truth table for that specific input combination.

**Minimal Form:**

By analyzing the K-map, we can identify the following groupings:

* **Group 1:** A large group of eight '1's covering the bottom two rows (XT = 11 and 10). This group represents the term **X**.
* **Group 2:** A group of four '1's covering the top-left corner (YZ = 00). This group represents the term **Y'Z'**.
* **Group 3:** A group of four '1's covering t

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [17]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
The Karnaugh map for the provided truth table is:

```
    YZ
XT  00 01 11 10
00  1  0  1  1
01  1  0  1  0
11  1  1  1  1
10  1  1  1  1
```

The minimal form of the Boolean function is: **X + Z**.

**Explanation:**

1. **Identify the largest groups of '1's:** There are two major groups of '1's in the K-map:
    * All cells where X = 1
    * All cells where Z = 1

2. **Determine the Boolean expression for each group:**
    * **X = 1 group:**  Since X is the only variable that remains constant within this group, the expression is simply **X**.
    * **Z = 1 group:** Similarly, Z is the only constant variable within this group, resulting in the expression **Z**.

3. **Combine the expressions using OR:** Since either group being true results in F = 1, we combine the expressions using OR: **X + Z**.

This minimal form represents the simplest Boolean expression that satisfies the given truth table. 


## 1.7 Esercizio con recupero mediante tool

In [20]:
responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

```
YZ
  \  00  01  11  10
XT \----------------
  00|  1   1   0   1
  01|  1   0   1   0
  11|  1   1   1   1
  10|  1   1   1   1
```

The minimal form of the Boolean function is **F = X + Z + Y'T'**. 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "Karnaugh map for the truth table provided."
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "Karnaugh map for the truth table provided."
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [18]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askGemini(prompt, generation_config1)

--------Response--------
Input: | W | X | Y | Z | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 1 |
| 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 | 1 |

Output: {"Karnaugh Map":
"        YZ
         00  01  11  10
        ----------------
WX  00 |  1   1   0   0
    01 |  1   1   1   0
    11 |  1   1   1   0
    10 |  1   1   1   0 ",
 "Sum-of-products expression":
 "F = !Y + WXZ + !WXZ",
 "Explanation":
 "We can group the cells containing ones into three prime implicants.
  The first group contains the eight cells (0000, 0001, 0100, 0101, 1000, 1001, 1100, 1101). Since the cells contain a one when Y is zero, the first term is equal to !Y.
  The second group contains the two cells (0111, 1111). Since the cells con

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [19]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
```json
{"Karnaugh Map":
"        TY
         00  01  11  10
        ----------------
ZX  00 |  1   1   1   1
    01 |  1   0   1   1
    11 |  1   1   1   1
    10 |  1   1   1   1 ",
 "Sum-of-products expression":
 "F = X + !YZ + T",
 "Explanation":
 "We can group the cells containing ones into three prime implicants.
  The first group contains the eight cells (1000, 1001, 1010, 1011, 1100, 1101, 1110, 1111). Since the cells contain a one when X is one, the first term is equal to X.
  The second group contains the four cells (0000, 0001, 0100, 0101). Since the cells contain a one when Y is zero and Z is zero, the second term is equal to !YZ.
  The third group contains the eight cells (0010, 0011, 0110, 0111, 1010, 1011, 1110, 1111). Since the cells contain a one when T is one, the third term is equal to T."}
```

# Esercizio 7

## 1.0 Definizione del prompt

In [ ]:
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video7>.mp4")

input = """| A1 | A0 | B1 | B0 | G |
|----|----|----|----|---|
|  0 |  0 |  0 |  0 | 0 |
|  0 |  0 |  0 |  1 | 0 |
|  0 |  0 |  1 |  0 | 0 |
|  0 |  0 |  1 |  1 | 0 |
|  0 |  1 |  0 |  0 | 1 |
|  0 |  1 |  0 |  1 | 0 |
|  0 |  1 |  1 |  0 | 0 |
|  0 |  1 |  1 |  1 | 0 |
|  1 |  0 |  0 |  0 | 1 |
|  1 |  0 |  0 |  1 | 1 |
|  1 |  0 |  1 |  0 | 0 |
|  1 |  0 |  1 |  1 | 0 |
|  1 |  1 |  0 |  0 | 1 |
|  1 |  1 |  0 |  1 | 1 |
|  1 |  1 |  1 |  0 | 1 |
|  1 |  1 |  1 |  1 | 0 |
"""

prompt_text = f""" Karnaugh maps (K-maps) are used to simplify Boolean expressions. A K-map is a graphical representation of truth tables and helps determine the minimal form of a Boolean function.
Create a Karnaugh map using the truth table provided: {input}.
Determine the minimal form of the represented Boolean function."""

## 1.1 Esercizio senza ulteriore contesto

In [38]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askGemini(prompt, generation_config1)

--------Response--------
## Karnaugh Map and Minimal Form:

Here's the Karnaugh map based on your provided truth table:

```
             B1B0
       00  01  11  10
A1A0 +---+---+---+---+
 00  | 0 | 0 | 0 | 0 |
     +---+---+---+---+
 01  | 1 | 0 | 0 | 0 |
     +---+---+---+---+
 11  | 1 | 1 | 0 | 1 |
     +---+---+---+---+
 10  | 1 | 1 | 0 | 0 |
     +---+---+---+---+
```

Now, let's find the minimal form by grouping the '1's:

1. **Group 1:** We can group the four '1's forming a square in the bottom-left corner:
   - This group covers A1A0 = 11, 10 and B1B0 = 00, 01.
   - This simplifies to **A1**.

2. **Group 2:** We can group the two '1's on the top row of the third column:
   - This group covers A1A0 = 01, 11 and B1B0 = 00.
   - This simplifies to **A0B1'B0'**.

Therefore, the minimal form of the Boolean function represented by the Karnaugh map is:

**G = A1 + A0B1'B0'** 


## 1.2 Esercizio con recupero diretto

In [39]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askGemini(prompt, generation_config1)

--------Response--------
```
A1A0\B1B0 | 00  | 01  | 11 | 10 
-------|----|----|----|----
    00   | 0  | 0  | 0  | 0
    01   | 1  | 0  | 0  | 0
    11   | 1  | 1  | 0  | 1
    10   | 1  | 1  | 0  | 0 
```

**Looping and Simplification:**

1. **Largest Loop:**  Notice the group of four 1s forming a square in the center. This represents the term  `A1`.

2. **Remaining 1:** The isolated '1' at the top of the second column (A1A0 = 01, B1B0 = 00) can be combined with the '1' below it to form a group of two. This group represents the term `~A1 * ~B1 * ~B0`.

**Minimal SOP Expression:**

Combining the terms from the loops, the minimal SOP form of the Boolean function is:

**G = A1 + (~A1 * ~B1 * ~B0)** 


## 1.3 Esercizio con recupero diretto e video soluzione

In [40]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
```
 A1A0\B1B0 | 00  | 01  | 11  | 10
----------|-----|-----|-----|-----
      00   |  0  |  0  |  0  |  0
      01   |  1  |  0  |  0  |  0
      11   |  1  |  1  |  0  |  1
      10   |  1  |  1  |  0  |  0 
```

**Minimal Form:**

The Karnaugh map reveals the following groupings:

* **Green Group:** This group of four represents `A1¬B1`.
* **Red Group:** This group of two represents `A0¬B1¬B0`.
* **Blue Group:** This group of two represents `A0A1B0`.

Therefore, the minimal form of the Boolean function G is:

**G = A1¬B1 + A0¬B1¬B0 + A0A1B0** 


## 1.4 Esercizio con video soluzione

In [41]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
The video shows how to fill a Karnaugh map using the truth table provided and determine the minimal form of the represented Boolean function.

First, the speaker explains how to map the truth table values to the corresponding cells in the Karnaugh map. The rows of the truth table correspond to the cells of the Karnaugh map. For example, the first row of the truth table, "0000", corresponds to the cell in the Karnaugh map with coordinates A1=0, A0=0, B1=0, B0=0. The value of G for this combination is 0, so a 0 is written in that cell. The speaker repeats this process for all 16 rows of the truth table, filling all the cells of the Karnaugh map.

Next, the speaker explains how to identify the groups of ones in the Karnaugh map. The goal is to find the largest possible groups of adjacent cells containing ones. The size of each group must be a power of two (1, 2, 4, 8, etc.). In this example, the largest group is a group of four cells in the center of the map. This

## 1.5 Esercizio con libro di testo nel prompt

In [42]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook)

--------Response--------
## Karnaugh Map and Minimal Form

Here's the Karnaugh map for the given truth table:

```
      B1B0
 A1A0  00  01  11  10
-----------------------
 00    0   0   0   0
 01    1   0   0   0 
 11    1   1   0   1
 10    1   1   0   0
```

To find the minimal form, we look for the largest possible groupings of '1's that are powers of 2 (1, 2, 4, 8, etc.).  We can wrap around the edges of the map.

Here's the optimal grouping:

```
      B1B0
 A1A0  00  01  11  10
-----------------------
 00    0   0   0   0
 01    1   0   0   0 
 11    1   1   0   1
 10    1   1   0   0
       \_______/|
         |       |
         \_______/
```

* **Group 1:** Covers the four '1's in the bottom left corner. This group represents A1.
* **Group 2:** Covers the two '1's in the top right of the A1A0 = 11 row. This group represents A1A0B0.

Therefore, the minimal form of the Boolean function is:

**G = A1 + A1A0B0** 


## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [43]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
The Karnaugh map for the provided truth table is:

```
B1B0
  \ A1A0 00 01 11 10
   +------+------+------+------+
  0|  0   |  1   |  1   |  1   |
   +------+------+------+------+
  1|  0   |  0   |  0   |  1   |
   +------+------+------+------+
```

From this K-map, we can derive the minimal form of the Boolean function by identifying the largest groupings of '1's. There are three such groupings:

1. **Top row:** This grouping covers all four cells in the top row where `A1 = 0`. The variables `B1` and `B0` change within the group, so they are eliminated. The resulting term is **`A1'`**.

2. **Bottom right corner:** This grouping covers the single cell where `A1 = 1`, `A0 = 0`, `B1 = 1`, and `B0 = 0`. The resulting term is **`A1 A0' B1 B0'`**.

Therefore, the minimal form of the Boolean function is:

**`G = A1' + A1 A0' B1 B0'`** 


## 1.7 Esercizio con recupero mediante tool

In [46]:
responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

The Karnaugh map for the provided truth table is:

```
     B1B0
A1A0 00 01 10 11
------------------
 00 | 0  0  0  0
 01 | 1  0  0  0
 10 | 1  1  0  0
 11 | 1  1  1  0
```

The minimal form of the Boolean function is **G = A1 + A0B0'**. 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "Karnaugh map for the truth table provided."
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "Karnaugh map for the truth table provided."
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [44]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askGemini(prompt, generation_config1)

--------Response--------
Input: | A | B | C | D | G |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 1 |
| 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 | 1 |

Output: {"Karnaugh Map":
"        CD
         00  01  11  10
        ----------------
AB  00 |  1   1   0   0
    01 |  1   1   1   0
    11 |  0   1   1   0
    10 |  0   0   0   0 ",
 "Sum-of-products expression":
 "f = !AB!C + !AD + BCD",
 "Explanation":
 "We can group the cells containing ones into three prime implicants.
  The first group contains the four cells (0000, 0001, 0100, 0101). Since the cells contain a one when A is zero, B is zero and C is zero, the first term is equal to !AB!C.
  The second group contains the two cells (0001, 0101). Since the cell

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [45]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
```json
{"Karnaugh Map":
"        B1B0
         00  01  11  10
        ----------------
A1A0  00 |  0   1   0   0
    01 |  1   1   1   0
    11 |  1   1   0   1
    10 |  1   1   1   1 ",
 "Sum-of-products expression":
 "G = A1!B1 + A1A0 + !B0A0 + B1!A0B0",
 "Explanation":
 "We can group the cells containing ones into four prime implicants.
  The first group contains the eight cells (1000, 1001, 1100, 1101, 1110, 1111, 1010, 1011). Since the cells contain a one when A1 is one and B1 is zero, the first term is equal to A1!B1.
  The second group contains the four cells (1100, 1101, 1110, 1111). Since the cells contain a one when A1 is one and A0 is one, the second term is equal to A1A0.
  The third group contains the four cells (0100, 1100, 1000, 1010). Since the cells contain a one when B0 is zero and A0 is one, the third term is equal to !B0A0.
  The fourth group contains the two cells (0101, 0111). Since the cells contain a one when B1 is one, A0 is zero, and

# Esercizio 8

## 1.0 Definizione del prompt

In [ ]:
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video8>.mp4")

input = """| A1 | A0 | B1 | B0 | E |
|----|----|----|----|---|
|  0 |  0 |  0 |  0 | 1 |
|  0 |  0 |  0 |  1 | 0 |
|  0 |  0 |  1 |  0 | 0 |
|  0 |  0 |  1 |  1 | 0 |
|  0 |  1 |  0 |  0 | 0 |
|  0 |  1 |  0 |  1 | 1 |
|  0 |  1 |  1 |  0 | 0 |
|  0 |  1 |  1 |  1 | 0 |
|  1 |  0 |  0 |  0 | 0 |
|  1 |  0 |  0 |  1 | 0 |
|  1 |  0 |  1 |  0 | 1 |
|  1 |  0 |  1 |  1 | 0 |
|  1 |  1 |  0 |  0 | 0 |
|  1 |  1 |  0 |  1 | 0 |
|  1 |  1 |  1 |  0 | 0 |
|  1 |  1 |  1 |  1 | 1 |
"""

prompt_text = f""" Karnaugh maps (K-maps) are used to simplify Boolean expressions. A K-map is a graphical representation of truth tables and helps determine the minimal form of a Boolean function.
Create a Karnaugh map using the truth table provided: {input}.
Determine the minimal form of the represented Boolean function."""

## 1.1 Esercizio senza ulteriore contesto

In [48]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askGemini(prompt, generation_config1)

--------Response--------
## Karnaugh Map and Minimal Form:

Here's the Karnaugh map for the given truth table:

```
             B1B0
       A1A0  00  01  11  10
          ----------------
       00  | 1 |  0 |  0 |  0 |
          ----------------
       01  | 0 |  1 |  0 |  0 |
          ----------------
       11  | 0 |  0 |  1 |  0 |
          ----------------
       10  | 0 |  0 |  0 |  1 |
          ----------------
```

**Explanation:**

1. **Arrangement:** The K-map is arranged with the input variables A1A0 and B1B0. Each cell represents a unique combination of these inputs.
2. **Filling the Map:** The 'E' values from the truth table are placed in their corresponding cells based on the input combinations.

**Finding the Minimal Form:**

We can now group the '1's in the K-map to simplify the Boolean expression. Here's how:

* **Group 1:** The lone '1' in the top-left corner (A1A0=00, B1B0=00) cannot be grouped with any other '1'. This represents the term **A1'A0'B1'B0'**.
* **Gro

## 1.2 Esercizio con recupero diretto

In [49]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askGemini(prompt, generation_config1)

--------Response--------
```
A1A0\B1B0 | 00  | 01  | 11  | 10 
-------|-----|-----|-----|-----
   00    |  1  |  0  |  0  |  0  
   01    |  0  |  1  |  0  |  0  
   11    |  0  |  0  |  1  |  0  
   10    |  0  |  0  |  0  |  1  
```

**Looping and Simplification**

1. **Isolated 1s:** There are four isolated 1s, which cannot be combined with any others. These will be represented as individual product terms.

2. **Pairs, Quads, Octets:** No other groupings are possible.

**Forming the Minimal SOP Expression**

Reading the isolated 1s from the K-map, we get the following minimal SOP expression:

**E =  A1'A0'B1'B0' + A1'A0B1'B0 + A1A0B1B0 + A1A0'B1B0'** 


## 1.3 Esercizio con recupero diretto e video soluzione

In [50]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
```
E = A1' * A0 * B1' * B0' + A1 * A0' * B1 * B0' + A0 * B1' * B0 + A1' * A0' * B1' * B0
```

**Explanation:**

1. **Construct the K-map:** A 4-variable K-map is created with A1 and A0 representing the rows and B1 and B0 representing the columns.

   ```
   B1 B0
   A1A0 00  01  11  10
   00   1   0   0   0
   01   0   1   0   0
   11   0   0   1   0
   10   0   0   0   1
   ```

2. **Fill the K-map:** The 'E' values from the truth table are placed in the corresponding cells of the K-map.

3. **Identify the groups:** The goal is to group adjacent '1' cells in the largest possible groups of powers of 2 (1, 2, 4, 8, etc.).  The following groups are identified:

   * **Red:** A single cell at A1=1, A0=0, B1=1, B0=0. This corresponds to the term `A1 * A0' * B1 * B0'`.

   * **Blue:** A single cell at A1=0, A0=1, B1=0, B0=1. This corresponds to the term `A1' * A0 * B1' * B0`.

   * **Green:** A single cell at A1=1, A0=1, B1=1, B0=1. This corresponds to the term `A1

## 1.4 Esercizio con video soluzione

In [57]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
The user is requesting a Karnaugh map and the minimal form of the Boolean function represented by a provided truth table. 

The video provided shows the step-by-step creation of the Karnaugh map based on the truth table. The map is a 4x4 grid, with the rows representing the values of B1 and B0, and the columns representing the values of A1 and A0. Each cell in the Karnaugh map corresponds to a row in the truth table. The speaker fills the cells with 1s and 0s based on the output value (E) in the corresponding truth table row.

After filling the Karnaugh map, the speaker identifies four groups of 1s:
* A single cell with combination 1010
* A single cell with combination 1111
* A single cell with combination 0101
* A single cell with combination 0000

Each of these groups represents an essential prime implicant for the Boolean function. The speaker then writes down the Boolean expression for each group:

*  ¬A1 A0 ¬B1 B0
*  A1 A0 B1 B0
*  ¬A1 A0 B1 ¬B0
*  ¬A1 ¬A0

## 1.5 Esercizio con libro di testo nel prompt

In [52]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook)

--------Response--------
## Karnaugh Map and Minimal Form

Here's the Karnaugh map for the given truth table:

```
B1B0
  00 01 11 10
A1A0 +---+---+---+---+
 00  | 1 | 0 | 0 | 0 |
 +---+---+---+---+
 01  | 0 | 1 | 0 | 0 |
 +---+---+---+---+
 11  | 0 | 0 | 1 | 0 |
 +---+---+---+---+
 10  | 0 | 0 | 0 | 1 |
 +---+---+---+---+
```

**Minimal Form:**

By analyzing the Karnaugh map, we can identify the following prime implicants:

* **A1'A0'B1'B0'**:  This covers the '1' at (A1A0 = 00, B1B0 = 00).
* **A1'A0B1'B0**: This covers the '1' at (A1A0 = 01, B1B0 = 01).
* **A1A0B1B0**: This covers the '1' at (A1A0 = 11, B1B0 = 11).
* **A1A0'B1B0'**: This covers the '1' at (A1A0 = 10, B1B0 = 10).

All these prime implicants are essential, as they each cover a '1' that no other prime implicant covers.

Therefore, the minimal form of the Boolean function is:

**E = A1'A0'B1'B0' + A1'A0B1'B0 + A1A0B1B0 + A1A0'B1B0'** 


## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [53]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
The Karnaugh map for the provided truth table is:

```
B1B0
  \ A1A0 00 01 11 10
   +------+------+------+------+
  0|  1   |  0   |  0   |  0   |
   +------+------+------+------+
  1|  0   |  1   |  1   |  1   |
   +------+------+------+------+
```

The minimal form of the Boolean function is: **E = A1'B0' + A1B1'B0 + A1B1B0**

**Explanation:**

1. **Identify the '1's in the K-map.** The '1's represent the combinations of inputs where the output E is true.
2. **Group adjacent '1's.** Groups should be as large as possible and contain a power of 2 number of cells (1, 2, 4, 8, etc.). In this case, we have three groups:
    * A group of one cell containing the '1' at A1'A0'B1'B0'.
    * A group of two cells containing the '1's at A1A0B1'B0 and A1A0B1B0.
    * A group of one cell containing the '1' at A1A0B1B0'.
3. **Write the Boolean expression for each group.** For each group, determine the variables that remain constant within the group.
    * Group 1: A1'B0' (A

## 1.7 Esercizio con recupero mediante tool

In [56]:
responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

The Karnaugh map for the provided truth table is:

```
     B1B0
A1A0  00  01  11  10
---------------------
 00 |  1   0   0   0
 01 |  0   1   0   0
 11 |  0   0   1   0
 10 |  0   0   0   1
```

From this K-map, the minimal form of the Boolean function can be derived as:  **E = A1'A0'B1'B0' + A1'A0B1'B0 + A1A0B1B0 + A1A0'B1B0'** 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "Karnaugh map for the truth table provided."
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "Karnaugh map for the truth table provided."
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [54]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askGemini(prompt, generation_config1)

--------Response--------
Input: | A | B | C | D | E |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 1 |
| 0 | 0 | 1 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 | 1 |

Output: {"Karnaugh Map":
"        CD
         00  01  11  10
        ----------------
AB  00 |  0   0   1   1
    01 |  0   1   1   0
    11 |  1   1   1   0
    10 |  0   0   0   0 ",
 "Sum-of-products expression":
 "f = AB!C + !AD + BCD",
 "Explanation":
 "We can group the cells containing ones into three prime implicants.
  The first group contains the four cells (0010, 0011, 0110, 0111). Since the cells contain a one when A is zero and D is one, the first term is equal to !AD.
  The second group contains the four cells (1100, 1101, 0101, 0011). Since the cells 

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [55]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on Karnaugh maps, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
```json
{"Karnaugh Map":
"        B1B0
         00  01  11  10
        ----------------
A1A0  00 |  1   0   0   1
    01 |  1   1   0   0
    11 |  0   1   1   0
    10 |  0   0   0   1 ",
 "Sum-of-products expression":
 "E = !A1!A0!B1B0 + !A1A0B0 + A1A0B1 + A1!A0!B0",
 "Explanation":
 "We can group the cells containing ones into four prime implicants.
  The first group contains the cell (0000). Since the cell contains a one when A1 is zero, A0 is zero, B1 is zero, and B0 is zero, the first term is equal to !A1!A0!B1B0.
  The second group contains the two cells (0101, 0111). Since the cells contain a one when A1 is zero, A0 is one, and B0 is one, the second term is equal to !A1A0B0.
  The third group contains the two cells (1111, 1110). Since the cells contain a one when A1 is one, A0 is one, and B1 is one, the third term is equal to A1A0B1.
  The fourth group contains the cell (1010). Since the cell contains a one when A1 is one, A0 is zero, B1 is one, and B0 

# Esempi e contesto per la categoria 3: Reti sequenziali

In [ ]:
# Immagini degli esempi per il few-shot prompting
example_image1 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image1-cartegory3>.png")
example_image2 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image2-cartegory3>.png")
example_image3 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image3-cartegory3>.png")

# Descrizione delle immagini degli esempi per il few-shot prompting
example_image1_description = """The sequential network is synchronous (all flip-flops share the same clock) and has three outputs (OUT1, OUT2, OUT3). The structure of the network is described as follows:

* **First JK-pet flip-flop:**
    * Receives the common clock signal (CK) for all flip-flops.
    * The J and K inputs are both connected to the EN input.
    * The output (Q) is connected to one of the two inputs of the first AND gate. The output (Q) is the output of the network OUT1.
    * The negated output (-Q) is not connected to anything.
    * The CLEAR input signal is negated and then connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **Second JK-pet flip-flop:**
    * Receives the common clock signal (CK).
    * The J and K inputs are both connected to the output of the first AND gate.
    * The output (Q) is connected to one of the two inputs of the second AND gate. The output (Q) is the output of the network OUT2.
    * The negated output (-Q) is not connected to anything.
    * The CLEAR input signal is negated and then connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **Third JK-pet flip-flop:**
    * Receives the common clock signal (CK).
    * The J and K inputs are both connected to the output of the second AND gate.
    * The output (Q) is the output of the network OUT3.
    * The negated output (-Q) is not connected to anything.
    * The CLEAR input signal is negated and then connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **First AND gate:**
    * Its two inputs are connected to:
        * The Q output of the first JK flip-flop, called OUT1.
        * The EN signal.
    * The output of the first AND gate goes to the second JK flip-flop and to the second AND gate.
* **Second AND gate:**
    * Its two inputs are connected to:
        * The Q output of the second JK flip-flop, called OUT2.
        * The output of the first AND gate.
    * The output of the second AND gate goes to the third JK flip-flop.
* **NOT gate:**
    * Its input is the CLEAR signal.
    * The output of the NOT gate is connected to the -CLEAR of all three flip-flops.

**Control signals:**

* **CK (Clock):** Determines the timing of the circuit, activating the flip-flops on each rising edge.
* **CLEAR (Clear):** This is an active-high signal. When active (1), it forces the outputs of all flip-flops to zero (Q=0). 
* **EN (Enable):** This is a signal that enables the first JK flip-flop and the first AND gate.
* ** '1' (high):** This is a constant signal at 1.
"""
example_image2_description = """The sequential network is synchronous (all flip-flops share the same clock) and has three outputs (OUT1, OUT2, OUT3). The structure of the network is described as follows:

* **First E-pet flip-flop:**
    * Receives the common clock signal (CK) for all flip-flops.
    * The D input is connected to the output of the second EXOR gate.
    * The output (Q) is connected to the D input of the second E-pet flip-flop and to the first input of the NOR gate. The output (Q) is the output of the network OUT3.
    * The negated output (-Q) is not connected to anything.
    * The -CLEAR input signal is connected to the -CLEAR input of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
    * The EN input signal is connected to the E input of the flip-flop.
* **Second E-pet flip-flop:**
    * Receives the common clock signal (CK).
    * The D input is connected to the Q output of the first E-pet flip-flop.
    * The output (Q) is connected to the D input of the third E-pet flip-flop, to the second input of the NOR gate, and to the first input of the first EXOR gate. The output (Q) is the output of the network OUT2.
    * The negated output (-Q) is not connected to anything.
    * The -CLEAR input signal is connected to the -CLEAR input of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
    * The EN input signal is connected to the E input of the flip-flop.
* **Third E-pet flip-flop:**
    * Receives the common clock signal (CK).
    * The D input is connected to the Q output of the second E-pet flip-flop.
    * The output (Q) is connected to the second input of the first EXOR gate. The output (Q) is the output of the network OUT1.
    * The negated output (-Q) is not connected to anything.
    * The -CLEAR input signal is connected to the -CLEAR input of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
    * The EN input signal is connected to the E input of the flip-flop.
* **NOR gate:**
    * Its two inputs are connected to:
        * The Q output of the first E-pet flip-flop, called OUT3.
        * The Q output of the second E-pet flip-flop, called OUT2.
    * The output of the NOR gate goes to the second EXOR gate.
* **First EXOR gate:**
    * Its two inputs are connected to:
        * The Q output of the second E-pet flip-flop, called OUT2.
        * The Q output of the third E-pet flip-flop, called OUT1.
    * The output of the first EXOR gate goes to the second EXOR gate.
* **Second EXOR gate:**
    * Its two inputs are connected to:
        * The output of the NOR gate.
        * The output of the first EXOR gate.
    * The output of the second EXOR gate goes to the first E-pet flip-flop.

**Control signals:**

* **CK (Clock):** Determines the timing of the circuit, activating the flip-flops on each rising edge.
* **CLEAR (Clear):** This is an active-low signal. When active (0), it forces all the flip-flop outputs to zero (Q=0).
* **EN (Enable):** This is a signal that enables the E-pet flip-flops when active. When inactive, it forces the flip-flops to maintain their previous state.
* ** '1' (high):** This is a constant signal at 1.
"""
example_image3_description ="""The sequential network is synchronous (all flip-flops share the same clock) and has two outputs (OUT1, OUT2). 
The structure of the network is described as follows:

* **D-pet Flip-Flop:**
    * Receives the common clock signal (CK).
    * The D input is connected to the output of the EXOR gate.
    * The output (Q) is connected to the first of three inputs of the EXOR gate. The output (Q) is the output of the network OUT1.
    * The negated output (-Q) is not connected to anything.
    * The input signal -CLEAR is connected to the -CLEAR input of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **E-pet Flip-Flop:**
    * Receives the common clock signal (CK).
    * The D input is connected to the negated output (-Q) of the same E-pet flip-flop.
    * The output (Q) is connected to the third of three inputs of the EXOR gate. The output (Q) is the output of the network OUT2.
    * The negated output (-Q) is connected to the D input of the same flip-flop.
    * The input signal -CLEAR is connected to the -CLEAR input of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
    * The input signal EN is connected to the E input of the flip-flop.
* **EXOR Gate:**
    * Its three inputs are connected respectively to:
        * Output (Q) of the D-pet flip-flop called OUT1.
        * Input signal IN.
        * Output (Q) of the E-pet flip-flop called OUT2.
    * The output of the EXOR gate enters as input to the D-pet flip-flop.

**Control Signals:**

* **CK (Clock):** Determines the timing of the circuit, activating the flip-flops on each rising edge.
* **CLEAR (Clear):** This is an active-low signal. When active (0), it forces all the flip-flop outputs to zero (Q=0)
* **EN (Enable):** This is a signal that enables the E-pet flip-flops when active. When inactive, it forces the flip-flops to maintain the previous state.
* **IN (Input):** This is an input signal connected to the second input of the EXOR gate.
* **'1' (high):** This is a constant signal at 1.
"""

# Soluzioni degli esempi per il few-shot prompting
solution1 = """Edge 1: OUT1 is indeterminate, OUT2 is indeterminate, OUT3 is indeterminate.
Between Edge 1 and Edge 2: The CLEAR input goes from zero to one. When CLEAR switches, OUT1, OUT2, and OUT3 go from indeterminate to zero.
Edge 2: OUT1 is zero, OUT2 is zero, OUT3 is zero.
Between Edge 2 and Edge 3: The CLEAR input goes from one to zero.
Edge 3: OUT1 is zero, OUT2 is zero, OUT3 is zero.
Edge 4: OUT1 is zero, OUT2 is zero, OUT3 is zero.
Between Edge 4 and Edge 5: The EN input goes from zero to one.
Edge 5: OUT1 goes from zero to one shortly after the edge, OUT2 is zero, OUT3 is zero.
Edge 6: OUT1 goes from one to zero shortly after the edge, OUT2 goes from zero to one shortly after the edge, OUT3 is zero.
Edge 7: OUT1 goes from zero to one shortly after the edge, OUT2 is one, OUT3 is zero.
Edge 8: OUT1 goes from one to zero shortly after the edge, OUT2 goes from one to zero shortly after the edge, OUT3 goes from zero to one shortly after the edge.
Edge 9: OUT1 goes from zero to one shortly after the edge, OUT2 is zero, OUT3 is one.
Edge 10: OUT1 goes from one to zero shortly after the edge, OUT2 goes from zero to one shortly after the edge, OUT3 is one.
Edge 11: OUT1 goes from zero to one shortly after the edge, OUT2 is one, OUT3 is one.
Between Edge 11 and Edge 12: The EN input goes from one to zero.
Edge 12: OUT1 is one, OUT2 is one, OUT3 is one.
Edge 13: OUT1 is one, OUT2 is one, OUT3 is one.
Edge 14: OUT1 is one, OUT2 is one, OUT3 is one.
Edge 15: OUT1 is one, OUT2 is one, OUT3 is one.
Edge 16: OUT1 is one, OUT2 is one, OUT3 is one.
"""
solution2 = """Edge 1: OUT1 is indeterminate, OUT2 is indeterminate, OUT3 is indeterminate.
Between Edge 1 and Edge 2: The CLEAR input goes from one to zero. When CLEAR switches, OUT1, OUT2, and OUT3 go from indeterminate to zero.
Edge 2: OUT1 is zero, OUT2 is zero, OUT3 is zero.
Between Edge 2 and Edge 3: The CLEAR input goes from zero to one.
Edge 3: OUT1 is zero, OUT2 is zero, OUT3 is zero.
Between Edge 3 and Edge 4: The EN input goes from zero to one.
Edge 4: OUT1 is zero, OUT2 is zero, OUT3 goes from zero to one shortly after the edge.
Edge 5: OUT1 is zero, OUT2 goes from zero to one shortly after the edge, OUT3 goes from one to zero shortly after the edge.
Edge 6: OUT1 goes from zero to one shortly after the edge, OUT2 goes from one to zero shortly after the edge, OUT3 goes from zero to one shortly after the edge.
Edge 7: OUT1 goes from one to zero shortly after the edge, OUT2 goes from zero to one shortly after the edge, OUT3 is one.
Edge 8: OUT1 goes from zero to one shortly after the edge, OUT2 is one, OUT3 is one.
Edge 9: OUT1 is one, OUT2 is one, OUT3 goes from one to zero shortly after the edge.
Edge 10: OUT1 is one, OUT2 goes from one to zero shortly after the edge, OUT3 is zero.
Edge 11: OUT1 goes from one to zero shortly after the edge, OUT2 is zero, OUT3 is zero.
Edge 12: OUT1 is zero, OUT2 is zero, OUT3 goes from zero to one shortly after the edge.
Edge 13: OUT1 is zero, OUT2 goes from zero to one shortly after the edge, OUT3 goes from one to zero shortly after the edge.
Edge 14: OUT1 goes from zero to one shortly after the edge, OUT2 goes from one to zero shortly after the edge, OUT3 goes from zero to one shortly after the edge.
Edge 15: OUT1 goes from one to zero shortly after the edge, OUT2 goes from zero to one shortly after the edge, OUT3 is one.
Edge 16: OUT1 goes from zero to one shortly after the edge, OUT2 is one, OUT3 is one.
Between Edge 16 and Edge 17: The EN input goes from one to zero.
Edge 17: OUT1 is one, OUT2 is one, OUT3 is one.
"""
solution3 = """Edge 1: OUT1 is indeterminate, OUT2 is indeterminate.
Between Edge 1 and Edge 2: The CLEAR input goes from one to zero. When CLEAR switches, OUT1 and OUT2 go from indeterminate to zero. Then, the CLEAR input goes from zero to one.
Edge 2: OUT1 is zero, OUT2 is zero.
Between Edge 2 and Edge 3: The IN input goes from zero to one.
Edge 3: OUT1 goes from zero to one shortly after the edge, OUT2 is zero.
Edge 4: OUT1 goes from one to zero shortly after the edge, OUT2 is zero.
Between Edge 4 and Edge 5: The IN input goes from one to zero.
Edge 5: OUT1 is zero, OUT2 is zero.
Edge 6: OUT1 is zero, OUT2 is zero.
Between Edge 6 and Edge 7: The EN input goes from zero to one.
Edge 7: OUT1 is zero, OUT2 goes from zero to one shortly after the edge.
Edge 8: OUT1 goes from zero to one shortly after the edge, OUT2 goes from one to zero shortly after the edge.
Between Edge 8 and Edge 9: The IN input goes from zero to one.
Edge 9: OUT1 goes from one to zero shortly after the edge, OUT2 goes from zero to one shortly after the edge.
Edge 10: OUT1 is zero, OUT2 goes from one to zero shortly after the edge.
Edge 11: OUT1 goes from zero to one shortly after the edge, OUT2 goes from zero to one shortly after the edge.
Between Edge 11 and Edge 12: The IN input goes from one to zero.
Edge 12: OUT1 goes from one to zero shortly after the edge, OUT2 goes from one to zero shortly after the edge.
Edge 13: OUT1 is zero, OUT2 goes from zero to one shortly after the edge.
Edge 14: OUT1 goes from zero to one shortly after the edge, OUT2 goes from one to zero shortly after the edge.
Between Edge 14 and Edge 15: The EN input goes from one to zero.
Edge 15: OUT1 is one, OUT2 is zero.
Edge 16: OUT1 is one, OUT2 is zero.
"""

# Richietse degli esempi per il few-shot prompting
prompt_text1 = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on the previous ones.
A time diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.
Edge 1: CLEAR is zero and EN is zero.
Between Edge 1 and Edge 2: CLEAR goes from zero to one and EN is zero.
Edge 2: CLEAR is one and EN is zero.
Between Edge 2 and Edge 3: CLEAR goes from one to zero and EN is zero.
Edges 3-4: CLEAR is zero and EN is zero.
Between Edge 4 and Edge 5: CLEAR is zero and EN goes from zero to one.
Edges 5-11: CLEAR is zero and EN is one.
Between Edge 11 and Edge 12: CLEAR is zero and EN goes from one to zero.
Edges 12-16: CLEAR is zero and EN is one.
Given the above values for the CLEAR and EN inputs, perform the time analysis of the sequential network shown in {example_image1}, determining the value of each output at each clock edge (from 1 to 16).
"""
prompt_text2 = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on the previous ones.
A time diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.
Edge 1: -CLEAR is one and EN is zero.
Between Edge 1 and Edge 2: -CLEAR goes from one to zero and EN is zero.
Edge 2: -CLEAR is zero and EN is zero.
Between Edge 2 and Edge 3: -CLEAR goes from zero to one and EN is zero.
Edge 3: -CLEAR is one and EN is zero.
Between Edge 3 and Edge 4: -CLEAR is one and EN goes from zero to one.
Edges 4-16: -CLEAR is one and EN is one.
Between Edge 16 and Edge 17: -CLEAR is one and EN goes from one to zero.
Edge 17: -CLEAR is one and EN is zero.
Given the above values for the inputs -CLEAR and EN, perform the time analysis of the sequential network shown in {example_image2}, determining the value of each output at each clock edge (from 1 to 17).
"""
prompt_text3 = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on the previous ones.
A time diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.
Edge 1: -CLEAR is one, IN is zero, and EN is zero.
Between Edge 1 and Edge 2: -CLEAR goes from one to zero and then immediately from zero to one, IN is zero, and EN is zero.
Edge 2: -CLEAR is one, IN is zero, and EN is zero.
Between Edge 2 and Edge 3: -CLEAR is one, IN goes from zero to one, and EN is zero.
Edges 3-4: -CLEAR is one, IN is one, and EN is zero.
Between Edge 4 and Edge 5: -CLEAR is one, IN goes from one to zero, and EN is zero.
Edges 5-6: -CLEAR is one, IN is zero, and EN is zero.
Between Edge 6 and Edge 7: -CLEAR is one, IN is zero, and EN goes from zero to one.
Edges 7-8: -CLEAR is one, IN is zero, and EN is one.
Between Edge 8 and Edge 9: -CLEAR is one, IN goes from zero to one, and EN is one.
Edges 9-11: -CLEAR is one, IN is one, and EN is one.
Between Edge 11 and Edge 12: -CLEAR is one, IN goes from one to zero, and EN is one.
Edges 12-14: -CLEAR is one, IN is zero, and EN is one.
Between Edge 14 and Edge 15: -CLEAR is one, IN is zero, and EN goes from one to zero.
Edges 15-16: -CLEAR is one, IN is zero, and EN is zero.
Given the above values for the inputs -CLEAR, IN, and EN, perform the time analysis of the sequential network shown in {example_image3}, determining the value of each output at each clock edge (from 1 to 16).
"""

# Costruzione dei prompt degli esempi per il few-shot prompting
input1 = f'{{"Sequential network": "{example_image1}", "Task": "{prompt_text1}", "Network description": "{example_image1_description}"}}'
input2 = f'{{"Sequential network": "{example_image2}", "Task": "{prompt_text2}", "Network description": "{example_image2_description}"}}'
input3 = f'{{"Sequential network": "{example_image3}", "Task": "{prompt_text3}", "Network description": "{example_image3_description}"}}'

# Recupero esplicito delle informazioni sull'argomento (direct retrieval)
retrieval_text = """What is a sequential network?
How is a sequential network analyzed?
What are PET flip-flops and which ones are they?
What is a time diagram of a sequential network?
How is a time diagram of a sequential network constructed?"""

text_retrieved = retrieveInformationFromCustomDataSource(retrieval_text)

# Libro di testo sull'argomento
textbook = Part.from_uri(mime_type="application/pdf", uri="gs://reti-logiche-esercizi/Libro-capitoli-5-e-6 (en).pdf")

--------Text Retrieved--------


Logical gates along the clock path are also invalid.5.6 Edge-Triggered Flip-Flops 167
Thus, we make use of a different structure where the input of the D-PET flip-flop
is driven by a multiplexer, as in the figure below left. At right is the circuit symbol
of the E-PET flip-flop.
The multiplexer’s input E (“Enable”) allows us to copy the value of output Q to the
input of the D flip-flop if E = 0, or the value of the external input D if E = 1. When
the rising edge of Clock arrives, in the first case the memorized data stays the same.
In the second case, output Q takes on the value of external input D. Note that the
Clock is not affected.
See below the function table of the E-PET flip-flop with no initialization inputs.
E-PET Flip-flop
E D Clock Q Q
0 − Qp Qp Previous value
1 0 0 1 Memorizes 0
1 1 1 0 Memorizes 1
In the timing diagram below, we see the effect of input E, in response to the active
edges of the Clock. The rising edges of the Clock where the 

# Esercizio 11

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image11>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video11>.mp4")

image_description = """The sequential network is synchronous (all flip-flops share the same clock) and has four outputs (A, B, C, D). The structure of the network is described as follows:

* **D-PET Flip-Flop:**
    * Receives the common clock signal (CK) for all flip-flops.
    * Its data input (D) is connected to the output of the NAND gate.
    * The output (Q) is connected to the first input of the NAND gate and to the J input of the first JK-PET flip-flop.
    * The negated output (-Q) is connected to the K input of the first JK-PET flip-flop.
    * The -RES signal is connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **First JK-PET Flip-Flop:**
    * Receives the common clock signal (CK).
    * The J input is connected to the Q output of the D flip-flop. The K input is connected to the negated output (-Q) of the D flip-flop.
    * The output (Q) is connected to the second input of the NAND gate and to the J input of the second JK-PET flip-flop.
    * The negated output (-Q) is connected to the K input of the second JK-PET flip-flop.
    * The -RES signal is connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **Second JK-PET Flip-Flop:**
    * Receives the common clock signal (CK).
    * The J input is connected to the Q output of the first JK flip-flop. The K input is connected to the negated output (-Q) of the first JK flip-flop.
    * The output (Q) is connected to the third input of the NAND gate.
    * The negated output (-Q) is not connected to anything.
    * The -RES signal is connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **NAND Gate:**
    * Its four inputs are connected to:
        * The Q output of the D flip-flop, called A.
        * The Q output of the first JK flip-flop, called B.
        * The Q output of the second JK flip-flop, called C.
        * The EN signal.
    * The output of the NAND gate produces the signal D.

**Control Signals:**

* **CK (Clock):** Determines the timing of the circuit, activating the flip-flops on each rising edge.
* **-RES (Reset):** This is an active-low signal. When active (0), it forces the outputs of all flip-flops to zero (Q=0). 
* **EN (Enable):** Input to the NAND gate.
* ** '1' (high):** This is a constant signal at 1.
"""

prompt_text = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on the previous ones.
A time diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.

Edges 1-4: -RES is one and EN is zero.
Between Edge 4 and Edge 5: -RES goes from one to zero and EN is zero.
Edge 5: -RES is zero and EN is zero.
Between Edge 5 and Edge 6: -RES goes from zero to one and EN is zero.
Edge 6: -RES is one and EN is zero.
Between Edge 6 and Edge 7: EN goes from zero to one.
Edges 7-12: -RES is one and EN is one.

Given the above values for the inputs -RES and EN, perform the time analysis of the sequential network shown in {image}, determining the value of each output at each clock edge (from 1 to 12).

The description of the network structure is provided below:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [9]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Time Analysis of the Sequential Network

Here's the time analysis of the provided sequential network:

| Clock Edge | -RES | EN | D Flip-Flop (A) | JK1 Flip-Flop (B) | JK2 Flip-Flop (C) | NAND Output (D) |
|---|---|---|---|---|---|---|
| **1** | 1 | 0 | 0 | 0 | 0 | 1 |
| **2** | 1 | 0 | 1 | 0 | 0 | 1 |
| **3** | 1 | 0 | 1 | 1 | 0 | 1 |
| **4** | 1 | 0 | 0 | 1 | 1 | 1 |
| **5** | 0 | 0 | 0 | 0 | 0 | 1 |
| **6** | 1 | 0 | 1 | 0 | 0 | 1 |
| **7** | 1 | 1 | 1 | 1 | 0 | 1 |
| **8** | 1 | 1 | 0 | 1 | 1 | 1 |
| **9** | 1 | 1 | 0 | 0 | 1 | 1 |
| **10** | 1 | 1 | 0 | 0 | 0 | 1 |
| **11** | 1 | 1 | 1 | 0 | 0 | 0 |
| **12** | 1 | 1 | 0 | 1 | 0 | 1 |

**Explanation:**

* **Edges 1-4:** -RES is active, resetting all flip-flops to 0. Since EN is 0, the NAND gate always outputs 1, regardless of the flip-flop states. The D flip-flop toggles on each clock edge due to the '1' connected to its D input and the JK flip-flops act as a counter because J and K are tied to the outpu

## 1.2 Esercizio con recupero diretto

In [12]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
```
Clock Edge | -RES | EN | A | B | C | D 
------- | -------- | -------- | -------- | -------- | -------- | -------- 
1       | 1       | 0       | 0       | 0       | 0       | 1      
2       | 1       | 0       | 1       | 0       | 0       | 1      
3       | 1       | 0       | 1       | 1       | 0       | 1      
4       | 1       | 0       | 1       | 1       | 1       | 1      
5       | 0       | 0       | 0       | 0       | 0       | 1      
6       | 1       | 0       | 1       | 0       | 0       | 1      
7       | 1       | 1       | 1       | 1       | 0       | 0      
8       | 1       | 1       | 1       | 1       | 1       | 0      
9       | 1       | 1       | 1       | 1       | 1       | 0      
10      | 1       | 1       | 1       | 1       | 1       | 0      
11      | 1       | 1       | 1       | 1       | 1       | 0      
12      | 1       | 1       | 1       | 1       | 1       | 0      
```

**Explanation:**

* **Clock Edges 1

## 1.3 Esercizio con recupero diretto e video soluzione

In [13]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The provided circuit consists of a D flip-flop, two JK flip-flops, and a NAND gate. The outputs of the flip-flops are A, B, and C, while the output of the NAND gate is D. The circuit also has two inputs: -RES (reset) and EN (enable).

Here's a step-by-step analysis of the circuit's behavior at each clock edge, taking into account the given input values for -RES and EN:

**Initial State (Before Edge 1)**

- -RES is 1 (inactive), EN is 0.
- All flip-flops are in an indeterminate state as they haven't been initialized yet.
- Since EN is 0, the output of the NAND gate (D) is 1.

**Edge 1**

- -RES is 1, EN is 0.
- The D flip-flop receives 1 at its input and its output (A) becomes 1.
- The first JK flip-flop receives an indeterminate value for J and K, so its output (B) remains indeterminate.
- The second JK flip-flop also receives indeterminate values for J and K, so its output (C) remains indeterminate.
- D remains 1 as EN is still 0.

**Edges 2-4**

- -RES is 1, 

## 1.4 Esercizio con video soluzione

In [14]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is drawing a time diagram of the sequential network. The user is drawing a horizontal line in the row of D before the first rising edge and marking it as 1. The user is drawing squiggly lines in the rows of A, B, and C before the first rising edge. The user is drawing a vertical line in the row of A from the first rising edge to the fifth rising edge and marking it as 1. The user is drawing a horizontal line in the row of B from the first rising edge to the second rising edge and then drawing a vertical line from the second rising edge to the fifth rising edge and marking it as 1. The user is drawing a horizontal line in the row of C from the first rising edge to the third rising edge and then drawing a vertical line from the third rising edge to the fifth rising edge and marking it as 1. The user is drawing a vertical line in the row of D from the first rising edge to the eighth rising edge and marking it as 1. The user is drawing a horizontal line in

## 1.5 Esercizio con libro di testo nel prompt

In [15]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
```
Edge | -RES | EN  | A  | B  | C  | D
------- | -------- | -------- | -------- | -------- | -------- | --------
1       | 1       | 0       | 0       | 0       | 0       | 1
2       | 1       | 0       | 1       | 0       | 0       | 1
3       | 1       | 0       | 1       | 1       | 0       | 1
4       | 1       | 0       | 1       | 1       | 1       | 1
5       | 0       | 0       | 0       | 0       | 0       | 1
6       | 1       | 0       | 1       | 0       | 0       | 1
7       | 1       | 1       | 1       | 1       | 0       | 1
8       | 1       | 1       | 1       | 1       | 1       | 0
9       | 1       | 1       | 1       | 1       | 1       | 0
10      | 1       | 1       | 1       | 1       | 1       | 0
11      | 1       | 1       | 1       | 1       | 1       | 0
12      | 1       | 1       | 1       | 1       | 1       | 0
```

**Explanation:**

* **Edges 1-4:** -RES is active, resetting all flip-flops to 0. EN is 0, so D is always 1 (NA

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [16]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
Okay, here's the time analysis of the sequential network, considering the behavior of each output (A, B, C, D) at each clock edge (1 to 12).

**Initial State (Before Edge 1):**

* A, B, C are indeterminate (due to the nature of JK flip-flops)
* D = 1 (EN is 0, forcing the NAND output to 1)

**Edge 1 - 4:**

* **-RES = 1, EN = 0:**
    * All flip-flops are in reset state, forcing their outputs to 0 (A = B = C = 0).
    * D remains at 1 due to EN being 0.

**Between Edge 4 and Edge 5:**

* **-RES transitions from 1 to 0, EN = 0:**
    * The D flip-flop is now released from reset and will respond to its D input, which is 1.
    * The JK flip-flops remain in reset (B = C = 0).
    * D remains at 1.

**Edge 5:**

* **-RES = 0, EN = 0:**
    * A transitions to 1 (D input was 1 at the previous edge).
    * B and C remain at 0.
    * D remains at 1.

**Between Edge 5 and Edge 6:**

* **-RES transitions from 0 to 1, EN = 0:**
    * The D flip-flop is again put into rese

## 1.7 Esercizio con recupero mediante tool

In [17]:
prompt_text = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on the previous ones.
A time diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.

Edges 1-4: -RES is one and EN is zero.
Between Edge 4 and Edge 5: -RES goes from one to zero and EN is zero.
Edge 5: -RES is zero and EN is zero.
Between Edge 5 and Edge 6: -RES goes from zero to one and EN is zero.
Edge 6: -RES is one and EN is zero.
Between Edge 6 and Edge 7: EN goes from zero to one.
Edges 7-12: -RES is one and EN is one.

Given the above values for the inputs -RES and EN, perform the time analysis of the sequential network described:
{image_description}

Determine the value of each output at each clock edge (from 1 to 12)."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

## Time Analysis of the Sequential Network:

**Edge 1:**

* All flip-flops are cleared (-RES = 0) so A = B = C = 0.
* D = 1 (NAND output is high when any input is low).

**Edges 2-4:**

* -RES = 1, so the flip-flops are no longer forced to zero.
* EN = 0, so D = 1 (EN is an input to the NAND gate).
* Since the D flip-flop's -PRESET is always 1, and its -CLEAR (connected to -RES) is 1, the flip-flop is triggered by the rising edge of the clock. A (the output of the D flip-flop) toggles on each rising edge.
* The JK flip-flops toggle when both J and K are 1. Since the first JK flip-flop's J and K inputs are connected to A and -A respectively, it toggles on each rising edge. The same applies to the second JK flip-flop.
* Therefore, A, B, and C toggle on each rising edge.

**Edge 5:**

* -RES goes low but becomes high before the rising edge. This momentary change does not affect the flip-flops as it does not remain low until the rising edge.
* The outputs A, B, and C continue to toggle.
* 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "Sequential network time analysis"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "Sequential network time analysis"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [18]:
input = f'{{"Sequential network": "{image}", "Task": "{prompt_text}", "Network description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Edge 1: A is indeterminate, B is indeterminate, C is indeterminate, D is indeterminate.
Between Edge 1 and Edge 2: The -RES input goes from one to zero. When -RES switches, A, B, and C go from indeterminate to zero.
Edge 2: A is zero, B is zero, C is zero, D is one.
Between Edge 2 and Edge 3: The -RES input goes from zero to one.
Edge 3: A is zero, B is zero, C is zero, D is one.
Edge 4: A is zero, B is zero, C is zero, D is one.
Edge 5: A goes from zero to one shortly after the edge, B is zero, C is zero, D is one.
Edge 6: A goes from one to zero shortly after the edge, B goes from zero to one shortly after the edge, C is zero, D is one.
Between Edge 6 and Edge 7: The EN input goes from zero to one.
Edge 7: A goes from zero to one shortly after the edge, B is one, C is zero, D is zero.
Edge 8: A goes from one to zero shortly after the edge, B goes from one to zero shortly after the edge, C goes from zero to one shortly after the edge, D is zero.
Edge 9: A goes

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [19]:
input = f'{{"Sequential network": "{image}", "Task": "{prompt_text}", "Network description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
Edge 1: A is indeterminate, B is indeterminate, C is indeterminate, D is one.
Between Edge 1 and Edge 2: -RES is one and EN is zero.
Edge 2: A is indeterminate, B is indeterminate, C is indeterminate, D is one.
Between Edge 2 and Edge 3: -RES is one and EN is zero.
Edge 3: A is indeterminate, B is indeterminate, C is indeterminate, D is one.
Between Edge 3 and Edge 4: -RES is one and EN is zero.
Edge 4: A is indeterminate, B is indeterminate, C is indeterminate, D is one.
Between Edge 4 and Edge 5: -RES goes from one to zero and EN is zero. When -RES switches, A, B, and C go from indeterminate to zero.
Edge 5: A is zero, B is zero, C is zero, D is one.
Between Edge 5 and Edge 6: -RES goes from zero to one and EN is zero.
Edge 6: A is zero, B is zero, C is zero, D is one.
Between Edge 6 and Edge 7: EN goes from zero to one.
Edge 7: A goes from zero to one shortly after the edge, B is zero, C is zero, D is one.
Edge 8: A goes from one to zero shortly after the ed

# Esercizio 12

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image12>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video12>.mp4")

image_description = """The sequential network is synchronous (all flip-flops share the same clock) and has four outputs (Q0, Q1, Q2, -TC).  
The structure of the network is described below:

* **First JK-PET Flip-Flop:**  
    * Receives the common clock signal (CK) shared by all flip-flops. 
    * The J and K inputs are connected together and linked to the input signal EN. 
    * The output (Q) is not connected to anything. 
    * The negated output (-Q) is connected to both the J and K inputs of the second JK-PET flip-flop and to one input of the AND gate. The -Q output is labeled Q0. 
    * A constant value of '1' is connected to the -CLEAR input of the flip-flop. 
    * The -PRESET signal is connected to the -PRESET input of the flip-flop.

* **Second JK-PET Flip-Flop:**  
    * Receives the common clock signal (CK). 
    * The J and K inputs are connected together and linked to the negated output of the first JK-PET flip-flop. 
    * The output (Q) is not connected to anything. 
    * The negated output (-Q) is connected to one input of the AND gate. The -Q output is labeled Q1. 
    * A constant value of '1' is connected to the -CLEAR input of the flip-flop. 
    * The -PRESET signal is connected to the -PRESET input of the flip-flop.

* **Third JK-PET Flip-Flop:**  
    * Receives the common clock signal (CK). 
    * The J and K inputs are connected together and linked to the output of the AND gate. 
    * The output (Q) is not connected to anything. The negated output (-Q) is connected to one input of the NAND gate. The -Q output is labeled Q2. 
    * A constant value of '1' is connected to the -CLEAR input of the flip-flop. 
    * The -PRESET signal is connected to the -PRESET input of the flip-flop.

* **AND Gate:**  
    * Its three inputs are connected to the following: 
        * The negated output (-Q) of the first JK-PET flip-flop labeled Q0. 
        * The negated output (-Q) of the second JK-PET flip-flop labeled Q1.
        * The EN signal. 
    * The output of the AND gate is connected to one input of the NAND gate.

* **NAND Gate:**  
    * Its two inputs are connected to the following: 
        * The output of the AND gate.
        * The negated output (-Q) of the third JK-PET flip-flop labeled Q2.
    * The output of the NAND gate produces the output signal -TC.

**Control Signals:**  
* **CK (Clock):** Determines the circuit's timing, activating the flip-flops on each rising edge.  
* **-PRESET (Preset):** This is an active-low signal. When active (0), it forces the outputs of all flip-flops to one (Q=1).  
* **EN (Enable):** Connected to the AND gate and the first JK-PET flip-flop.  
* **'1' (high):** This is a constant signal with a value of 1.
"""

prompt_text = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on previous ones.
A timing diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.

Edge 1: -PRESET is one, and EN is zero.
Between Edge 1 and Edge 2: -PRESET transitions from one to zero and then immediately back to one, and EN is zero.
Edge 2: -PRESET is one, and EN is zero.
Between Edge 2 and Edge 3: -PRESET is one, and EN transitions from zero to one.
Edges 3-9: -PRESET is one, and EN is one.
Between Edge 9 and Edge 10: -PRESET is one, and EN transitions from one to zero.
Edges 10-16: -RESET is one, and EN is zero.

Given the above values for the -PRESET and EN inputs, perform the timing analysis of the sequential network shown in {image}, determining the value of each output at every clock edge (from 1 to 16).

The description of the network structure is provided below:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [11]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Timing Analysis of the Sequential Network

Here's the timing analysis of the provided sequential network, considering the given input conditions and the behavior of JK-PET flip-flops:

| Clock Edge | -PRESET | EN | Q0 | Q1 | Q2 | -TC | Explanation |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | **1** | **1** | **1** | **1** | -PRESET is active (1), forcing all Q outputs to 1. -TC is 1 because Q2 is 1 and the AND output is 0. |
| 2 | 1 | 0 | **1** | **1** | **1** | **1** | -PRESET momentarily goes low but returns high before the clock edge, so the outputs remain unchanged. |
| 3 | 1 | 1 | **0** | **1** | **1** | **1** | EN becomes 1, allowing the first flip-flop to toggle. Q0 changes to 0. |
| 4 | 1 | 1 | **1** | **0** | **1** | **1** | Q0 being 0 toggles the second flip-flop, changing Q1 to 0. |
| 5 | 1 | 1 | **0** | **1** | **1** | **1** | Q1 being 0 allows the AND gate to output 0, keeping Q2 at 1. |
| 6 | 1 | 1 | **1** | **0** | **1** | **1** |  |
| 7 |

## 1.2 Esercizio con recupero diretto

In [12]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| Clock Edge | -PRESET | EN | Q0 | Q1 | Q2 | -TC |
|---|---|---|---|---|---|---|
| 1        | 1       | 0 | 1 | 1 | 1 | 1  |
| 2        | 1       | 0 | 1 | 1 | 1 | 1  |
| 3        | 1       | 1 | 0 | 1 | 1 | 1  |
| 4        | 1       | 1 | 0 | 0 | 1 | 1  |
| 5        | 1       | 1 | 1 | 0 | 1 | 1  |
| 6        | 1       | 1 | 1 | 1 | 1 | 0  |
| 7        | 1       | 1 | 0 | 1 | 0 | 1  |
| 8        | 1       | 1 | 0 | 0 | 0 | 1  |
| 9        | 1       | 1 | 1 | 0 | 1 | 1  |
| 10       | 1       | 0 | 1 | 1 | 1 | 1  |
| 11       | 1       | 0 | 1 | 1 | 1 | 1  |
| 12       | 1       | 0 | 1 | 1 | 1 | 1  |
| 13       | 1       | 0 | 1 | 1 | 1 | 1  |
| 14       | 1       | 0 | 1 | 1 | 1 | 1  |
| 15       | 1       | 0 | 1 | 1 | 1 | 1  |
| 16       | 1       | 0 | 1 | 1 | 1 | 1  |

**Explanation:**

* **Clock Edge 1:** -PRESET is active, forcing all Q outputs to 1. EN is 0, so the first JK flip-flop is disabled, and the AND gate output is 0. This results in -TC being 

## 1.3 Esercizio con recupero diretto e video soluzione

In [13]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The provided timing diagram accurately represents the behavior of the sequential network based on the given input values for -PRESET and EN. Here's a breakdown of the analysis:

**Initial State (Before Edge 1):**

* -PRESET is 1, EN is 0, and the initial state of the flip-flops is unknown.
* Since EN is 0, the output of the AND gate is 0.
* The NAND gate receives 0 from the AND gate and an unknown value from Q2. Therefore, -TC is 1.

**Edge 1 to Edge 2:**

* -PRESET briefly pulses to 0, forcing Q0, Q1, and Q2 to 1.
* EN remains 0, so the AND gate output stays 0, and -TC remains 1.

**Edge 2 to Edge 3:**

* -PRESET is 1, and EN transitions from 0 to 1.
* All flip-flops maintain their previous state (Q0 = Q1 = Q2 = 1) because the clock edge hasn't occurred yet.
* The AND gate output is still 0 because Q0 and Q1 are 1, but EN was 0 at the clock edge.
* -TC remains 1.

**Edges 3 to 9:**

* -PRESET is 1, and EN is 1.
* The first JK flip-flop toggles on each clock ed

## 1.4 Esercizio con video soluzione

In [31]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The video provides a step-by-step timing analysis of the sequential network, explaining the behavior of each flip-flop and gate at every clock edge. Here's a summary of the analysis, presented as a table for clarity:

| Clock Edge | -PRESET | EN | Q0 | Q1 | Q2 | -TC | Explanation |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | X | X | X | 1 | Outputs Q0, Q1, and Q2 are indeterminate (X) as their initial states are unknown. -TC is 1 because EN is 0, forcing the AND gate output to 0, which in turn makes the NAND gate output 1. |
| 2 | 1 | 0 | X | X | X | 1 | All flip-flops maintain their previous indeterminate states as both J and K inputs are 0 (due to EN=0). -TC remains 1 for the same reason as in edge 1. |
| 3 | 1 | 1 | 1 | X | X | 1 | -PRESET briefly goes low, forcing Q0, Q1, and Q2 to 1. However, since EN is still 0, the flip-flops revert to their previous indeterminate states before the next clock edge. |
| 4 | 1 | 1 | 0 | X | X | 1 | The first flip-flop 

## 1.5 Esercizio con libro di testo nel prompt

In [13]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Timing Analysis of the Sequential Network

Here is the timing analysis of the provided sequential network, considering the given input values for -PRESET and EN, and determining the value of each output at every clock edge (1 to 16):

**Initial State:**

Before the first clock edge, we assume all flip-flops are in an unknown state.

**Edge 1: -PRESET = 1, EN = 0**

* **-PRESET = 1:**  Flip-flops are not preset.
* **EN = 0:**  First JK-FF J and K inputs are 0, so it will hold its previous state. AND gate output is 0. Third JK-FF J and K inputs are 0, so it will hold its previous state.
* **Outputs:** Q0, Q1, and Q2 are unknown. -TC is 1 (NAND gate with one input 0).

**Between Edge 1 and Edge 2: -PRESET transitions 1->0->1, EN = 0**

* **-PRESET 1->0:** All flip-flops are preset (Q = 1, -Q = 0).
* **-PRESET 0->1:** Flip-flops are no longer preset.
* **EN = 0:** Same behavior as Edge 1.
* **Outputs:** During the -PRESET = 0 phase, Q0 = Q1 = Q2 = 0.  -TC = 1. A

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [14]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
Okay, here's the timing analysis of the sequential network, considering the given inputs and the circuit description:

**Initial State (Before Edge 1):**

* All Q outputs (Q0, Q1, Q2) are indeterminate since the initial state of the flip-flops is unknown.
* -TC is 1 because EN is 0, forcing the AND gate output to 0, and a 0 input to the NAND gate results in a 1 output.

**Edge 1: -PRESET = 1, EN = 0**

* -PRESET = 1 has no effect on the flip-flops.
* All Q outputs remain indeterminate.
* -TC remains 1 (as explained in the initial state).

**Between Edge 1 and Edge 2: -PRESET pulses to 0 then 1, EN = 0**

* The -PRESET pulse to 0 forces all Q outputs to 1 (Q0 = Q1 = Q2 = 1).
* -TC remains 1.

**Edge 2: -PRESET = 1, EN = 0**

* Q0, Q1, and Q2 remain at 1 due to the -PRESET pulse before the edge.
* -TC remains 1.

**Between Edge 2 and Edge 3: -PRESET = 1, EN transitions from 0 to 1**

* Q0, Q1, and Q2 remain at 1.
* -TC remains 1 until EN becomes 1, then it transi

## 1.7 Esercizio con recupero mediante tool

In [15]:
prompt_text = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on previous ones.
A timing diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.

Edge 1: -PRESET is one, and EN is zero.
Between Edge 1 and Edge 2: -PRESET transitions from one to zero and then immediately back to one, and EN is zero.
Edge 2: -PRESET is one, and EN is zero.
Between Edge 2 and Edge 3: -PRESET is one, and EN transitions from zero to one.
Edges 3-9: -PRESET is one, and EN is one.
Between Edge 9 and Edge 10: -PRESET is one, and EN transitions from one to zero.
Edges 10-16: -RESET is one, and EN is zero.

Given the above values for the -PRESET and EN inputs, perform the timing analysis of the sequential network described:
{image_description} 

Determine the value of each output at every clock edge (from 1 to 16)."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

**Edge 1:**
* Q0 = 1 (Due to -PRESET being active)
* Q1 = 1 (Due to -PRESET being active)
* Q2 = 1 (Due to -PRESET being active)
* -TC = 0 (Q0 = 1, Q1 = 1, Q2 = 1, EN = 0,  The output of the AND gate is 0. Therefore, the output of the NAND gate is 1)

**Edge 2:**
* Q0 = 1 (-PRESET was activated between Edge 1 and Edge 2)
* Q1 = 1 (-PRESET was activated between Edge 1 and Edge 2)
* Q2 = 1 (-PRESET was activated between Edge 1 and Edge 2)
* -TC = 0 (Q0 = 1, Q1 = 1, Q2 = 1, EN = 0,  The output of the AND gate is 0. Therefore, the output of the NAND gate is 1)

**Edge 3:**
* Q0 = 0 (EN = 1, J = K = 1 toggles the first flip-flop)
* Q1 = 1 (The second flip-flop is triggered by the previous state of Q0, which was 1)
* Q2 = 1 (The third flip-flop is triggered by the output of the AND gate, which is 0)
* -TC = 0 (Q0 = 0, Q1 = 1, Q2 = 1, EN = 1, The output of the AND gate is 0. Therefore, the output of the NAND gate is 1)

**Edge 4:**
* Q0 = 1 
* Q1 = 0 
* Q2 = 1 
* -TC = 0 

**Edge 5:**
* Q0 = 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "1. Draw a timing diagram for the sequential network described.\n2. Label the edges of the timing diagram with the values of the -PRESET and EN inputs.\n3. Label the outputs of the sequential network at each clock edge."
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "1. Draw a timing diagram for the sequential network described.\n2. Label the edges of the timing diagram with the values of the -PRESET and EN inputs.\n3. Label the outputs of the sequential network at each clock edge."
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [16]:
input = f'{{"Sequential network": "{image}", "Task": "{prompt_text}", "Network description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Edge 1: Q0 is one, Q1 is one, Q2 is one, -TC is zero.
Between Edge 1 and Edge 2: The -PRESET input goes from one to zero. When -PRESET switches, Q0, Q1, and Q2 go from one to zero. Then, the -PRESET input goes from zero to one.
Edge 2: Q0 is one, Q1 is one, Q2 is one, -TC is zero.
Between Edge 2 and Edge 3: The EN input goes from zero to one.
Edges 3-9: Q0 toggles between zero and one with each clock edge, Q1 is zero, Q2 is one, -TC is one.
Between Edge 9 and Edge 10: The EN input goes from one to zero.
Edges 10-16: Q0 is zero, Q1 is zero, Q2 is one, -TC is one. 


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [17]:
input = f'{{"Sequential network": "{image}", "Task": "{prompt_text}", "Network description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
Edge 1: Q0 is indeterminate, Q1 is indeterminate, Q2 is indeterminate, -TC is one.
Between Edge 1 and Edge 2: The -PRESET input pulses to zero. When -PRESET switches, Q0, Q1, and Q2 go from indeterminate to one. Then, the -PRESET input goes back to one.
Edge 2: Q0 is one, Q1 is one, Q2 is one, -TC is one.
Between Edge 2 and Edge 3: The EN input goes from zero to one.
Edges 3-9: Q0 toggles on each rising edge (from one to zero and vice versa), Q1 toggles on each rising edge following Q0's switch from zero to one, Q2 toggles on each rising edge following Q1's switch from zero to one, -TC is zero when Q0, Q1, and EN are one, otherwise -TC is one.
Between Edge 9 and Edge 10: The EN input goes from one to zero.
Edges 10-16: Q0 maintains its state, Q1 maintains its state, Q2 maintains its state, -TC is one. 


# Esercizio 13

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image13>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video13>.mp4")

image_description = """The sequential network is synchronous (all flip-flops share the same clock) and has four outputs (A, B, C, D).  
The structure of the network is described below:

**First D-PET Flip-Flop:**  
    * Receives the common clock signal (CK) shared by all flip-flops.  
    * Its data input (D) is connected to the output of the second EXOR gate.  
    * The output (Q) is connected to the D input of the second D-PET flip-flop. The Q output is labeled D.  
    * The negated output (-Q) is not connected to anything.  
    * The -RESET signal is connected to the -CLEAR input of the flip-flop.  
    * A constant value of '1' is connected to the -PRESET input of the flip-flop.

**Second D-PET Flip-Flop:**  
    * Receives the common clock signal (CK).  
    * Its data input (D) is connected to the output of the first D-PET flip-flop.  
    * The output (Q) is connected to the D input of the third D-PET flip-flop. The Q output is labeled C.  
    * The negated output (-Q) is not connected to anything.  
    * The -RESET signal is connected to the -CLEAR input of the flip-flop.  
    * A constant value of '1' is connected to the -PRESET input of the flip-flop.

**Third D-PET Flip-Flop:**  
    * Receives the common clock signal (CK).  
    * Its data input (D) is connected to the output of the second D-PET flip-flop.  
    * The output (Q) is connected to the D input of the fourth D-PET flip-flop and one input of the first EXOR gate. The Q output is labeled B.  
    * The negated output (-Q) is not connected to anything.  
    * The -RESET signal is connected to the -CLEAR input of the flip-flop.  
    * A constant value of '1' is connected to the -PRESET input of the flip-flop.

**Fourth D-PET Flip-Flop:**  
    * Receives the common clock signal (CK) shared by all flip-flops.  
    * Its data input (D) is connected to the output of the third D-PET flip-flop.  
    * The output (Q) is connected to one input of the first EXOR gate. The Q output is labeled A.  
    * The negated output (-Q) is not connected to anything.  
    * The -RESET signal is connected to the -CLEAR input of the flip-flop.  
    * A constant value of '1' is connected to the -PRESET input of the flip-flop.

**First EXOR Gate:**  
    * Its two inputs are connected to the following:  
        * The Q output of the third D-PET flip-flop, labeled B.  
        * The Q output of the fourth D-PET flip-flop, labeled A.  
    * The output of the first EXOR gate is connected to one input of the second EXOR gate.

**Second EXOR Gate:**  
    * Its two inputs are connected to the following:  
        * The output of the first EXOR gate.  
        * The IN signal.  
    * The output of the second EXOR gate is connected to the D input of the first D-PET flip-flop.

**Control Signals:**

**CK (Clock):** Determines the circuit's timing, activating the flip-flops on each rising edge.  
**-RESET (Reset):** This is an active-low signal. When active (0), it forces the outputs of all flip-flops to zero (Q=0).  
**IN (Input):** Connected to the second EXOR gate.  
**'1' (high):** This is a constant signal with a value of 1.
"""

prompt_text = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on previous ones.
A timing diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.

Edge 1: -RESET is one, and IN is zero.
Between Edge 1 and Edge 2: -RESET transitions from one to zero, and IN is zero.
Edge 2: -RESET is zero, and IN is zero.
Between Edge 2 and Edge 3: -RESET transitions from zero to one, and IN is zero.
Edges 3-4: -RESET is one, and IN is zero.
Between Edge 4 and Edge 5: -RESET is one, and IN transitions from zero to one.
Edges 5-13: -RESET is one, and IN is one.

Given the above values for the -RESET and IN inputs, perform the timing analysis of the sequential network shown in {image}, determining the value of each output at every clock edge (from 1 to 13).

The description of the network structure is provided below:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [19]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Timing Analysis of the Sequential Network

Here's a breakdown of the circuit's behavior at each clock edge, considering the provided input signals and the network structure:

**Initial State:**
Before Edge 1, we assume all flip-flops are in an undefined state.

**Edge 1:**
* -RESET = 1, IN = 0
* All flip-flops are preset to 1 due to the constant '1' connected to their -PRESET inputs.
* **A = 1, B = 1, C = 1, D = 1**

**Between Edge 1 and Edge 2:**
* -RESET transitions from 1 to 0, IN = 0
* The change in -RESET doesn't affect the outputs as the flip-flops are already at 1.

**Edge 2:**
* -RESET = 0, IN = 0
* -RESET being active (0) clears all flip-flops.
* **A = 0, B = 0, C = 0, D = 0**

**Between Edge 2 and Edge 3:**
* -RESET transitions from 0 to 1, IN = 0

**Edge 3:**
* -RESET = 1, IN = 0
* The first rising edge after -RESET goes inactive will capture the data present at the D input of each flip-flop.
    * D (Flip-flop 1) receives 1 (from the second EXOR 

## 1.2 Esercizio con recupero diretto

In [20]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
```
Clock Edge | -RESET | IN | A | B | C | D 
------- | -------- | -------- | -------- | -------- | -------- | --------
1       | 1       | 0       | 0 | 0 | 0 | 0
2       | 0       | 0       | 0 | 0 | 0 | 0
3       | 1       | 0       | 1 | 0 | 0 | 0 
4       | 1       | 0       | 1 | 1 | 0 | 0
5       | 1       | 1       | 0 | 1 | 1 | 0
6       | 1       | 1       | 0 | 0 | 1 | 1
7       | 1       | 1       | 0 | 1 | 0 | 1
8       | 1       | 1       | 1 | 0 | 1 | 0
9       | 1       | 1       | 1 | 1 | 0 | 1
10      | 1       | 1       | 0 | 1 | 1 | 0
11      | 1       | 1       | 0 | 0 | 1 | 1
12      | 1       | 1       | 0 | 1 | 0 | 1
13      | 1       | 1       | 1 | 0 | 1 | 0 
```

**Explanation:**

* **Edge 1:** -RESET is active, forcing all flip-flop outputs (A, B, C, D) to 0, regardless of the clock edge.
* **Edge 2:** -RESET transitions to inactive. However, the flip-flops maintain their previous state (0) until the next clock edge.
* **Edge 3:** -R

## 1.3 Esercizio con recupero diretto e video soluzione

In [21]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The provided timing diagram shows the Clock, IN, and -RESET signals. The outputs A, B, C, and D are all initialized to 0 due to the -RESET signal being active (low) before the first clock edge. Here's a breakdown of the output values at each clock edge:

**Edge 1:**
* A = 0
* B = 0
* C = 0
* D = 0 

**Edge 2:**
* A = 0
* B = 0
* C = 0
* D = 0 (-RESET is still active)

**Edge 3:**
* A = 0
* B = 0
* C = 0
* D = 0

**Edge 4:**
* A = 0
* B = 0
* C = 0
* D = 0

**Edge 5:**
* A = 0
* B = 0
* C = 0
* D = 1 (IN transitions to 1, propagating through the XOR gates to the first D flip-flop)

**Edge 6:**
* A = 0
* B = 0
* C = 1 (D = 1 from the previous edge is shifted to the second flip-flop)
* D = 1

**Edge 7:**
* A = 0
* B = 1 (C = 1 from the previous edge is shifted to the third flip-flop)
* C = 1
* D = 1

**Edge 8:**
* A = 1 (B = 1 from the previous edge is shifted to the fourth flip-flop)
* B = 1
* C = 1
* D = 0 (B XOR A = 1, 1 XOR IN = 0)

**Edge 9:**
* A = 1
* B = 1

## 1.4 Esercizio con video soluzione

In [22]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is trying to determine the output values of a sequential network at each clock edge from 1 to 13, given specific values for the -RESET and IN inputs. The network consists of four D-PET flip-flops and two XOR gates. 

Here's a breakdown of the solution:

**Understanding the Network**

* **D-PET Flip-Flops:** These flip-flops store a value (0 or 1) and change their output based on the input at the rising edge of the clock signal (CK). The -CLEAR input resets the flip-flop to 0 when activated.
* **XOR Gates:**  These gates produce a 1 output only when their inputs are different (0 and 1, or 1 and 0).

**Timing Analysis**

The user provides a detailed description of the input signals (-RESET and IN) at each clock edge. The timing analysis involves determining the output of each flip-flop (A, B, C, D) at each rising clock edge, considering the current input values and the previous state of the flip-flops.

**Solution**

The video solution carefully walks th

## 1.5 Esercizio con libro di testo nel prompt

In [45]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Timing Analysis of the Sequential Network

The provided network is a 4-bit Johnson counter with a synchronous reset. Here's the timing analysis of the outputs (A, B, C, D) at each clock edge (1-13) considering the given input values for -RESET and IN:

| Clock Edge | -RESET | IN | A | B | C | D | Explanation |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | 0 | All outputs are reset to 0 due to -RESET being active. |
| 2 | 0 | 0 | 0 | 0 | 0 | 0 | -RESET transitions to inactive, but the outputs retain their previous values (0) until the next active clock edge. |
| 3 | 1 | 0 | 1 | 0 | 0 | 0 | The first D flip-flop receives a '1' from the second XOR gate (because B and A are 0, and IN is 0). This '1' is loaded onto output D at the clock edge. |
| 4 | 1 | 0 | 1 | 1 | 0 | 0 | The '1' on D is shifted to C, and a new '1' is loaded onto D due to the same logic as in edge 3. |
| 5 | 1 | 1 | 1 | 1 | 1 | 0 | The '1's are shifted, and a new '1' is loaded ont

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [23]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
```
Clock Edge | -RESET | IN | D | C | B | A 
------- | -------- | -------- | -------- | -------- | -------- | --------
1        | 1       | 0       | X       | X       | X       | X
2        | 0       | 0       | 0       | 0       | 0       | 0
3        | 1       | 0       | 0       | 0       | 0       | 0
4        | 1       | 0       | 0       | 0       | 0       | 0
5        | 1       | 1       | 1       | 0       | 0       | 0
6        | 1       | 1       | 0       | 1       | 0       | 0
7        | 1       | 1       | 1       | 0       | 1       | 0
8        | 1       | 1       | 1       | 1       | 0       | 1
9        | 1       | 1       | 0       | 1       | 1       | 1
10       | 1       | 1       | 0       | 1       | 1       | 0
11       | 1       | 1       | 1       | 1       | 0       | 1
12       | 1       | 1       | 0       | 1       | 1       | 1
13       | 1       | 1       | 0       | 1       | 1       | 0
```

**Explanation:**

* **Edge 1:**

## 1.7 Esercizio con recupero mediante tool

In [24]:
prompt_text = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on previous ones.
A timing diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.

Edge 1: -RESET is one, and IN is zero.
Between Edge 1 and Edge 2: -RESET transitions from one to zero, and IN is zero.
Edge 2: -RESET is zero, and IN is zero.
Between Edge 2 and Edge 3: -RESET transitions from zero to one, and IN is zero.
Edges 3-4: -RESET is one, and IN is zero.
Between Edge 4 and Edge 5: -RESET is one, and IN transitions from zero to one.
Edges 5-13: -RESET is one, and IN is one.

Given the above values for the -RESET and IN inputs, perform the timing analysis of the sequential network described:
{image_description}

Determine the value of each output at every clock edge (from 1 to 13)."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

**Edge 1:**
* All outputs (A, B, C, D) are 0 because -RESET is active (0).

**Between Edge 1 and Edge 2:**
* -RESET transitions to inactive (1). The flip-flops are still cleared.

**Edge 2:**
* All outputs remain 0. Although -RESET is inactive, the flip-flops were cleared on the previous clock cycle.

**Between Edge 2 and Edge 3:**
* -RESET transitions to active (0), clearing the flip-flops.

**Edges 3 and 4:**
* All outputs remain 0 as -RESET is active.

**Between Edge 4 and Edge 5:**
* -RESET is inactive, and IN changes to 1. 

**Edge 5:**
* D = 1 (Due to the EXOR gates and IN = 1). 
* A, B, and C remain 0.

**Edge 6:**
* D = 0 
* A = 1 (Previous D value)
* B and C remain 0

**Edge 7:**
* D = 1 
* A = 0 
* B = 1 (Previous A value)
* C remains 0

**Edge 8:**
* D = 0
* A = 1 
* B = 0 
* C = 1 (Previous B value)

**Edge 9 - Edge 13:**
* The output sequence from Edge 8 will repeat (1010, 1010...). This is because the network has reached a stable state where the input (IN = 1) and the fee

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "sequential circuit timing analysis"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "sequential circuit timing analysis"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [25]:
input = f'{{"Sequential network": "{image}", "Task": "{prompt_text}", "Network description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Edge 1: A is indeterminate, B is indeterminate, C is indeterminate, D is indeterminate.
Between Edge 1 and Edge 2: The -RESET input goes from one to zero. When -RESET switches, A, B, C, and D go from indeterminate to zero.
Edge 2: A is zero, B is zero, C is zero, D is zero.
Between Edge 2 and Edge 3: The -RESET input goes from zero to one.
Edges 3-13: -RESET is one, so the flip-flops operate normally according to their inputs. The sequential network acts as a 4-bit shift register, shifting the value of IN into the register on each clock edge.
Edge 3: A is zero, B is zero, C is zero, D is zero.
Edge 4: A is zero, B is zero, C is zero, D is zero.
Edge 5: A is zero, B is zero, C is zero, D is one.
Edge 6: A is zero, B is zero, C is one, D is one.
Edge 7: A is zero, B is one, C is one, D is one.
Edge 8: A is one, B is one, C is one, D is one.
Edge 9: A is one, B is one, C is one, D is one.
Edge 10: A is one, B is one, C is one, D is one.
Edge 11: A is one, B is one

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [26]:
input = f'{{"Sequential network": "{image}", "Task": "{prompt_text}", "Network description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
Edge 1: A is indeterminate, B is indeterminate, C is indeterminate, D is indeterminate.
Between Edge 1 and Edge 2: -RESET goes from one to zero, so A, B, C, and D go from indeterminate to zero.
Edge 2: A is zero, B is zero, C is zero, D is zero.
Between Edge 2 and Edge 3: -RESET goes from zero to one.
Edges 3-4: A is zero, B is zero, C is zero, D is zero.
Between Edge 4 and Edge 5: IN goes from zero to one.
Edge 5: A is zero, B is zero, C is zero, D goes from zero to one shortly after the edge.
Edge 6: A is zero, B is zero, C goes from zero to one shortly after the edge, D goes from one to zero shortly after the edge.
Edge 7: A is zero, B goes from zero to one shortly after the edge, C goes from one to zero shortly after the edge, D goes from zero to one shortly after the edge.
Edge 8: A goes from zero to one shortly after the edge, B goes from one to zero shortly after the edge, C goes from zero to one shortly after the edge, D is one.
Edge 9: A goes from one 

# Esercizio 14

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image14>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video14>.mp4")

image_description = """The sequential network is synchronous (all flip-flops share the same clock) and has three outputs (O1, O2, O3). 
The structure of the network is described below:

* **First JK-PET Flip-Flop:** 
    * Receives the common clock signal (CK) shared by all the flip-flops.
    * The J and K inputs are tied together and connected to the input signal IN.
    * The output (Q) is connected to the D input of the D-PET flip-flop. The Q output is called O1. 
    * The negated output (-Q) is not connected to anything.
    * The -RESET signal is connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **D-PET Flip-Flop:** 
    * Receives the common clock signal (CK).
    * The D input is connected to the output of the first JK-PET flip-flop.
    * The output (Q) is not connected to anything. The Q output is called O2. 
    * The negated output (-Q) is connected to the J and K inputs of the second JK-PET flip-flop.
    * The -RESET signal is connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.
* **Second JK-PET Flip-Flop:** 
    * Receives the common clock signal (CK).
    * The J and K inputs are tied together and connected to the negated output of the D-PET flip-flop.
    * The output (Q) is not connected to anything. The Q output is called O3. 
    * The negated output (-Q) is not connected to anything.
    * The -RESET signal is connected to the -CLEAR of the flip-flop.
    * The constant value '1' is connected to the -PRESET input of the flip-flop.

**Control Signals:**

* **CK (Clock):**  Determines the timing of the circuit, activating the flip-flops on each rising edge.
* **-RESET (Reset):**  This is an active-low signal. When active (0), it forces the outputs of all flip-flops to zero (Q=0). 
* **IN (Enable):**  Connected to the J and K inputs of the first JK-PET flip-flop.
* **'1' (high):** This is a constant signal set to 1.
"""

prompt_text = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on the previous ones.
A timing diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.

Edges 1-2: -RESET is one, and IN is zero.
Between edge 2 and edge 3: -RESET goes from one to zero and then from zero to one, and IN is zero.
Edges 3-5: -RESET is one, and IN is zero.
Between edge 5 and edge 6: -RESET is one, and IN goes from zero to one.
Edges 6-11: -RESET is one, and IN is one.
Between edge 11 and edge 12: -RESET is one, and IN goes from one to zero.
Edges 12-21: -RESET is one, and IN is zero.

Given the above values for the inputs -RESET and IN, perform the timing analysis of the sequential network depicted in {image}, determining the value for each output at each clock edge (from 1 to 21).

The description of the network structure is provided below:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [33]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| Clock Edge | -RESET | IN | O1 | O2 | O3 | Notes |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | All outputs are initially 0 due to -RESET being inactive (1). |
| 2 | 1 | 0 | 0 | 0 | 0 | No change as IN is 0 and -RESET is still inactive. |
| 3 | 1 | 0 | 0 | 0 | 0 | Although -RESET briefly goes to 0, it returns to 1 before the clock edge, so no reset occurs. |
| 4 | 1 | 0 | 0 | 0 | 0 |  No change as IN is 0. |
| 5 | 1 | 0 | 0 | 0 | 0 | No change as IN is 0. |
| 6 | 1 | 1 | 1 | 0 | 0 | IN becomes 1, so the first JK flip-flop toggles on the rising edge, setting O1 to 1. |
| 7 | 1 | 1 | 0 | 1 | 0 | O1 was 1, so the D flip-flop latches a 1, setting O2 to 1. |
| 8 | 1 | 1 | 1 | 1 | 1 |  O2 is 1, so -Q of the D flip-flop is 0, causing the second JK flip-flop to toggle, setting O3 to 1. |
| 9 | 1 | 1 | 0 | 0 | 1 | O1 toggles to 0. O2 becomes 0 because D input is now 0. |
| 10 | 1 | 1 | 1 | 0 | 0 | O1 toggles to 1. O3 toggles to 0 because J and K are 1 (-Q of

## 1.2 Esercizio con recupero diretto

In [34]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| Clock Edge | -RESET | IN | JK1 Q (O1) | D Q (O2) | JK2 Q (O3) |
|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 1 |
| 2 | 1 | 0 | 0 | 0 | 1 |
| 3 | 1 | 0 | 0 | 0 | 1 |
| 4 | 1 | 0 | 0 | 0 | 1 |
| 5 | 1 | 0 | 0 | 0 | 1 |
| 6 | 1 | 1 | 1 | 0 | 1 |
| 7 | 1 | 1 | 0 | 1 | 1 |
| 8 | 1 | 1 | 1 | 1 | 0 |
| 9 | 1 | 1 | 0 | 0 | 0 |
| 10 | 1 | 1 | 1 | 0 | 1 |
| 11 | 1 | 1 | 0 | 1 | 1 |
| 12 | 1 | 0 | 0 | 1 | 0 |
| 13 | 1 | 0 | 1 | 0 | 0 |
| 14 | 1 | 0 | 0 | 0 | 1 |
| 15 | 1 | 0 | 0 | 0 | 1 |
| 16 | 1 | 0 | 0 | 0 | 1 |
| 17 | 1 | 0 | 0 | 0 | 1 |
| 18 | 1 | 0 | 0 | 0 | 1 |
| 19 | 1 | 0 | 0 | 0 | 1 |
| 20 | 1 | 0 | 0 | 0 | 1 |
| 21 | 1 | 0 | 0 | 0 | 1 |

**Explanation:**

* **Clock Edges:** The outputs (O1, O2, O3) only change on the rising edge of the clock signal (CK).
* **-RESET:** When -RESET is active (0), it overrides all other inputs and forces all outputs to 0. However, in this timing diagram, -RESET is always inactive (1), so it doesn't directly affect the outputs

## 1.3 Esercizio con recupero diretto e video soluzione

In [35]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The provided timing diagram shows the correct values for O1, O2, and O3. 

Here is a breakdown of the output values at each clock edge:

**Edge 1:**
* O1: Indeterminate (due to unknown initial state)
* O2: Indeterminate (due to unknown initial state)
* O3: Indeterminate (due to unknown initial state)

**Edge 2:**
* O1: Indeterminate (maintained due to J=K=0)
* O2: Indeterminate (maintained due to D being indeterminate)
* O3: Indeterminate (maintained due to J=K being indeterminate)

**Edge 3:**
* O1: 0 (reset to 0 by -RESET signal)
* O2: 0 (reset to 0 by -RESET signal)
* O3: 0 (reset to 0 by -RESET signal)

**Edge 4:**
* O1: 0 (maintained due to J=K=0)
* O2: 0 (maintained due to D=0)
* O3: 1 (toggles due to J=K=1)

**Edge 5:**
* O1: 0 (maintained due to J=K=0)
* O2: 0 (maintained due to D=0)
* O3: 0 (toggles due to J=K=1)

**Edge 6:**
* O1: 1 (toggles due to J=K=1)
* O2: 0 (maintained due to D=0)
* O3: 1 (toggles due to J=K=1)

**Edge 7:**
* O1: 0 (toggles due 

## 1.4 Esercizio con video soluzione

In [10]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is asking for a timing analysis of a sequential network given a circuit diagram and a timing diagram for the inputs. The user also provides a detailed description of the network structure. The video provided shows the step-by-step process of determining the value of each output at each clock edge.

Here's a summary of the analysis:

**Initial State:**

* Before the first rising edge of the clock, the state of all flip-flops is unknown, so the outputs O1, O2, and O3 are indeterminate.

**Edge 1-2:**

* -RESET is active (0), forcing all outputs to 0.
* IN is 0, so the first JK flip-flop maintains its state (0).
* The D flip-flop reads the output of the first JK flip-flop (0) and its output remains 0.
* The second JK flip-flop reads the negated output of the D flip-flop (1) and toggles its output to 1.

**Between Edge 2 and Edge 3:**

* -RESET goes from 0 to 1 and back to 0, causing a brief reset pulse. This forces all outputs back to 0 momentarily.

**Ed

## 1.5 Esercizio con libro di testo nel prompt

In [11]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Timing Analysis of the Sequential Network

Here's the timing analysis of the provided sequential network, considering the behavior of each flip-flop and the given input values for -RESET and IN at each clock edge (1 to 21).

**Understanding the Network:**

* **FF1 (JK-PET):** Toggles on rising clock edge if J=K=1 (IN=1). Reset to 0 when -RESET=0.
* **FF2 (D-PET):**  Latches the value of FF1's output (O1) on the rising clock edge. Reset to 0 when -RESET=0.
* **FF3 (JK-PET):** Toggles on rising clock edge if J=K=1 (FF2's -Q = 1). Reset to 0 when -RESET=0.

**Timing Diagram:**

| Edge | -RESET | IN | O1 | O2 | O3 | Explanation |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | All outputs are 0 due to initial -RESET. |
| 2 | 1 | 0 | 0 | 0 | 0 | No change, IN=0 so FF1 doesn't toggle. |
| 3 | 1 | 0 | 0 | 0 | 0 | -RESET pulse doesn't affect outputs as they are already 0. |
| 4 | 1 | 0 | 0 | 0 | 0 | No change, IN=0. |
| 5 | 1 | 0 | 0 | 0 | 0 | No change, IN=

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [12]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
The provided timing diagram for outputs O1, O2, and O3 is correct. 

Here's a breakdown of the analysis, referencing the clock edges (1 to 21) as described in the problem:

**Initial State (Before Edge 1):**

* All outputs (O1, O2, O3) are indeterminate as the initial state of the flip-flops is unknown.

**Edges 1-2:**

* -RESET is active (low), forcing all outputs to 0 regardless of the IN signal.

**Between Edge 2 and Edge 3:**

* -RESET briefly deactivates and reactivates. Since it's an asynchronous reset, this forces the outputs back to 0 before edge 3.

**Edges 3-5:**

* -RESET is inactive (high), IN is 0. The first JK flip-flop maintains its previous state (0), so O1 remains 0. This also means the D input of the D flip-flop is 0, so O2 remains 0. The second JK flip-flop also maintains its state (0), so O3 remains 0.

**Between Edge 5 and Edge 6:**

* IN transitions from 0 to 1. This sets the J and K inputs of the first JK flip-flop to 1, preparing it to t

## 1.7 Esercizio con recupero mediante tool

In [13]:
prompt_text = f"""A sequential network is a circuit whose output values depend not only on the current input values but also on the previous ones.
A timing diagram is a graph that schematically represents how the inputs and outputs of a digital circuit vary over time.

Edges 1-2: -RESET is one, and IN is zero.
Between edge 2 and edge 3: -RESET goes from one to zero and then from zero to one, and IN is zero.
Edges 3-5: -RESET is one, and IN is zero.
Between edge 5 and edge 6: -RESET is one, and IN goes from zero to one.
Edges 6-11: -RESET is one, and IN is one.
Between edge 11 and edge 12: -RESET is one, and IN goes from one to zero.
Edges 12-21: -RESET is one, and IN is zero.

Given the above values for the inputs -RESET and IN, perform the timing analysis of the sequential network described:
{image_description}

Determine the value for each output at each clock edge (from 1 to 21)."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

**Edge 1:**
* -RESET is active, forcing all Q outputs to 0.
* O1 = 0, O2 = 0, O3 = 0

**Edge 2:** 
* -RESET is deactivated. With J and K tied to '0' (IN=0), the first JK-PET flip-flop maintains its previous state (Q=0).
* O1 = 0, O2 = 0, O3 = 0

**Edge 3:**
* Same as Edge 2.
* O1 = 0, O2 = 0, O3 = 0

**Edge 4:**
* Same as Edge 2.
* O1 = 0, O2 = 0, O3 = 0

**Edge 5:**
* Same as Edge 2.
* O1 = 0, O2 = 0, O3 = 0

**Edge 6:**
* IN changes to '1'. The first JK-PET flip-flop toggles its state (Q=1) as J and K are now tied to '1'.
* O1 = 1, O2 = 0, O3 = 0

**Edge 7:**
* The D-PET flip-flop receives '1' from the first JK-PET flip-flop and updates its output (Q=1).
* O1 = 1, O2 = 1, O3 = 0

**Edge 8:**
* The second JK-PET flip-flop receives '0' (-Q of the D-PET flip-flop) and resets its output (Q=0).
* O1 = 1, O2 = 1, O3 = 0

**Edge 9:**
* The first JK-PET flip-flop toggles (Q=0) as J and K are tied to '1'.
* O1 = 0, O2 = 1, O3 = 0

**Edge 10:**
* The D-PET flip-flop receives '0' and updates it

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "Sequential network timing analysis"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "Sequential network timing analysis"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [14]:
input = f'{{"Sequential network": "{image}", "Task": "{prompt_text}", "Network description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Edge 1: O1 is indeterminate, O2 is indeterminate, O3 is indeterminate.
Between Edge 1 and Edge 2: -RESET goes from one to zero. When -RESET switches, O1, O2, and O3 go from indeterminate to zero. Then, -RESET goes from zero to one.
Edge 2: O1 is zero, O2 is zero, O3 is zero.
Edge 3: O1 is zero, O2 is zero, O3 is zero.
Edge 4: O1 is zero, O2 is zero, O3 is zero.
Edge 5: O1 is zero, O2 is zero, O3 is zero.
Between Edge 5 and Edge 6: IN goes from zero to one.
Edge 6: O1 goes from zero to one shortly after the edge, O2 is zero, O3 is zero.
Edge 7: O1 goes from one to zero shortly after the edge, O2 goes from zero to one shortly after the edge, O3 is zero.
Edge 8: O1 goes from zero to one shortly after the edge, O2 is one, O3 is zero.
Edge 9: O1 goes from one to zero shortly after the edge, O2 goes from one to zero shortly after the edge, O3 goes from zero to one shortly after the edge.
Edge 10: O1 goes from zero to one shortly after the edge, O2 is zero, O3 is one.

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [15]:
input = f'{{"Sequential network": "{image}", "Task": "{prompt_text}", "Network description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on sequential networks, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
Edge 1: O1 is indeterminate, O2 is indeterminate, O3 is indeterminate.
Between Edge 1 and Edge 2: -RESET goes from one to zero. When -RESET switches, O1, O2, and O3 go from indeterminate to zero. Then, -RESET goes from zero to one.
Edge 2: O1 is zero, O2 is zero, O3 is zero.
Between Edge 2 and Edge 3: -RESET goes from one to zero and then immediately from zero to one.
Edge 3: O1 is zero, O2 is zero, O3 is zero.
Edge 4: O1 is zero, O2 is zero, O3 is zero.
Edge 5: O1 is zero, O2 is zero, O3 is zero.
Between Edge 5 and Edge 6: IN goes from zero to one.
Edge 6: O1 goes from zero to one shortly after the edge, O2 is zero, O3 is zero.
Edge 7: O1 goes from one to zero shortly after the edge, O2 goes from zero to one shortly after the edge, O3 is one.
Edge 8: O1 goes from zero to one shortly after the edge, O2 goes from one to zero shortly after the edge, O3 goes from one to zero shortly after the edge.
Edge 9: O1 goes from one to zero shortly after the edge, O2 goes f

# Esempi e contesto per la categoria 4: Macchine a stati finiti

In [ ]:
# Immagini degli esempi per il few-shot prompting
example_image1 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image1-category4>.png")
example_image2 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image2-category4>.png")
example_image3 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image3-category4>.png")

# Descrizione delle immagini degli esempi per il few-shot prompting
example_image1_description = """The system features a controller-datapath structure.  
The system has six inputs (GO, D0, D1, D2, CK, -Reset) and four outputs (DFF, EFF, SER, TC).  
The FSM acts as the controller and needs to be designed.  
* **The FSM has:**
    * state variables X, Y, Z, and W;
    * outputs DFF and EFF;
    * inputs GO, D0, D1, D2, and TC.

The datapath consists of:  
* **E-PET Flip-flop**
    * Receives the clock signal (CK) common to all synchronous components.
    * The -Clear input of the flip-flop is connected to the -Reset signal.
    * The -Preset input of the flip-flop is constantly set to one. 
    * The D input is connected to the DFF signal coming from the FSM.
    * The E input is connected to the EFF signal coming from the FSM.
    * The output (Q) constitutes the system's output and is called SER. 
    * The inverted output (-Q) is not connected to anything. 

* **4-bit Universal Counter**
    * Receives the clock signal (CK) common to all synchronous components.
    * The count enable signal (En) is constantly set to one.
    * The TC enable signal (Et) is constantly set to one. 
    * The U/-D signal, which indicates the counting direction, is constantly set to zero.
    * Receives -Reset as the -Clear signal. 
    * The Load signal (LD) is connected to the TC output.
    * The TC output enters the FSM.
    * The value of inputs P3...P0 is constantly set to two.
    * The Q3...Q0 outputs are not connected to anything.
"""
example_image2_description = """The system features a controller - datapath structure.  
The system has three inputs (SER, CK, -Reset) and eight outputs (CMP, ENR, MM, NEG, Q0, Q1, CLR, RDY). The FSM acts as the controller and needs to be designed.

* **The FSM has:**
    * State variables X, Y, Z, and W;
    * Outputs E0, E1, NEG, MM, ENR, CLR, and RDY;
    * Inputs SER, Q0, Q1, and CMP.

The datapath consists of:

* **First E-PET Flip-Flop**
    * Receives the clock signal (CK) common to all synchronous components.
    * The -Clear input of the flip-flop is connected to the -Reset signal.
    * The -Preset input of the flip-flop is constant at one.
    * The D input is connected to the SER signal.
    * The E input is connected to the E0 signal coming from the FSM.
    * The Q output is a system output and is named Q0.
    * The negated output (-Q) is not connected to anything.

* **Second E-PET Flip-Flop**
    * Receives the clock signal (CK) common to all synchronous components.
    * The -Clear input of the flip-flop is connected to the -Reset signal.
    * The -Preset input of the flip-flop is constant at one.
    * The D input is connected to the SER signal.
    * The E input is connected to the E1 signal coming from the FSM.
    * The Q output is a system output and is named Q1.
    * The negated output (-Q) is not connected to anything.

* **4-bit PiPo Register**
    * Receives the clock signal (CK) common to all synchronous components.
    * The enable signal E is connected to the ENR signal coming from the FSM.
    * Inputs P0...P3 are connected to the outputs S0...S3 of the adder.
    * Outputs Q0...Q3 are connected to the B3...B0 inputs of the comparator and the A0...A3 inputs of the adder.
    * The initialization signal -CL is connected to the negated CLR signal coming from the FSM.

* **4-bit Adder**
    * Input B0 is constant at one. Inputs B1...B3 are connected together and depend on the NEG signal coming from the FSM.
    * Inputs A0...A3 are connected to the outputs Q0...Q3 of the PiPo register.
    * Outputs S0...S3 are connected to the inputs P0...P3 of the PiPo register.
    * The carry-in input Ci is constant at zero.

* **4-bit Comparator**
    * Inputs A3...A0 are connected together and depend on the MM signal coming from the FSM.
    * Inputs B3...B0 are connected to the outputs Q0...Q3 of the PiPo register.
    * The A=B output is fed back to the FSM as the CMP signal.

* **NOT Logic Gate**
   * Has the CLR signal, which comes from the FSM, as input.
   * The output is connected to the -CL input of the PiPo register.
"""
example_image3_description ="""The system features a controller - datapath structure.  
The system has three inputs (GO, CK, -Reset) and two outputs (PRS, SER). The FSM acts as the controller and needs to be designed.

* **The FSM has:**
    * State variables X, Y, Z, and W;
    * Outputs PRS, ENC, LDC, ENS, and STR;
    * Input GO.

The datapath consists of:

* **8-bit Universal Counter**
    * Receives the clock signal (CK) common to all synchronous components.
    * The count enable signal (En) is connected to the ENC signal coming from the FSM.
    * The TC enable signal (Et) is constant at one.
    * The U/-D signal, which indicates the counting direction, is constant at zero.
    * Receives -Reset as the -Clear signal.
    * The load signal (LD) is connected to the TC output.
    * The TC output is not connected to anything. 
    * Inputs P7...P0 are set to a constant value of 0Bh.
    * Outputs Q7...Q0 are connected to the inputs P7...P0 of the PiSo register.

* **8-bit PiSo Register**
    * Receives the clock signal (CK) common to all synchronous components.
    * The enable signal (E) is connected to the ENS signal coming from the FSM.
    * The load signal (LD) is connected to the LDC signal coming from the FSM.
    * Inputs P7...P0 are connected to the outputs Q7...Q0 of the counter.
    * The output Q0 is connected to the input of the EXOR logic gate.
    * Receives -Reset as the -Clear signal.

* **D-PET Flip-Flop**
    * Receives the clock signal (CK) common to all synchronous components.
    * The -Clear input of the flip-flop is connected to the -Reset signal.
    * The -Preset input of the flip-flop is constant at one.
    * The D input is connected to the output of the EXOR logic gate.
    * The Q output constitutes the system output and is named SER.
    * The negated output (-Q) is not connected to anything.

* **EXOR Logic Gate**
    * Its two inputs are connected to:
        * Output Q0 of the PiSo register.
        * STR signal coming from the FSM.
    * The output of the EXOR gate is connected to the D input of the D-PET flip-flop and is named D.
"""

# Soluzioni degli esempi per il few-shot prompting
solution1 = """+-------+---+---+---+---+----+----+----+----+----+-----+-----+-----------------+----+----+----+----+
| State | W | Z | Y | X | GO | D0 | D1 | D2 | TC | DFF | EFF |    Next State    | iW | iZ | iY | iX |
+-------+---+---+---+---+----+----+----+----+----+-----+-----+-----------------+----+----+----+----+
|  rs   | 0 | 0 | 0 | 0 |  0 |  - |  - |  - |  - |  0  |  0  |        rs        |  0 |  0 |  0 |  0 |
|  rs   | 0 | 0 | 0 | 0 |  1 |  - |  - |  - |  - |  0  |  0  |        a         |  0 |  0 |  0 |  1 |
|  a    | 0 | 0 | 0 | 1 |  0 |  - |  - |  - |  - |  0  |  0  |        ws        |  0 |  0 |  1 |  0 |
|  a    | 0 | 0 | 0 | 1 |  1 |  - |  - |  - |  - |  0  |  0  |        a         |  0 |  0 |  0 |  1 |
|  ws   | 0 | 0 | 1 | 0 |  - |  - |  - |  - |  0 |  0  |  0  |        ws        |  0 |  0 |  1 |  0 |
|  ws   | 0 | 0 | 1 | 0 |  - |  - |  - |  - |  1 |  0  |  0  |        st        |  0 |  0 |  1 |  1 |
|  st   | 0 | 0 | 1 | 1 |  - |  - |  - |  - |  - |  1  |  1  |        w0        |  0 |  1 |  0 |  0 |
|  w0   | 0 | 1 | 0 | 0 |  - |  - |  - |  - |  0 |  0  |  0  |        w0        |  0 |  1 |  0 |  0 |
|  w0   | 0 | 1 | 0 | 0 |  - |  0 |  - |  - |  1 |  0  |  0  |        n0        |  0 |  1 |  0 |  1 |
|  w0   | 0 | 1 | 0 | 0 |  - |  1 |  - |  - |  1 |  0  |  0  |        d0        |  0 |  1 |  1 |  0 |
|  n0   | 0 | 1 | 0 | 1 |  - |  - |  - |  - |  - |  0  |  1  |        w1        |  0 |  1 |  1 |  1 |
|  d0   | 0 | 1 | 1 | 0 |  - |  - |  - |  - |  - |  1  |  1  |        w1        |  0 |  1 |  1 |  1 |
|  w1   | 0 | 1 | 1 | 1 |  - |  - |  - |  - |  0 |  0  |  0  |        w1        |  0 |  1 |  1 |  1 |
|  w1   | 0 | 1 | 1 | 1 |  - |  - |  0 |  - |  1 |  0  |  0  |        n1        |  1 |  0 |  0 |  0 |
|  w1   | 0 | 1 | 1 | 1 |  - |  - |  1 |  - |  1 |  0  |  0  |        d1        |  1 |  0 |  0 |  1 |
|  n1   | 1 | 0 | 0 | 0 |  - |  - |  - |  - |  - |  0  |  1  |        w2        |  1 |  0 |  1 |  0 |
|  d1   | 1 | 0 | 0 | 1 |  - |  - |  - |  - |  - |  1  |  1  |        w2        |  1 |  0 |  1 |  0 |
|  w2   | 1 | 0 | 1 | 0 |  - |  - |  - |  - |  0 |  0  |  0  |        w2        |  1 |  0 |  1 |  0 |
|  w2   | 1 | 0 | 1 | 0 |  - |  - |  - |  0 |  1 |  0  |  0  |        n2        |  1 |  0 |  1 |  1 |
|  w2   | 1 | 0 | 1 | 0 |  - |  - |  - |  1 |  1 |  0  |  0  |        d2        |  1 |  1 |  0 |  0 |
|  n2   | 1 | 0 | 1 | 1 |  - |  - |  - |  - |  - |  0  |  1  |        w3        |  1 |  1 |  0 |  1 |
|  d2   | 1 | 1 | 0 | 0 |  - |  - |  - |  - |  - |  1  |  1  |        w3        |  1 |  1 |  0 |  1 |
|  w3   | 1 | 1 | 0 | 1 |  - |  - |  - |  - |  0 |  0  |  0  |        w3        |  1 |  1 |  0 |  1 |
|  w3   | 1 | 1 | 0 | 1 |  - |  - |  - |  - |  1 |  0  |  0  |        n3        |  1 |  1 |  1 |  0 |
|  n3   | 1 | 1 | 1 | 0 |  - |  - |  - |  - |  - |  0  |  1  |        sp        |  1 |  1 |  1 |  1 |
|  sp   | 1 | 1 | 1 | 1 |  - |  - |  - |  - |  0 |  0  |  0  |        sp        |  1 |  1 |  1 |  1 |
|  sp   | 1 | 1 | 1 | 1 |  - |  - |  - |  - |  1 |  0  |  0  |        rs        |  0 |  0 |  0 |  0 |
+-------+---+---+---+---+----+----+----+----+----+-----+-----+-----------------+----+----+----+----+
"""
solution2 = """+-------+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
| State | W | Z | Y | X | SER | Q0| Q1|CMP| E0| E1|NEG| MM|ENR|CLR|RDY|    Next State   | iW| iZ| iY| iX|
+-------+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 |  0  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   a   | 0 | 0 | 0 | 0 |  1  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        b        | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 1 |  -  | - | - | - | 1 | 0 | 0 | 0 | 0 | 0 | 0 |        c        | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 1 | 0 |  -  | - | - | - | 0 | 1 | 0 | 0 | 0 | 0 | 0 |        s        | 1 | 0 | 1 | 0 |
|   s   | 1 | 0 | 1 | 0 |  0  | 0 | 0 | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   s   | 1 | 0 | 1 | 0 |  0  | 1 | 0 | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        e        | 0 | 1 | 0 | 0 |
|   s   | 1 | 0 | 1 | 0 |  0  | 0 | 1 | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        f        | 0 | 1 | 0 | 1 |
|   s   | 1 | 0 | 1 | 0 |  0  | 1 | 1 | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        h        | 0 | 1 | 1 | 0 |
|   s   | 1 | 0 | 1 | 0 |  1  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        d        | 0 | 0 | 1 | 1 |
|   d   | 0 | 0 | 1 | 1 |  0  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   d   | 0 | 0 | 1 | 1 |  1  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        d        | 0 | 0 | 1 | 1 |
|   e   | 0 | 1 | 0 | 0 |  -  | - | - | - | 0 | 0 | 0 | 0 | 0 | 1 | 0 |        p        | 1 | 0 | 0 | 1 |
|   f   | 0 | 1 | 0 | 1 |  -  | - | - | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |        g        | 0 | 1 | 1 | 1 |
|   f   | 0 | 1 | 0 | 1 |  -  | - | - | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   h   | 0 | 1 | 1 | 0 |  -  | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        m        | 1 | 0 | 0 | 0 |
|   h   | 0 | 1 | 1 | 0 |  -  | - | - | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   g   | 0 | 1 | 1 | 1 |  -  | - | - | - | 0 | 0 | 0 | 0 | 1 | 0 | 0 |        p        | 1 | 0 | 0 | 1 |
|   m   | 1 | 0 | 0 | 0 |  -  | - | - | - | 0 | 0 | 1 | 0 | 1 | 0 | 0 |        p        | 1 | 0 | 0 | 1 |
|   p   | 1 | 0 | 0 | 1 |  -  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 1 |        a        | 0 | 0 | 0 | 0 |
+-------+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
"""
solution3 = """+-------+---+---+---+---+----+----+----+----+----+----+-----------------+---+---+---+---+
| State | W | Z | Y | X | GO | PRS| ENC| LDC| ENS| STR|   Next State    | iW| iZ| iY| iX|
+-------+---+---+---+---+----+----+----+----+----+----+-----------------+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 |  0 |  0 |  1 |  0 |  0 |  0 |        a        | 0 | 0 | 0 | 0 |
|   a   | 0 | 0 | 0 | 0 |  1 |  0 |  1 |  0 |  0 |  0 |        b        | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 1 |  0 |  1 |  0 |  0 |  0 |  0 |        st       | 0 | 0 | 1 | 0 |
|   b   | 0 | 0 | 0 | 1 |  1 |  1 |  0 |  0 |  0 |  0 |        b        | 0 | 0 | 0 | 1 |
|  st   | 0 | 0 | 1 | 0 |  - |  0 |  0 |  1 |  0 |  1 |        d0       | 0 | 0 | 1 | 1 |
|  d0   | 0 | 0 | 1 | 1 |  - |  0 |  0 |  0 |  1 |  0 |        d1       | 0 | 1 | 0 | 0 |
|  d1   | 0 | 1 | 0 | 0 |  - |  0 |  0 |  0 |  1 |  0 |        d2       | 0 | 1 | 0 | 1 |
|  d2   | 0 | 1 | 0 | 1 |  - |  0 |  0 |  0 |  1 |  0 |        d3       | 0 | 1 | 1 | 0 |
|  d3   | 0 | 1 | 1 | 0 |  - |  0 |  0 |  0 |  1 |  0 |        d4       | 0 | 1 | 1 | 1 |
|  d4   | 0 | 1 | 1 | 1 |  - |  0 |  0 |  0 |  1 |  0 |        d5       | 1 | 0 | 0 | 0 |
|  d5   | 1 | 0 | 0 | 0 |  - |  0 |  0 |  0 |  1 |  0 |        d6       | 1 | 0 | 0 | 1 |
|  d6   | 1 | 0 | 0 | 1 |  - |  0 |  0 |  0 |  1 |  0 |        d7       | 1 | 0 | 1 | 0 |
|  d7   | 1 | 0 | 1 | 0 |  - |  0 |  0 |  0 |  1 |  0 |        a        | 0 | 0 | 0 | 0 |
+-------+---+---+---+---+----+----+----+----+----+----+-----------------+---+---+---+---+

"""

# Richietse degli esempi per il few-shot prompting
prompt_text1 = f"""Consider the digital system represented in the figure {example_image1}, consisting of the controller (FSM), the "Cnt4" counter, and a flip-flop E-PET.

The task of the system (a serial transmitter) is to generate, on the SER output, through the E-PET flip-flop, a 5-bit packet consisting of a start bit '1', three data bits (in order: D0, D1, and D2), and a stop bit '0'. 
The duration N of each bit in the packet (the bit time) is a multiple of the clock CK period and is determined by the number P set at the P3..P0 inputs of the counter. 
The data to be transmitted, D0, D1, and D2, are available at the controller inputs. 
A falling edge on the GO input triggers the start of the packet generation.

Design the FSM as a synchronous Moore machine and create the state table for the FSM.
"""
prompt_text2 = f""" Consider the digital system represented in the figure {example_image2}, consisting of the controller (FSM) that receives data from a serial line and manages a network composed of two E-PET flip-flops, a register, and some arithmetic circuits.

The controller receives, on SER, a 4-bit packet composed of a start bit set to 1, two data bits (in the order D0 and D1), and a stop bit set to 0. The duration of the bit time is equal to the period of CK; the two data bits encode an operation that the system executes on the outputs N3..N0. 
If an incorrect stop bit (= 1) is received, no operations are performed, and the system waits for SER to return to zero before waiting for the next packet.
Upon receiving a valid packet (stop bit = 0), depending on the received value of D1 and D0, the system performs the following operations:

| D1 | D0 | Operation                     |
|----|----|-------------------------------|
|  0 |  0 | No Operation (NOP)            |
|  0 |  1 | Clear the output N3..N0       |
|  1 |  0 | Increment the output N3..N0   |
|  1 |  1 | Decrement the output N3..N0   |

The output N3..N0 is not incremented when it has reached the maximum value, nor decremented when it has reached the minimum. 
The RDY output is activated for one clock cycle upon receiving commands, except for the NOP.
It is noted that the FSM stores the received bits D1 and D0 respectively in flip-flops Q1 and Q0, to retain the received values even after checking the stop bit; the NEG output of the FSM allows the Add4 adder input to be set to two different values, while the MM output enables the Cp4 comparator to compare the number N3..N0 with two different values.

Design the FSM as a synchronous Moore machine and create the state table for the FSM.
"""
prompt_text3 = f"""Consider the digital system represented in figure {example_image3}, consisting of the controller (FSM), an 8-bit counter "Cnt8", an 8-bit shift register "PiSo8", a D-PET flip-flop, and some logic gates.

Upon pressing the GO button, the system transmits a serial signal on SER, consisting of a start bit, 8 data bits, and a stop bit. 
The 8 data bits to be transmitted are taken from the outputs of the counter.

When at rest, with the button not pressed (GO = 0), the counter is enabled via the ENC line; at all other times, counting is disabled. 
When the button is pressed (GO = 1), the FSM activates the PRS line.

Upon releasing the button, the controller begins the transmission process by loading the counter outputs into the shift register via the LDC line. 
Note that the STR line generates the start bit for one clock cycle, and the serial output SER is generated through the flip-flop.

Serialization is controlled by the FSM via the ENS line, which enables the shift register; after the SER generation is completed, the system returns to monitor the GO button.

Design the FSM as a synchronous Moore machine and create the state table for the FSM.
"""

# Costruzione dei prompt degli esempi per il few-shot prompting
input1 = f'{{"System": "{example_image1}", "Task": "{prompt_text1}", "System description": "{example_image1_description}"}}'
input2 = f'{{"System": "{example_image2}", "Task": "{prompt_text2}", "System description": "{example_image2_description}"}}'
input3 = f'{{"System": "{example_image3}", "Task": "{prompt_text3}", "System description": "{example_image3_description}"}}'

# Recupero esplicito delle informazioni sull'argomento (direct retrieval)
retrieval_text = """What is a finite state machine (FSM)?
What is a controller-datapath structure?
How do you design a system using a controller-datapath structure?
What are PET flip-flops and what types are there?
What is meant by a counter, SiPo register, PiPo register, PiSo register, adder, and comparator?"""

text_retrieved = retrieveInformationFromCustomDataSource(retrieval_text)

# Libro di testo sull'argomento
textbook = Part.from_uri(mime_type="application/pdf", uri="gs://<bucket-name-en>/<textbook-file>.pdf")

--------Text Retrieved--------


As we have seen with the shift register, the number of states in data management
systems depends on all the possible configurations that the data can take on. For
example, an 8-bit shift register will be described by an algorithm with at least 256
states. To describe a microprocessor in terms of states, one needs to only observe
that a single 32-bit register (a microprocessor could contain many of them) would
require 232 states.
Regular digital structures for data management are available as blocks. They can be
combinational (e.g., multiplexer, demultiplexer, encoders, decoders, and arithmetic
circuits) or sequential (registers, counters, memories, and many others). Although
the FSM can do arithmetic and logical operations, they can be done more effectively
by dedicated devices.
© Springer International Publishing AG, part of Springer Nature 2019
G. Donzellini et al., Introduction to Digital Systems Design,
https://doi.org/10.1007/978-3-319-92804-3_8
36

# Esercizio 16

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image16>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video16>.mp4")

image_description = """The system features a controller-datapath structure.  
The system has two inputs (LN, CK) and two outputs (LNA, LNB).  
The FSM (Finite State Machine) serves as the controller and needs to be designed.

* **The FSM has:**
  * state variables X, Y, Z, W, and K;
  * outputs LNA, LNB, E0, E1, and E2;
  * inputs LN, P, Q0, Q1, and Q2.

The datapath consists of:

* **First E-PET flip-flop**
  * Receives the clock signal (CK), which is common to all flip-flops.
  * The D input is connected to the LN input signal.
  * The output (Q) is fed into an EXOR logic gate and into the FSM. The output Q is called Q0.
  * The E input is connected to the E0 output of the FSM.

* **Second E-PET flip-flop**
  * Receives the clock signal (CK), which is common to all flip-flops.
  * The D input is connected to the LN input signal.
  * The output (Q) is fed into an EXOR logic gate and into the FSM. The output Q is called Q1.
  * The E input is connected to the E1 output of the FSM.

* **Third E-PET flip-flop**
  * Receives the clock signal (CK), which is common to all flip-flops.
  * The D input is connected to the LN input signal.
  * The output (Q) is fed into the FSM. The output Q is called Q2.
  * The E input is connected to the E2 output of the FSM.

* **EXOR logic gate**
  * Its two inputs are connected respectively to:
    * Output (Q) of the first E-PET flip-flop, called Q0.
    * Output (Q) of the second E-PET flip-flop, called Q1.
  * The output of the EXOR gate is fed into the FSM and is called P.
"""

prompt_text = f"""The goal is to design a synchronous digital system that receives and retransmits serial data.

The input LN receives a sequence of bits from an external source, synchronized with the clock, grouped into packets of five bits each.
Each packet starts with the first bit always set to '1' (start bit) and ends with the fifth bit always set to '0' (stop bit). These two bits do not carry information but are used to identify the position of the other 3 bits (B0, B1, B2), which contain the data carried by the sequence.
Each bit has a fixed duration, called the bit time. In our case, the bit time is equal to the clock period, and the rising edges of the clock correspond to the center of this interval. The LN input, when idle, has the value '0'. The device waits for the arrival of the start bit, then receives and processes all the other bits in the packet.

The figure {image} shows the architecture of the system, where the finite state machine (FSM) controls three E-PET flip-flops, which temporarily store the information bits received from the LN line (B0 is stored in Q0, B1 in Q1, and B2 in Q2). The finite state machine drives the two lines LNA and LNB, on which the serial output sequences will be generated.
The system checks that the received packet is valid, i.e., it ends correctly with the stop bit set to '0'. If this condition is not met, the system ignores the received data, does not activate the output lines, waits for the LN line to return to idle, and then waits for a new packet.

If the packet is valid, the finite state machine uses the value of Q2 to select the output line to transmit on (LNA if Q2 = '0', or LNB if Q2 = '1'). The transmitted sequence is: start bit - Q0 - Q1 - P - stop bit. Afterward, it is necessary to wait one clock cycle before receiving a new packet.
The bit P represents the parity calculated over Q0 and Q1 by the EXOR gate present in the system. The state variables are X, Y, Z, W, and K, the outputs are LNA, LNB, E0, E1, and E2, and the inputs are LN, P, Q0, Q1, and Q2.

Below is the description of the system:
{image_description}

Design the FSM as a  synchronous Moore machine and create the state table of the FSM.
"""

prompt_text_FW = f"""The goal is to design a synchronous digital system that receives and retransmits serial data.

The input LN receives a sequence of bits from an external source, synchronized with the clock, grouped into packets of five bits each.
Each packet starts with the first bit always set to '1' (start bit) and ends with the fifth bit always set to '0' (stop bit). These two bits do not carry information but are used to identify the position of the other 3 bits (B0, B1, B2), which contain the data carried by the sequence.
Each bit has a fixed duration, called the bit time. In our case, the bit time is equal to the clock period, and the rising edges of the clock correspond to the center of this interval. The LN input, when idle, has the value '0'. The device waits for the arrival of the start bit, then receives and processes all the other bits in the packet.

The figure {image} shows the architecture of the system, where the finite state machine (FSM) controls three E-PET flip-flops, which temporarily store the information bits received from the LN line (B0 is stored in Q0, B1 in Q1, and B2 in Q2). The finite state machine drives the two lines LNA and LNB, on which the serial output sequences will be generated.
The system checks that the received packet is valid, i.e., it ends correctly with the stop bit set to '0'. If this condition is not met, the system ignores the received data, does not activate the output lines, waits for the LN line to return to idle, and then waits for a new packet.

If the packet is valid, the finite state machine uses the value of Q2 to select the output line to transmit on (LNA if Q2 = '0', or LNB if Q2 = '1'). The transmitted sequence is: start bit - Q0 - Q1 - P - stop bit. Afterward, it is necessary to wait one clock cycle before receiving a new packet.
The bit P represents the parity calculated over Q0 and Q1 by the EXOR gate present in the system. The state variables are X, Y, Z, W, and K, the outputs are LNA, LNB, E0, E1, and E2, and the inputs are LN, P, Q0, Q1, and Q2.

Design the FSM as a  synchronous Moore machine and create the state table of the FSM.
"""

## 1.1 Esercizio senza ulteriore contesto

In [11]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## State Table for the Synchronous Moore Machine:

**Inputs:** LN, P, Q0, Q1, Q2
**Outputs:** LNA, LNB, E0, E1, E2
**State Variables:** X, Y, Z, W, K

| **Current State** | **LN** | **P** | **Q0** | **Q1** | **Q2** | **Next State** | **LNA** | **LNB** | **E0** | **E1** | **E2** | **Description**                                                                                                                                                                                |
|-------------------|:-------:|:-----:|:-------:|:-------:|:-------:|----------------:|--------|--------|-------|-------|-------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **S0 (Idle)**     |    0    |   X   |   X    |   X    |   X    |       S0       |    0   |    0   |   0    |   0    |   0    | Wait for start bit (LN=1)                    

## 1.2 Esercizio con recupero diretto

In [12]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## State Table for the Synchronous Moore Machine (FSM)

| **Current State** | **Inputs (LN, P, Q0, Q1, Q2)** | **Next State** | **Outputs (LNA, LNB, E0, E1, E2)** | **Description** |
|---|---|---|---|---|
| S0 (Idle) | 0, X, X, X, X | S0 | 0, 0, 0, 0, 0 | Wait for start bit (LN = 1) |
| S0 | 1, X, X, X, X | S1 | 0, 0, 0, 0, 0 | Start bit detected, move to S1 |
| S1 | X, X, X, X, X | S2 | 0, 0, 1, 1, 1 | Enable flip-flops to capture data bits |
| S2 | X, X, X, X, X | S3 | 0, 0, 0, 0, 0 | Data bits captured, move to S3 |
| S3 | X, X, X, X, X | S4 | 0, 0, 0, 0, 0 | Wait for stop bit |
| S4 | 0, X, X, X, 0 | S5 | 0, 1, 0, 0, 0 | Valid stop bit, Q2=0, transmit on LNB |
| S4 | 0, X, X, X, 1 | S6 | 1, 0, 0, 0, 0 | Valid stop bit, Q2=1, transmit on LNA |
| S4 | 1, X, X, X, X | S0 | 0, 0, 0, 0, 0 | Invalid stop bit, discard data, return to idle |
| S5 | X, X, X, X, X | S7 | 1, 0, 0, 0, 0 | Transmit start bit on LNB |
| S6 | X, X, X, X, X | S7 | 0, 1, 0, 0, 0 | Transmit 

## 1.3 Esercizio con recupero diretto e video soluzione

In [15]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
## State Table for the Serial Data Processor FSM

| State | K | W | Z | Y | X | LN | P | Q0 | Q1 | Q2 | LNA | LNB | E0 | E1 | E2 | Next State | KWZYX |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 | 0 | 0 | 0 | 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | a | 00000 |
| a | 0 | 0 | 0 | 0 | 0 | 1 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | b | 00001 |
| b | 0 | 0 | 0 | 0 | 1 | - | - | - | - | - | 0 | 0 | 1 | 0 | 0 | c | 00010 |
| c | 0 | 0 | 0 | 1 | 0 | - | - | - | - | - | 0 | 0 | 0 | 1 | 0 | d | 00100 |
| d | 0 | 0 | 1 | 0 | 0 | - | - | - | - | - | 0 | 0 | 0 | 0 | 1 | f | 01000 |
| f | 0 | 1 | 0 | 0 | 0 | 0 | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | g | 10000 |
| f | 0 | 1 | 0 | 0 | 0 | 0 | - | - | - | 1 | 0 | 0 | 0 | 0 | 0 | h | 10001 |
| f | 0 | 1 | 0 | 0 | 0 | 1 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | e | 10001 |
| e | 0 | 1 | 0 | 0 | 1 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | a | 10000 |
| e | 0 | 1 | 0 | 0 | 1 | 1 | - | - | - | - | 0 | 0

## 1.4 Esercizio con video soluzione

In [22]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The video provides a complete state table for the FSM, however, it does not explicitly name the states with letters. It uses numbers to identify them. Here's the state table derived from the video, with state names added for clarity:

**State Table**

| State | K W Z Y X | LN | P | Q0 | Q1 | Q2 | LNA | LNB | E0 | E1 | E2 | Next State | K' W' Z' Y' X' |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| A | 0 0 0 0 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | A | 0 0 0 0 0 |
| A | 0 0 0 0 0 | 1 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | B | 0 0 0 0 1 |
| B | 0 0 0 0 1 | - | - | - | - | - | 0 | 0 | 1 | 0 | 0 | C | 0 0 0 1 0 |
| C | 0 0 0 1 0 | - | - | - | - | - | 0 | 0 | 0 | 1 | 0 | D | 0 0 0 1 1 |
| D | 0 0 0 1 1 | - | - | - | - | - | 0 | 0 | 0 | 0 | 1 | F | 0 0 1 0 0 |
| F | 0 0 1 0 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | E | 0 0 1 0 1 |
| F | 0 0 1 0 0 | 1 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | E | 0 0 1 0 1 |
| E | 0 0 1 0 1 | 0 | - | - | - | - | 0 | 0 

## 1.5 Esercizio con libro di testo nel prompt

In [17]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## State Table of the FSM

**Inputs:** LN, P, Q0, Q1, Q2

**Outputs:** LNA, LNB, E0, E1, E2

**State Variables:** X, Y, Z, W, K

**State Table:**

| State | X Y Z W K | LN | P | Q0 | Q1 | Q2 | LNA | LNB | E0 | E1 | E2 | Next State | Description |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| S0 | 0 0 0 0 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | S0 | Idle, waiting for start bit |
| S1 | 0 0 0 0 1 | 1 | - | - | - | - | 0 | 0 | 1 | 1 | 1 | S2 | Start bit detected, enable flip-flops |
| S2 | 0 0 0 1 0 | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | S3 | Receive B0, disable flip-flops |
| S3 | 0 0 0 1 1 | - | - | - | 1 | 0 | 0 | 0 | 0 | 0 | 0 | S4 | Receive B1, disable flip-flops |
| S4 | 0 0 1 0 0 | - | - | - | - | 1 | 0 | 0 | 0 | 0 | 0 | S5 | Receive B2, disable flip-flops |
| S5 | 0 0 1 0 1 | - | - | - | - | - | 0 | 0 | 0 | 0 | 0 | S6 | Check stop bit |
| S6 | 0 1 0 0 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | S7 | Valid stop bit, prepare for transmis

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [18]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
The provided text describes the design of a synchronous digital system for receiving and retransmitting serial data. The system includes a finite state machine (FSM) as the controller and a datapath consisting of three E-PET flip-flops and an XOR gate.

Here's the state table of the FSM designed as a synchronous Moore machine:

| State | K W Z Y X | LN | P | Q2 | Q1 | Q0 | LNA | LNB | E0 | E1 | E2 | Next State |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| A | 0 0 0 0 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | A |
| A | 0 0 0 0 0 | 1 | - | - | - | - | 0 | 0 | 1 | 0 | 0 | B |
| B | 0 0 0 0 1 | - | - | - | - | - | 0 | 0 | 1 | 0 | 0 | C |
| C | 0 0 0 1 0 | - | - | - | - | - | 0 | 0 | 0 | 1 | 0 | D |
| D | 0 0 0 1 1 | - | - | - | - | - | 0 | 0 | 0 | 0 | 1 | E |
| E | 0 0 1 0 0 | 0 | - | 0 | - | - | 1 | 0 | 0 | 0 | 0 | G |
| E | 0 0 1 0 0 | 1 | - | 0 | - | - | 0 | 0 | 0 | 0 | 0 | E |
| E | 0 0 1 0 0 | 0 | - | 1 | - | - | 0 | 1 | 0 | 0 | 0 | H |
| E | 0 0

## 1.7 Esercizio con recupero mediante tool

In [21]:
prompt_text = f"""The goal is to design a synchronous digital system that receives and retransmits serial data.

The LN input receives a sequence of bits from an external source, synchronized with the clock, grouped into packets of five bits.
Each packet starts with the first bit always set to '1' (start bit) and ends with the fifth bit always set to '0' (stop bit).
These two bits do not carry information but serve to identify the position of the other 3 bits (B0, B1, B2), which contain the data carried by the sequence.
Each bit has a fixed duration, called bit time.
In our case, the bit time equals the clock period, and the rising edges of the clock correspond to the center of this interval.
The LN input, when idle, has the value '0'. The device waits for the arrival of the start bit and then receives and processes all the other bits in the packet.

Below is the description of the system:
{image_description}

In the system architecture described above, the finite state machine (FSM) controls three E-PET flip-flops, which temporarily store the information bits received from the LN line (B0 is stored in Q0, B1 in Q1, and B2 in Q2).
The finite state machine drives the two lines LNA and LNB, on which the serial output sequences will be generated.
The system checks that the received packet is valid, i.e., it ends correctly with the stop bit set to '0'. If this condition is not met, the system ignores the received data, does not activate the output lines, waits for the LN line to return to idle, and then waits for a new packet.
If the packet is valid, the finite state machine uses the value of Q2 to select the output line to transmit on (LNA if Q2 = '0', or LNB if Q2 = '1).
The transmitted sequence is: start bit - Q0 - Q1 - P - stop bit.
After that, it is necessary to wait one clock cycle before receiving a new packet.
The bit P represents the parity calculated over Q0 and Q1 by the EXOR gate present in the system.

The state variables are X, Y, Z, W, and K, the outputs are LNA, LNB, E0, E1, and E2, and the inputs are LN, P, Q0, Q1, and Q2.

Create the state table of the FSM."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

## State Table of the FSM

| Current State | LN | P | Q0 | Q1 | Q2 | Next State | LNA | LNB | E0 | E1 | E2 |
|---|---|---|---|---|---|---|---|---|---|---|---|
| **S0** | 0 | - | - | - | - | **S0** | 0 | 0 | 0 | 0 | 0 |
| **S0** | 1 | - | - | - | - | **S1** | 0 | 0 | 0 | 0 | 0 |
| **S1** | - | - | - | - | - | **S2** | 0 | 0 | 1 | 0 | 0 |
| **S2** | - | - | - | - | - | **S3** | 0 | 0 | 0 | 1 | 0 |
| **S3** | - | - | - | - | - | **S4** | 0 | 0 | 0 | 0 | 1 |
| **S4** | 0 | - | - | - | - | **S5** | 0 | 0 | 0 | 0 | 0 |
| **S4** | 1 | - | - | - | - | **S0** | 0 | 0 | 0 | 0 | 0 |
| **S5** | - | - | q0 | q1 | 0 | **S6** | 1 | 0 | 0 | 0 | 0 |
| **S5** | - | - | q0 | q1 | 1 | **S6** | 0 | 1 | 0 | 0 | 0 |
| **S6** | - | p | - | - | - | **S7** | q0 | q1 | 0 | 0 | 0 |
| **S7** | - | - | - | - | - | **S8** | p | p | 0 | 0 | 0 |
| **S8** | - | - | - | - | - | **S0** | 0 | 0 | 0 | 0 | 0 |

**Note:** 
* "-" indicates "don't care" - the input value doesn't matter for this transition.
* Lowercase letters 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "fsm serial data transmission"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "fsm serial data transmission"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [19]:
input = f'{{"System": "{image}", "Task": "{prompt_text_FW}", "System description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
+-------+---+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
| State | K | W | Z | Y | X |  LN | P | Q0| Q1| Q2| E0| E1| E2|LNA|LNB|   Next State    | iK| iW| iZ| iY| iX|
+-------+---+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 | 0 |  0  | - | - | - | - | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 | 0 |
|   a   | 0 | 0 | 0 | 0 | 0 |  1  | - | - | - | - | 0 | 0 | 0 | 0 | 0 |        b        | 0 | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 0 | 1 |  -  | - | - | - | - | 1 | 0 | 0 | 0 | 0 |        c        | 0 | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 0 | 1 | 0 |  -  | - | - | - | - | 0 | 1 | 0 | 0 | 0 |        d        | 0 | 0 | 1 | 0 | 0 |
|   d   | 0 | 0 | 1 | 0 | 0 |  -  | - | - | - | - | 0 | 0 | 1 | 0 | 0 |        e        | 0 | 1 | 0 | 0 | 0 |
|   e   | 0 | 1 | 0 | 0 | 0 |  -  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 |        h        | 0 | 1

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [20]:
input = f'{{"System": "{image}", "Task": "{prompt_text_FW}", "System description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
+-------+---+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
| State | K | W | Z | Y | X | LN | P |Q0|Q1|Q2|LNA|LNB|E0|E1|E2|    Next State   | iK| iW| iZ| iY| iX|
+-------+---+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 | 0 |  0  | - | - | - | - | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 | 0 |
|   a   | 0 | 0 | 0 | 0 | 0 |  1  | - | - | - | - | 0 | 0 | 0 | 0 | 0 |        b        | 0 | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 0 | 1 |  -  | - | - | - | - | 0 | 0 | 1 | 0 | 0 |        c        | 0 | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 0 | 1 | 0 |  -  | - | - | - | - | 0 | 0 | 0 | 1 | 0 |        d        | 0 | 0 | 0 | 1 | 1 |
|   d   | 0 | 0 | 0 | 1 | 1 |  -  | - | - | - | - | 0 | 0 | 0 | 0 | 1 |        e        | 0 | 0 | 1 | 0 | 0 |
|   e   | 0 | 0 | 1 | 0 | 0 |  -  | - | - | - | - | 0 | 0 | 0 | 0 | 0 |        f        | 0 | 0 | 1 | 0 | 

# Esercizio 17

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image17>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video17>.mp4")

image_description = """The system features a controller-datapath structure.
The system has three inputs (`IN`, `CK`, `-Reset`) and two outputs (`SER`, `BSY`).  
The two FSMs (FSM-A, FSM-B) act as controllers and need to be designed.

* **FSM-A has:**
  * state variables `X`, `Y`, and `Z`;
  * outputs `TRG`, `CL`, and `BSY`;
  * inputs `EN` and `ETX`;
  * clock signal `CK`;
  * initialization signal `-Reset`.
  
* **FSM-B has:**
  * state variables `X`, `Y`, `Z`, and `W`;
  * outputs `SER` and `ETX`;
  * inputs `TRG`, `Q0`, `Q1`, and `Q2`;
  * clock signal `CK`;
  * initialization signal `-Reset`.
  
The datapath consists of:

* **D-NET Flip-Flop**
  * Receives the clock signal (`CK`) common to all synchronous components.
  * The `D` input is connected to the `IN` input signal.
  * The output (`Q`) is fed into FSM-A and the counter (`CNT`) with the name `EN`.
  
* **Three-bit Counter (`CNT`)**
  * Receives the clock signal (`CK`) common to all synchronous components.
  * Receives `EN` as the enable signal.
  * Receives `CL` as the clear signal (`-Clear`).
  * The outputs `Q0`, `Q1`, and `Q2` are fed into FSM-B.
"""

prompt_text = f"""The goal is to design the digital system whose architecture is represented in the figure {image}. 
The system receives an input signal `IN`, normally at zero, on which pulses at a logic level of one appear, with durations varying from one to seven clock cycles of the clock signal `CK`.

The `D-net` flip-flop, visible at the top left, resynchronizes the received bit sequences on the falling edge of the clock.

The system measures the duration of the pulses in terms of clock periods and transmits the measured value on the serial line `SER`. To measure the pulse duration, the system uses the `CNT` counter. `CNT` is a three-bit binary counter with enable functionality. In the system, `CNT` is synchronized by the same clock `CK` as the finite state machine (FSM-A) and receives an asynchronous clear (`CL`) signal from it. FSM-A signals FSM-B by activating the `TRG` signal for one `CK` period, indicating that the pulse duration measurement is complete and the count value is available on the `Q2`, `Q1`, and `Q0` outputs of the counter. When necessary, FSM-A resets the `CNT` counter via the `CL` signal; additionally, for the entire time the system is engaged in receiving the pulse and transmitting on `SER`, FSM-A activates the `BSY` signal.

FSM-B starts generating the `SER` output when it receives the `TRG` pulse. The serial output data consists of a start bit set to '1', the three bits `Q0`, `Q1`, `Q2` (with the least significant bit (LSB) first) representing the pulse duration, and a stop bit set to '0'.  
Additionally, FSM-B generates the `ETX` output to inform FSM-A that the transmission on `SER` has ended.

Below is the description of the system:
{image_description}

Design FSM-A and FSM-B as synchronous Moore machines.
Create the state table for each.
"""

prompt_text_FW = f"""The goal is to design the digital system whose architecture is represented in the figure {image}. 
The system receives an input signal `IN`, normally at zero, on which pulses at a logic level of one appear, with durations varying from one to seven clock cycles of the clock signal `CK`.

The `D-net` flip-flop, visible at the top left, resynchronizes the received bit sequences on the falling edge of the clock.

The system measures the duration of the pulses in terms of clock periods and transmits the measured value on the serial line `SER`. To measure the pulse duration, the system uses the `CNT` counter. `CNT` is a three-bit binary counter with enable functionality. In the system, `CNT` is synchronized by the same clock `CK` as the finite state machine (FSM-A) and receives an asynchronous clear (`CL`) signal from it. FSM-A signals FSM-B by activating the `TRG` signal for one `CK` period, indicating that the pulse duration measurement is complete and the count value is available on the `Q2`, `Q1`, and `Q0` outputs of the counter. When necessary, FSM-A resets the `CNT` counter via the `CL` signal; additionally, for the entire time the system is engaged in receiving the pulse and transmitting on `SER`, FSM-A activates the `BSY` signal.

FSM-B starts generating the `SER` output when it receives the `TRG` pulse. The serial output data consists of a start bit set to '1', the three bits `Q0`, `Q1`, `Q2` (with the least significant bit (LSB) first) representing the pulse duration, and a stop bit set to '0'.  
Additionally, FSM-B generates the `ETX` output to inform FSM-A that the transmission on `SER` has ended.

Design FSM-A and FSM-B as synchronous Moore machines.
Create the state table for each.
"""

## 1.1 Esercizio senza ulteriore contesto

In [24]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## FSM-A State Table

| Present State | EN | ETX | Next State | TRG | CL  | BSY |
|--------------|----|-----|------------|-----|-----|-----|
| S0 (XYZ=000) | 0  | X   | S0        | 0   | 0   | 0   |
| S0 (XYZ=000) | 1  | X   | S1        | 0   | 1   | 1   |
| S1 (XYZ=001) | X  | 0   | S1        | 0   | 0   | 1   |
| S1 (XYZ=001) | X  | 1   | S2        | 1   | 0   | 1   |
| S2 (XYZ=010) | X  | X   | S0        | 0   | 0   | 0   |

**State Descriptions:**

* **S0 (Idle):** The system is idle, waiting for a pulse on `EN`.
* **S1 (Counting):** The system is counting clock cycles while `EN` is high.
* **S2 (Pulse End):** The system has detected the end of the pulse (`EN` is low) and triggers the transmission (`TRG`) to FSM-B.

**Output Logic:**

* **TRG:** 1 only when transitioning from S1 to S2 (pulse end detected).
* **CL:** 1 only when transitioning from S0 to S1 (start of a new pulse).
* **BSY:** 1 when in S1 or S2 (system is busy receiving/transmitting).


## FSM

## 1.2 Esercizio con recupero diretto

In [25]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## FSM-A State Table

| Present State | EN | ETX | Next State | TRG | CL   | BSY |
|--------------|----|------|-----------|------|-------|------|
| S0           | 0  | X    | S0        | 0    | 1     | 0    |
| S0           | 1  | X    | S1        | 0    | 0     | 1    |
| S1           | 0  | X    | S2        | 1    | 0     | 1    |
| S1           | 1  | X    | S1        | 0    | 0     | 1    |
| S2           | X  | 0    | S2        | 0    | 0     | 1    |
| S2           | X  | 1    | S0        | 0    | 1     | 0    |

**State Description:**

* **S0 (Idle):** The system is idle, waiting for a pulse on the `IN` line. The counter `CNT` is cleared, and `BSY` is deactivated.
* **S1 (Counting):** The system detects a pulse on `IN` and starts counting clock cycles using `CNT`. `BSY` is activated.
* **S2 (Wait for Transmission):** The pulse on `IN` has ended. The system activates `TRG` for one clock cycle to signal FSM-B that the count value is ready.  It waits for FS

## 1.3 Esercizio con recupero diretto e video soluzione

In [27]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The provided context does not contain information on how to complete the state tables for FSM-A and FSM-B. However, based on the provided image and the description of the system, here are the state tables for FSM-A and FSM-B:

**FSM-A State Table:**

| State | Z | Y | X | EN | ETX | TRG | CL | BSY | Next State |
|---|---|---|---|---|---|---|---|---|---|
| A | 0 | 0 | 0 | 0 | - | 0 | 1 | 0 | A |
| A | 0 | 0 | 0 | 1 | - | 0 | 1 | 0 | B |
| B | 0 | 0 | 1 | 0 | - | 0 | 0 | 1 | B |
| B | 0 | 0 | 1 | 1 | - | 0 | 0 | 1 | C |
| C | 0 | 1 | 0 | 0 | - | 0 | 0 | 1 | D |
| C | 0 | 1 | 0 | 1 | - | 0 | 0 | 1 | C |
| D | 0 | 1 | 1 | - | 0 | 1 | 0 | 1 | E |
| D | 0 | 1 | 1 | - | 1 | 1 | 0 | 1 | A |
| E | 1 | 0 | 0 | - | 0 | 0 | 0 | 1 | E |
| E | 1 | 0 | 0 | - | 1 | 0 | 1 | 0 | A |

**FSM-B State Table:**

| State | W | Z | Y | X | TRG | Q2Q1Q0 | SER | ETX | Next State |
|---|---|---|---|---|---|---|---|---|---|
| A | 0 | 0 | 0 | 0 | 0 | --- | 0 | 0 | A |
| A | 0 | 0 | 0 | 0 | 

## 1.4 Esercizio con video soluzione

In [28]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is designing a digital system with two finite state machines (FSM-A and FSM-B) to measure the duration of pulses in an input signal and transmit the measured value serially.

**FSM-A State Table:**

| State | Z | Y | X | EN | ETX | TRG | CL | BSY | Next State |
|---|---|---|---|---|---|---|---|---|---|
| A (000) | 0 | 0 | 0 | - | - | 0 | 1 | 0 | B |
| B (001) | 0 | 0 | 1 | 0 | - | 0 | 0 | 0 | B |
| B (001) | 0 | 0 | 1 | 1 | - | 0 | 0 | 0 | C |
| C (010) | 0 | 1 | 0 | 1 | - | 0 | 0 | 1 | C |
| C (010) | 0 | 1 | 0 | 0 | - | 1 | 0 | 1 | D |
| D (011) | 0 | 1 | 1 | - | 0 | 0 | 0 | 1 | E |
| D (011) | 0 | 1 | 1 | - | 1 | 0 | 0 | 1 | A |
| E (100) | 1 | 0 | 0 | - | 0 | 0 | 0 | 1 | E |
| E (100) | 1 | 0 | 0 | - | 1 | 0 | 0 | 1 | A |

**FSM-B State Table:**

| State | W | Z | Y | X | TRG | Q0 | Q1 | Q2 | SER | ETX | Next State |
|---|---|---|---|---|---|---|---|---|---|---|---|
| A (0000) | 0 | 0 | 0 | 0 | - | - | - | - | 0 | 0 | A |
| A (0000) | 0 | 0 | 0 | 0

## 1.5 Esercizio con libro di testo nel prompt

In [29]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## FSM-A State Table

| State Name | X | Y | Z | EN | ETX | TRG | CL | BSY |
|---|---|---|---|---|---|---|---|---|
| IDLE | 0 | 0 | 0 | 0 | - | 0 | 0 | 0 |
| IDLE | 0 | 0 | 0 | 1 | - | 0 | 0 | 0 |
| COUNT | 0 | 1 | 0 | 1 | - | 0 | 0 | 1 |
| COUNT | 0 | 1 | 0 | 0 | - | 1 | 1 | 1 |
| WAIT | 1 | 0 | 0 | - | 0 | 0 | 0 | 1 |
| WAIT | 1 | 0 | 0 | - | 1 | 0 | 0 | 0 |

**Description:**

* **IDLE:** The system is waiting for a pulse on the `IN` line. When `EN` becomes '1', indicating a pulse, the FSM transitions to the `COUNT` state.
* **COUNT:** The counter `CNT` is enabled and counts the duration of the pulse. When `EN` becomes '0', indicating the end of the pulse, the FSM sets `TRG` to '1' for one clock cycle to signal FSM-B and clears the counter (`CL`='1'). The FSM then transitions to the `WAIT` state.
* **WAIT:** The FSM waits for the `ETX` signal from FSM-B, indicating the end of the serial transmission. Once `ETX` becomes '1', the FSM returns to the `IDLE` state

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [30]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## State Table for FSM-A

| State | ZYX | EN | ETX | TRG | CL | BSY | Next State |
|---|---|---|---|---|---|---|---|
| A | 000 | 0 | - | 0 | 1 | 0 | A |
| A | 000 | 1 | - | 0 | 1 | 0 | B |
| B | 001 | - | - | 0 | 0 | 1 | C |
| C | 010 | 0 | - | 0 | 0 | 1 | C |
| C | 010 | 1 | - | 0 | 0 | 1 | C |
| C | 010 | - | 0 | 0 | 0 | 1 | C |
| C | 010 | - | 1 | 1 | 0 | 1 | D |
| D | 011 | - | - | 0 | 0 | 1 | E |
| E | 100 | - | - | 0 | 1 | 0 | A |


## State Table for FSM-B

| State | WXYZ | TRG | Q2Q1Q0 | SER | ETX | Next State |
|---|---|---|---|---|---|---|
| a | 0000 | 0 | --- | 0 | 0 | a |
| a | 0000 | 1 | --- | 1 | 0 | b |
| b | 0001 | - | --- | 1 | 0 | c |
| c | 0010 | - | --- | Q0 | 0 | d |
| d | 0011 | - | --- | Q1 | 0 | e |
| e | 0100 | - | --- | Q2 | 0 | f |
| f | 0101 | - | --- | 0 | 0 | g |
| g | 0110 | - | --- | 0 | 1 | a |

**Explanation of State Tables:**

**FSM-A:**

* **State A (000):** This is the initial/idle state. The counter is cleared (CL=1), BSY i

## 1.7 Esercizio con recupero mediante tool

In [33]:
prompt_text = f"""The goal is to design a digital system with the architecture described below:
{image_description}

The system receives an input signal IN, which is normally at zero, and on which pulses at a logic level one appear, with a duration varying from one to seven periods of the clock signal CK.

The D-net flip-flop, visible at the top left, resynchronizes the received bit sequences on the falling edge of the clock.

The system measures the duration of the pulses in terms of clock periods and transmits the measured value on the serial line SER. The system uses the counter CNT to measure the pulse duration. CNT is a 3-bit binary counter with an enable signal. In the system, CNT is synchronized by the same clock CK as the finite state machine FSM-A and receives the asynchronous clear signal CL from FSM-A. FSM-A signals to FSM-B, by activating the TRG signal for one CK period, that the pulse duration measurement is complete and the count value is available on the counter outputs Q2, Q1, and Q0. When necessary, FSM-A clears the counter CNT via the CL signal; additionally, during the time the system is busy receiving the pulse and transmitting on SER, FSM-A activates BSY.

FSM-B starts generating the SER output when it receives the TRG pulse. The serial data output consists of a start bit at '1', the three bits Q0, Q1, Q2 (LSB first) representing the pulse duration, and a stop bit at '0'.
Additionally, FSM-B generates the ETX output to inform FSM-A that the transmission on SER is complete.

Design FSM-A and FSM-B and create a state table for each."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

## FSM-A: Pulse Measurement Controller

**State Table:**

| Current State | EN  | ETX | Next State | TRG | CL  | BSY |
|--------------|-----|------|-----------|-----|-----|-----|
| IDLE         | 0    | 0    | COUNT    | 0    | 0    | 0    |
| COUNT        | 1    | 0    | COUNT    | 0    | 0    | 1    |
| COUNT        | 1    | 0    | SEND     | 0    | 0    | 1    |
| SEND         | 0    | 0    | WAIT     | 1    | 1    | 1    |
| WAIT         | 0    | 1    | IDLE     | 0    | 0    | 0    |

**State Description:**

* **IDLE:**  The system is idle, waiting for a pulse on input `IN`. `EN` is '0', keeping the counter disabled.
* **COUNT:** The system detects a pulse on `IN` and starts counting by enabling the counter (`EN` = '1'). `BSY` is set to '1' to indicate the system is busy.
* **SEND:** The pulse on `IN` ends. The system disables the counter (`EN` = '0') and prepares to send the measured value.
* **WAIT:** The system sends a pulse on `TRG` to signal FSM-B to start transmitting the co

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "FSM-A and FSM-B design for pulse duration measurement and transmission"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "FSM-A and FSM-B design for pulse duration measurement and transmission"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [31]:
input = f'{{"System": "{image}", "Task": "{prompt_text_FW}", "System description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
## FSM-A State Table:

| State | X | Y | Z | EN | ETX | TRG | CL  | BSY | Next State | iX | iY | iZ |
|-------|---|---|---|----|-----|-----|-----|-----|-----------|---|---|---|
|  S0   | 0 | 0 | 0 |  0  |  -  |  0  |  1  |  0  |    S0     | 0 | 0 | 0 |
|  S0   | 0 | 0 | 0 |  1  |  -  |  0  |  1  |  1  |    S1     | 0 | 0 | 1 |
|  S1   | 0 | 0 | 1 |  -  |  -  |  0  |  0  |  1  |    S1     | 0 | 0 | 1 |
|  S1   | 0 | 0 | 1 |  0  |  -  |  1  |  1  |  1  |    S2     | 0 | 1 | 0 |
|  S2   | 0 | 1 | 0 |  -  |  0  |  0  |  0  |  1  |    S2     | 0 | 1 | 0 |
|  S2   | 0 | 1 | 0 |  -  |  1  |  0  |  0  |  0  |    S0     | 0 | 0 | 0 |

**State Variables:**
* X, Y, Z: Represent the current state of FSM-A.

**Inputs:**
* EN: Indicates the presence of a pulse (1) or its absence (0).
* ETX: Signals the end of transmission from FSM-B.

**Outputs:**
* TRG: Triggers FSM-B to start transmission.
* CL: Clears the counter (CNT).
* BSY: Indicates if the system is busy processing a 

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [32]:
input = f'{{"System": "{image}", "Task": "{prompt_text_FW}", "System description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
## FSM-A State Table

| State | Z | Y | X | EN | ETX | TRG | CL | BSY | Next State | iZ | iY | iX |
|---|---|---|---|---|---|---|---|---|---|---|---|
| A | 0 | 0 | 0 | 0 | - | 0 | 1 | 0 | A | 0 | 0 | 0 |
| A | 0 | 0 | 0 | 1 | - | 0 | 1 | 0 | B | 0 | 0 | 1 |
| B | 0 | 0 | 1 | 0 | - | 0 | 0 | 1 | B | 0 | 0 | 1 |
| B | 0 | 0 | 1 | 1 | - | 0 | 0 | 1 | C | 0 | 1 | 0 |
| C | 0 | 1 | 0 | - | - | 0 | 0 | 1 | C | 0 | 1 | 0 |
| C | 0 | 1 | 0 | - | - | 1 | 0 | 1 | D | 0 | 1 | 1 |
| D | 0 | 1 | 1 | - | 0 | 1 | 0 | 1 | D | 0 | 1 | 1 |
| D | 0 | 1 | 1 | - | 1 | 1 | 0 | 1 | A | 0 | 0 | 0 |

## FSM-B State Table

| State | W | Z | Y | X | TRG | Q0 | Q1 | Q2 | SER | ETX | Next State | iW | iZ | iY | iX |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 | 0 | 0 | 0 | 0 | - | - | - | 0 | 0 | a | 0 | 0 | 0 | 0 |
| a | 0 | 0 | 0 | 0 | 1 | - | - | - | 0 | 0 | b | 0 | 0 | 0 | 1 |
| b | 0 | 0 | 0 | 1 | - | - | - | - | 1 | 0 | c | 0 | 0 | 1 | 0 |
| c | 0 | 0 | 1 | 

# Esercizio 18

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image18>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video18>.mp4")

image_description = """The system features a controller-datapath structure.  
The system has three inputs (LN, CK, -Reset) and two outputs (PS, SYN). The two FSMs (RX FSM, PULSE FSM) serve as controllers and need to be designed.

* **The RX FSM has:**
    * State variables X, Y, and Z;
    * Outputs GO, J0, K0, J1, K1, J2, and K2;
    * Input LS;
    * Clock signal CK;
    * Initialization signal -Reset.

* **The PULSE FSM has:**
    * State variables X, Y, and Z;
    * Outputs PS and SYN;
    * Inputs GO, P0, P1, and P2;
    * Clock signal CK;
    * Initialization signal -Reset.

The datapath consists of:

* **D-NET Flip-Flop**
    * Receives the clock signal (CK) common to all synchronous components.
    * The D input is connected to the LN input signal.
    * The output (Q) is connected to the input of the RX FSM and is named LS.

* **First JK-PET Flip-Flop**
    * Receives the clock signal (CK) common to all synchronous components.
    * The J input is connected to the J0 signal coming from the RX FSM.
    * The K input is connected to the K0 signal coming from the RX FSM.
    * The output (Q) is connected to the D input of the first D-PET flip-flop. The output Q is named Q0.

* **Second JK-PET Flip-Flop**
    * Receives the clock signal (CK) common to all synchronous components.
    * The J input is connected to the J1 signal coming from the RX FSM.
    * The K input is connected to the K1 signal coming from the RX FSM.
    * The output (Q) is connected to the D input of the second D-PET flip-flop. The output Q is named Q1.

* **Third JK-PET Flip-Flop**
    * Receives the clock signal (CK) common to all synchronous components.
    * The J input is connected to the J2 signal coming from the RX FSM.
    * The K input is connected to the K2 signal coming from the RX FSM.
    * The output (Q) is connected to the D input of the third D-PET flip-flop. The output Q is named Q2.

* **First D-PET Flip-Flop**
    * The clock signal (CK) is connected to the GO signal coming from the RX FSM.
    * The D input is connected to the output of the first JK-PET flip-flop.
    * The output (Q) is connected to the input of the PULSE FSM and is named P0.

* **Second D-PET Flip-Flop**
    * The clock signal (CK) is connected to the GO signal coming from the RX FSM.
    * The D input is connected to the output of the second JK-PET flip-flop.
    * The output (Q) is connected to the input of the PULSE FSM and is named P1.

* **Third D-PET Flip-Flop**
    * The clock signal (CK) is connected to the GO signal coming from the RX FSM.
    * The D input is connected to the output of the third JK-PET flip-flop.
    * The output (Q) is connected to the input of the PULSE FSM and is named P2.
"""

prompt_text = f"""The goal is to design a synchronous digital system capable of receiving commands through a serial line and, consequently, generating pulses whose duration depends on the information contained within those commands. The figure {image} shows the architecture of the system, composed of two finite state machines and several flip-flops.

The commands consist of serial packets received through the LN input. The packet format includes a start bit, three data bits D0, D1, and D2, and finally two stop bits. The bit time corresponds to one clock cycle, and the bit change is synchronous with the rising edges of the clock CK.

The D-net flip-flop re-synchronizes the received bit sequences on the falling edge of the clock.

The RX FSM (Finite State Machine) is responsible for receiving and decoding the sequences re-synchronized by the flip-flop. During the reception of a new data packet, the RX FSM stores the three information bits one after the other in the three JK-pet flip-flops (D0 is stored in Q0, D1 in Q1, and D2 in Q2). If the received sequence is valid (i.e., the two stop bits are correctly received as zero), the RX FSM activates the GO signal for the duration of one clock cycle.
The activation of the GO signal also copies the received information to the P0, P1, and P2 flip-flops. The purpose of this operation is to allow the RX FSM to receive a new bit sequence immediately after.

The PULSE FSM waits for the GO signal to be activated. When GO is set to '1', the PULSE FSM begins generating a pulse on the PS output, the duration of which is a function of the outputs of flip-flops P0 and P1. The duration is defined as T = (2 + P1P0), where P1P0 is a binary number (P1 = MSB, P0 = LSB).
If the output of flip-flop P2 is '1', the PULSE FSM activates the SYN output during the last clock cycle of the PS pulse to indicate that the pulse generation is ending.

The asynchronous reset !Reset forces the entire system into a rest state, where it waits for the arrival of a start bit on the LN line.

Below is the description of the system:
{image_description}

Design the RX FSM as a synchronous Mealy machine and the PULSE FSM as a synchronous Moore machine.
Create the state table for each.
"""

prompt_text_FW = f"""The goal is to design a synchronous digital system capable of receiving commands through a serial line and, consequently, generating pulses whose duration depends on the information contained within those commands. The figure {image} shows the architecture of the system, composed of two finite state machines and several flip-flops.

The commands consist of serial packets received through the LN input. The packet format includes a start bit, three data bits D0, D1, and D2, and finally two stop bits. The bit time corresponds to one clock cycle, and the bit change is synchronous with the rising edges of the clock CK.

The D-net flip-flop re-synchronizes the received bit sequences on the falling edge of the clock.

The RX FSM (Finite State Machine) is responsible for receiving and decoding the sequences re-synchronized by the flip-flop. During the reception of a new data packet, the RX FSM stores the three information bits one after the other in the three JK-pet flip-flops (D0 is stored in Q0, D1 in Q1, and D2 in Q2). If the received sequence is valid (i.e., the two stop bits are correctly received as zero), the RX FSM activates the GO signal for the duration of one clock cycle.
The activation of the GO signal also copies the received information to the P0, P1, and P2 flip-flops. The purpose of this operation is to allow the RX FSM to receive a new bit sequence immediately after.

The PULSE FSM waits for the GO signal to be activated. When GO is set to '1', the PULSE FSM begins generating a pulse on the PS output, the duration of which is a function of the outputs of flip-flops P0 and P1. The duration is defined as T = (2 + P1P0), where P1P0 is a binary number (P1 = MSB, P0 = LSB).
If the output of flip-flop P2 is '1', the PULSE FSM activates the SYN output during the last clock cycle of the PS pulse to indicate that the pulse generation is ending.

The asynchronous reset !Reset forces the entire system into a rest state, where it waits for the arrival of a start bit on the LN line.

Design the RX FSM as a synchronous Mealy machine and the PULSE FSM as a synchronous Moore machine.
Create the state table for each.
"""

## 1.1 Esercizio senza ulteriore contesto

In [35]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## State Tables for the RX and PULSE FSMs

Here are the state tables for the RX FSM (Mealy Machine) and the PULSE FSM (Moore Machine):

**RX FSM (Mealy Machine)**

| Current State | Input (LS) | Next State | Outputs (GO, J0, K0, J1, K1, J2, K2) |
|--------------|------------|-----------|---------------------------------------|
| S0 (Idle)    | 0          | S0        | 0, X, X, X, X, X, X                   |
| S0 (Idle)    | 1          | S1        | 0, X, X, X, X, X, X                   |
| S1 (Start)   | 0          | S2        | 0, 1, 0, X, X, X, X                   |
| S1 (Start)   | 1          | S0        | 0, X, X, X, X, X, X                   |
| S2 (D0)     | 0          | S3        | 0, X, X, 1, 0, X, X                   |
| S2 (D0)     | 1          | S3        | 0, X, X, 0, 1, X, X                   |
| S3 (D1)     | 0          | S4        | 0, X, X, X, X, 1, 0                   |
| S3 (D1)     | 1          | S4        | 0, X, X, X, X, 0, 1               

## 1.2 Esercizio con recupero diretto

In [36]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## RX FSM (Mealy Machine)

**State Table:**

| Current State | Input (LS) | Next State | Outputs (GO, J0, K0, J1, K1, J2, K2) |
|--------------|------------|-----------|----------------------------------------|
| S0 (Reset)   | 0          | S0        | 0, 0, X, 0, X, 0, X                     |
| S0          | 1          | S1        | 0, 0, X, 0, X, 0, X                     |
| S1          | 0          | S2        | 0, 1, 1, 0, X, 0, X                     |
| S1          | 1          | S2        | 0, 0, 0, 0, X, 0, X                     |
| S2          | 0          | S3        | 0, X, X, 1, 1, 0, X                     |
| S2          | 1          | S3        | 0, X, X, 0, 0, 0, X                     |
| S3          | 0          | S4        | 0, X, X, X, X, 1, 1                     |
| S3          | 1          | S4        | 0, X, X, X, X, 0, 0                     |
| S4          | 0          | S5        | 0, X, X, X, X, X, X                     |
| S4          | 

## 1.3 Esercizio con recupero diretto e video soluzione

In [43]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The video you provided is in Italian and I am unable to understand it. I can help you design the RX and PULSE FSMs based on your provided description.

## RX FSM (Mealy Machine)

**Inputs:** LS (Line Signal)
**Outputs:** GO, J0, K0, J1, K1, J2, K2
**State Variables:** X, Y, Z

| State | ZYX | LS | GO | J0 K0 | J1 K1 | J2 K2 | Next State |
|---|---|---|---|---|---|---|---|
| A (Idle) | 000 | 0 | 0 | 0 0 | 0 0 | 0 0 | A |
| A (Idle) | 000 | 1 | 0 | 1 0 | 0 0 | 0 0 | B |
| B (D0) | 001 | 0 | 0 | 0 K0 | 1 0 | 0 0 | C |
| B (D0) | 001 | 1 | 0 | 1 K0 | 1 0 | 0 0 | C |
| C (D1) | 010 | 0 | 0 | 0 0 | 0 K1 | 1 0 | D |
| C (D1) | 010 | 1 | 0 | 0 0 | 1 K1 | 1 0 | D |
| D (D2) | 011 | 0 | 0 | 0 0 | 0 0 | 0 K2 | E |
| D (D2) | 011 | 1 | 0 | 0 0 | 0 0 | 1 K2 | E |
| E (Stop1) | 100 | 0 | 0 | 0 0 | 0 0 | 0 0 | F |
| E (Stop1) | 100 | 1 | 0 | 0 0 | 0 0 | 0 0 | G |
| F (Stop2) | 101 | 0 | 1 | 0 0 | 0 0 | 0 0 | A |
| F (Stop2) | 101 | 1 | 0 | 0 0 | 0 0 | 0 0 | G |
| G (Error) | 

## 1.4 Esercizio con video soluzione

In [44]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is designing a synchronous digital system that receives commands through a serial line and generates pulses based on the information in the commands. The system consists of two finite state machines (FSMs), RX FSM and PULSE FSM.

**RX FSM (Mealy Machine)**

The RX FSM is responsible for receiving and decoding the serial packets. It stores the three information bits (D0, D1, D2) in JK flip-flops and activates the GO signal if the received sequence is valid.

**State Table for RX FSM:**

| State | Z | Y | X | LS | GO | J0 | K0 | J1 | K1 | J2 | K2 | Next State |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| A | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | A |
| A | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | B |
| B | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | C |
| B | 0 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | C |
| C | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | D |
| C | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | D |
| D | 0 | 1 | 1 | 0 | 

## 1.5 Esercizio con libro di testo nel prompt

In [45]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## RX FSM (Mealy Machine)

**State Table:**

| Current State | LS | Next State | GO | J0 | K0 | J1 | K1 | J2 | K2 |
|---|---|---|---|---|---|---|---|---|---|
| S0 (Reset) | 0 | S0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 |
| S0 | 1 | S1 | 0 | 0 | 1 | 0 | 1 | 0 | 1 |
| S1 | 0 | S2 | 0 | 1 | 0 | 0 | 1 | 0 | 1 |
| S1 | 1 | S0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 |
| S2 | 0 | S3 | 0 | 0 | 1 | 1 | 0 | 0 | 1 |
| S2 | 1 | S0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 |
| S3 | 0 | S4 | 0 | 0 | 1 | 0 | 1 | 1 | 0 |
| S3 | 1 | S0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 |
| S4 | 0 | S5 | 0 | 0 | 1 | 0 | 1 | 0 | 1 |
| S4 | 1 | S0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 |
| S5 | 0 | S0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 |
| S5 | 1 | S0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 |

**State Description:**

* **S0 (Reset):** Initial state, waiting for the start bit.
* **S1:** Start bit received, waiting for D0.
* **S2:** D0 received, waiting for D1.
* **S3:** D1 received, waiting for D2.
* **S4:** D2 received, waiting for the first stop bit.
* **S5:** First s

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [46]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## RX FSM State Table (Mealy Machine)

| State | ZYX | LS | GO | J0 | K0 | J1 | K1 | J2 | K2 | Next State |
|---|---|---|---|---|---|---|---|---|---|---|
| a | 000 | 0 | 0 | 0 | - | 0 | - | 0 | - | a |
| a | 000 | 1 | 0 | 0 | - | 0 | - | 0 | - | b |
| b | 001 | 0 | 0 | - | 1 | 0 | - | 0 | - | c |
| b | 001 | 1 | 0 | - | 1 | 0 | - | 0 | - | c |
| c | 010 | 0 | 0 | 0 | - | - | 1 | 0 | - | d |
| c | 010 | 1 | 0 | 0 | - | - | 1 | 0 | - | d |
| d | 011 | 0 | 0 | 0 | - | 0 | - | - | 1 | e |
| d | 011 | 1 | 0 | 0 | - | 0 | - | - | 1 | e |
| e | 100 | 0 | 0 | 0 | - | 0 | - | 0 | - | f |
| e | 100 | 1 | 0 | 0 | - | 0 | - | 0 | - | a |
| f | 101 | 0 | 1 | 0 | - | 0 | - | 0 | - | a |
| f | 101 | 1 | 0 | 0 | - | 0 | - | 0 | - | a |

## PULSE FSM State Table (Moore Machine)

| State | ZYX | GO | P0 | P1 | P2 | PS | SYN | Next State |
|---|---|---|---|---|---|---|---|---|
| a | 000 | 0 | - | - | - | 0 | 0 | a |
| a | 000 | 1 | - | - | - | 0 | 0 | b |
| b | 001 | - | 0 | 0 | 

## 1.7 Esercizio con recupero mediante tool

In [49]:
prompt_text = f"""A synchronous digital system is to be designed that can receive commands through a serial line and, consequently, generate pulses whose duration depends on the information contained in those commands. The system architecture, consisting of two finite state machines and several flip-flops, is described below:
{image_description}

The commands consist of serial packets received through the LN input. The packet includes a start bit, three data bits D0, D1, and D2, and finally two stop bits. The bit time is equal to one clock cycle, and the bit change is synchronous with the rising edges of the clock CK.

The D-net flip-flop re-synchronizes the received bit sequences on the falling edge of the clock.

The RX FSM (Finite State Machine) is responsible for receiving and decoding the sequences re-synchronized by the flip-flop. During the reception of a new data packet, the RX FSM stores the three information bits one by one in the three JK-pet flip-flops (D0 is stored in Q0, D1 in Q1, and D2 in Q2). If the received sequence is valid (i.e., the two stop bits are correctly received as zero), the RX FSM activates the GO signal for the duration of one clock cycle.
The activation of the GO signal also copies the received information to the P0, P1, and P2 flip-flops. The purpose of this operation is to allow the RX FSM to receive a new bit sequence immediately afterward.

The PULSE FSM waits for the GO signal to be activated. When GO is set to '1', the PULSE FSM begins generating a pulse on the PS output, the duration of which is a function of the outputs of flip-flops P0 and P1. The duration is defined as T = (2 + P1P0), where P1P0 is a binary number (P1 = MSB, P0 = LSB).
If the output of flip-flop P2 is '1', the PULSE FSM activates the SYN output during the last clock cycle of the PS pulse to indicate that the pulse generation is ending.

The asynchronous reset !Reset forces the entire system into a rest state, where it waits for the arrival of a start bit on the LN line.

Design the RX FSM and the PULSE FSM and create the state table for each.
"""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

## RX FSM State Table

| Current State | Input (LS) | Next State | Outputs (GO, J0, K0, J1, K1, J2, K2) |
|---|---|---|---|---|
| IDLE | 0 | IDLE | 0, 0, 0, 0, 0, 0, 0 |
| IDLE | 1 | STB | 0, 0, 0, 0, 0, 0, 0 |
| STB | X | D0 | 0, 0, 0, 0, 0, 0, 0 |
| D0 | 0 | D1 | 0, 0, 1, 0, 0, 0, 0 |
| D0 | 1 | D1 | 0, 1, 1, 0, 0, 0, 0 |
| D1 | 0 | D2 | 0, 0, 0, 0, 1, 0, 0 |
| D1 | 1 | D2 | 0, 0, 0, 1, 1, 0, 0 |
| D2 | 0 | STP1 | 0, 0, 0, 0, 0, 0, 1 |
| D2 | 1 | STP1 | 0, 0, 0, 0, 0, 1, 1 |
| STP1 | 0 | STP2 | 0, 0, 0, 0, 0, 0, 0 |
| STP1 | 1 | IDLE | 0, 0, 0, 0, 0, 0, 0 |
| STP2 | 0 | IDLE | 1, 0, 0, 0, 0, 0, 0 |
| STP2 | 1 | IDLE | 0, 0, 0, 0, 0, 0, 0 |

## PULSE FSM State Table

| Current State | Inputs (GO, P1, P0) | Next State | Outputs (PS, SYN) |
|---|---|---|---|
| WAIT | 0, X, X | WAIT | 0, 0 |
| WAIT | 1, X, X | COUNT2 | 1, 0 |
| COUNT2 | X, 0, 0 | COUNT3 | 1, 0 |
| COUNT2 | X, 0, 1 | COUNT4 | 1, 0 |
| COUNT2 | X, 1, 0 | COUNT5 | 1, 0 |
| COUNT2 | X, 1, 1 | COUNT6 | 1, 0 |
| COUNT3 | X, X,

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "synchronizing digital system design"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "synchronizing digital system design"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [47]:
input = f'{{"System": "{image}", "Task": "{prompt_text_FW}", "System description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
## RX FSM (Mealy Machine)

**State Table:**

| Current State | LS | Next State | GO | J0 | K0 | J1 | K1 | J2 | K2 |
|---|---|---|---|---|---|---|---|---|---|
| S0 (Reset) | 0 | S0 | 0 | 0 | X | 0 | X | 0 | X |
| S0 (Reset) | 1 | S1 | 0 | 0 | X | 0 | X | 0 | X |
| S1 | 0 | S2 | 0 | 1 | 0 | 0 | X | 0 | X |
| S1 | 1 | S2 | 0 | 1 | 0 | 0 | X | 0 | X |
| S2 | 0 | S3 | 0 | 0 | X | 1 | 0 | 0 | X |
| S2 | 1 | S3 | 0 | 0 | X | 1 | 0 | 0 | X |
| S3 | 0 | S4 | 0 | 0 | X | 0 | X | 1 | 0 |
| S3 | 1 | S4 | 0 | 0 | X | 0 | X | 1 | 0 |
| S4 | 0 | S5 | 0 | 0 | X | 0 | X | 0 | X |
| S4 | 1 | S0 | 0 | 0 | X | 0 | X | 0 | X |
| S5 | 0 | S0 | 1 | 0 | X | 0 | X | 0 | X |
| S5 | 1 | S0 | 0 | 0 | X | 0 | X | 0 | X |

**State Description:**

* **S0 (Reset):** Initial state, waiting for start bit (LS=1).
* **S1:** Start bit received, ready to receive D0.
* **S2:** D0 received, ready to receive D1.
* **S3:** D1 received, ready to receive D2.
* **S4:** D2 received, checking first stop bit

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [48]:
input = f'{{"System": "{image}", "Task": "{prompt_text_FW}", "System description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
**RX FSM State Table:**

| State | Z | Y | X | LS | GO | J0 | K0 | J1 | K1 | J2 | K2 | Next State |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 | 0 | 0 | 0 | 0 | 0 | - | 0 | - | 0 | - | a |
| a | 0 | 0 | 0 | 1 | 0 | 0 | - | 0 | - | 0 | - | b |
| b | 0 | 0 | 1 | 0 | 0 | - | 0 | 0 | - | 0 | - | c |
| b | 0 | 0 | 1 | 1 | 0 | - | 0 | 0 | - | 0 | - | b |
| c | 0 | 1 | 0 | 0 | 0 | - | - | - | 0 | 0 | - | d |
| c | 0 | 1 | 0 | 1 | 0 | - | - | - | 0 | 0 | - | c |
| d | 0 | 1 | 1 | 0 | 0 | - | - | - | - | - | 0 | e |
| d | 0 | 1 | 1 | 1 | 0 | - | - | - | - | - | 0 | d |
| e | 1 | 0 | 0 | 0 | 0 | - | - | - | - | - | - | f |
| e | 1 | 0 | 0 | 1 | 0 | - | - | - | - | - | - | e |
| f | 1 | 0 | 1 | 0 | 1 | - | - | - | - | - | - | a |
| f | 1 | 0 | 1 | 1 | 0 | - | - | - | - | - | - | a |


**PULSE FSM State Table:**

| State | Z | Y | X | GO | P0 | P1 | P2 | PS | SYN | Next State |
|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 | 0 | 0 | 0 | - | - | -

# Esercizio 19

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-en>/<image19>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-en>/<video19>.mp4")

image_description = """The system has a controller-datapath structure.
The system has fourteen inputs (CK, -Reset, PA3...PA0, PB3...PB0, PC3...PC0) and three outputs (UA, UB, UC).
The CONTROL FSM acts as the controller and needs to be designed.

* **The CONTROL FSM has:**
    * State variables X, Y, and Z;
    * Outputs UA, UB, UC, LDA, LDB, and LDC;
    * Inputs TCA, TCB, and TCC;
    * Clock signal CK;
    * Initialization signal -Reset.

The datapath consists of:

* **4-bit Universal Counter A**
    * Receives the clock signal (CK) common to all synchronous components.
    * The enable counting signal (En) is constantly set to one.
    * The terminal count enable signal (Et) is constantly set to one.
    * The U/-D signal, indicating the counting direction, is constantly set to zero.
    * Receives -Reset as the -Clear signal.
    * The load signal (LD) is connected to the LDA signal coming from the CONTROL FSM.
    * The TC output enters the CONTROL FSM as the TCA input.
    * Inputs P3...P0 are the PA3...PA0 inputs of the system.
    * The Q3...Q0 outputs are not connected to anything.

* **4-bit Universal Counter B**
    * Receives the clock signal (CK) common to all synchronous components.
    * The enable counting signal (En) is constantly set to one.
    * The terminal count enable signal (Et) is constantly set to one.
    * The U/-D signal, indicating the counting direction, is constantly set to zero.
    * Receives -Reset as the -Clear signal.
    * The load signal (LD) is connected to the LDB signal coming from the CONTROL FSM.
    * The TC output enters the CONTROL FSM as the TCB input.
    * Inputs P3...P0 are the PB3...PB0 inputs of the system.
    * The Q3...Q0 outputs are not connected to anything.

* **4-bit Universal Counter C**
    * Receives the clock signal (CK) common to all synchronous components.
    * The enable counting signal (En) is constantly set to one.
    * The terminal count enable signal (Et) is constantly set to one.
    * The U/-D signal, indicating the counting direction, is constantly set to zero.
    * Receives -Reset as the -Clear signal.
    * The load signal (LD) is connected to the LDC signal coming from the CONTROL FSM.
    * The TC output enters the CONTROL FSM as the TCC input.
    * Inputs P3...P0 are the PC3...PC0 inputs of the system.
    * The Q3...Q0 outputs are not connected to anything.
"""

prompt_text = f"""The goal is to design a programmable multiphase square-wave generator.

This device generates three cyclic signals, UA, UB, and UC, which behave as follows:
- Initially, UA, UB, and UC are all low for one clock cycle;
- Then, UA is high for a number of clock cycles equal to (TA+1), while UB and UC remain low.
- Then, UA, UB, and UC are all low for one clock cycle;
- Next, UA goes low, and UB becomes high for a number of clock cycles equal to (TB+1), while UC remains low.
- Then, UA, UB, and UC are all low for one clock cycle;
- Finally, UA remains low, UB goes low, and UC becomes high for a number of clock cycles equal to (TC+1).

The duration of the high time for each signal (TA, TB, TC) can be independently programmed. The system's architecture is described in the figure {image}.
All modules are synchronized with the main clock CK. The finite state machine (FSM) generates the required signals UA, UB, and UC. The three universal 4-bit counters (Cnt4) are responsible for setting the time duration of each output signal and are controlled by the FSM.

Below is the description of the system:
{image_description}

Design the CONTROL FSM as a synchronous Moore machine and create its state table.
"""

prompt_text_FW = f"""The goal is to design a programmable multiphase square-wave generator.

This device generates three cyclic signals, UA, UB, and UC, which behave as follows:
- Initially, UA, UB, and UC are all low for one clock cycle;
- Then, UA is high for a number of clock cycles equal to (TA+1), while UB and UC remain low.
- Then, UA, UB, and UC are all low for one clock cycle;
- Next, UA goes low, and UB becomes high for a number of clock cycles equal to (TB+1), while UC remains low.
- Then, UA, UB, and UC are all low for one clock cycle;
- Finally, UA remains low, UB goes low, and UC becomes high for a number of clock cycles equal to (TC+1).

The duration of the high time for each signal (TA, TB, TC) can be independently programmed. The system's architecture is described in the figure {image}.
All modules are synchronized with the main clock CK. The finite state machine (FSM) generates the required signals UA, UB, and UC. The three universal 4-bit counters (Cnt4) are responsible for setting the time duration of each output signal and are controlled by the FSM.

Design the CONTROL FSM as a synchronous Moore machine and create its state table.
"""

## 1.1 Esercizio senza ulteriore contesto

In [51]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## CONTROL FSM Design:

**States:**

We need a state for each phase of the output signals, plus an initial state for loading the counters. This gives us a total of 7 states:

* **S0: Idle/Load Counters:** All outputs low, load counters with programmed values.
* **S1: All Low:** All outputs low for one clock cycle.
* **S2: UA High:** UA high, UB and UC low.
* **S3: All Low (2):** All outputs low for one clock cycle.
* **S4: UB High:** UA low, UB high, UC low.
* **S5: All Low (3):** All outputs low for one clock cycle.
* **S6: UC High:** UA and UB low, UC high.

**State Variables:**

We can encode these 7 states using three state variables (X, Y, Z) as follows:

| State | X | Y | Z |
|---|---|---|---|
| S0 | 0 | 0 | 0 |
| S1 | 0 | 0 | 1 |
| S2 | 0 | 1 | 0 |
| S3 | 0 | 1 | 1 |
| S4 | 1 | 0 | 0 |
| S5 | 1 | 0 | 1 |
| S6 | 1 | 1 | 0 |

**State Table:**

| Present State | Inputs | Next State | Outputs | Load Signals |
|---|---|---|---|---|
| X Y Z | TCA TCB TCC | X* 

## 1.2 Esercizio con recupero diretto

In [52]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## State Table for the CONTROL FSM:

| Present State | X | Y | Z | TCA | TCB | TCC | -Reset | Next State | UA | UB | UC | LDA | LDB | LDC |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| S0 (Init) | 0 | 0 | 0 | X | X | X | 1 | S0 | 0 | 0 | 0 | 1 | 1 | 1 |
| S0 (Init) | 0 | 0 | 0 | X | X | X | 0 | S1 | 0 | 0 | 0 | 0 | 0 | 0 |
| S1 | 0 | 0 | 1 | X | X | X | 0 | S2 | 1 | 0 | 0 | 0 | 0 | 0 |
| S2 | 0 | 1 | 0 | 1 | X | X | 0 | S3 | 0 | 0 | 0 | 0 | 0 | 0 |
| S3 | 0 | 1 | 1 | X | X | X | 0 | S4 | 0 | 1 | 0 | 0 | 1 | 0 |
| S4 | 1 | 0 | 0 | X | 1 | X | 0 | S5 | 0 | 0 | 0 | 0 | 0 | 0 |
| S5 | 1 | 0 | 1 | X | X | X | 0 | S0 | 0 | 0 | 1 | 0 | 0 | 1 |

**State Variables:**

* X, Y, Z: Used to encode the eight states of the FSM.

**Inputs:**

* TCA: Terminal count signal from Counter A.
* TCB: Terminal count signal from Counter B.
* TCC: Terminal count signal from Counter C.
* -Reset: Initialization signal.

**Outputs:**

* UA: Output signal A.
* UB: Output 

## 1.3 Esercizio con recupero diretto e video soluzione

In [53]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Below is additional context that should be used to produce the correct response:
{text_retrieved}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is asking for the state table of a synchronous Moore machine that controls a multiphase square-wave generator. 

Here's the state table based on the provided context and diagram:

| State (W,Z,Y,X) | TCA | TCB | TCC | LDA | LDB | LDC | UA | UB | UC | Next State (W,Z,Y,X) |
|---|---|---|---|---|---|---|---|---|---|---|
| A (0,0,0,0) | - | - | - | 1 | 0 | 0 | 0 | 0 | 0 | B (0,0,0,1) |
| B (0,0,0,1) | 0 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | C (0,0,1,0) |
| B (0,0,0,1) | 1 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | B (0,0,0,1) |
| C (0,0,1,0) | - | - | - | 0 | 0 | 0 | 1 | 0 | 0 | D (0,0,1,1) |
| D (0,0,1,1) | - | - | - | 0 | 1 | 0 | 0 | 0 | 0 | E (0,1,0,0) |
| E (0,1,0,0) | - | 0 | - | 0 | 0 | 0 | 0 | 1 | 0 | F (0,1,0,1) |
| E (0,1,0,0) | - | 1 | - | 0 | 0 | 0 | 0 | 1 | 0 | E (0,1,0,0) |
| F (0,1,0,1) | - | - | - | 0 | 0 | 0 | 0 | 1 | 0 | G (0,1,1,0) |
| G (0,1,1,0) | - | - | - | 0 | 0 | 1 | 0 | 0 | 0 | H (0,1,1,1) |
| H (0,1,1,1) | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 

## 1.4 Esercizio con video soluzione

In [54]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
To provide the correct answer, use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is designing a programmable multiphase square-wave generator with a control FSM. The video provides a step-by-step solution for designing the control FSM and creating its state table. 

The user starts by drawing a diagram of the control FSM, including states A, B, C, D, E, F, G, H, and I. Each state represents a different stage in the generation of the square-wave signals UA, UB, and UC. The transitions between states are determined by the terminal count outputs of the three counters (TCA, TCB, TCC).

The user then creates a state table that lists the current state, the inputs (TCA, TCB, TCC), the outputs (LDA, LDB, LDC, UA, UB, UC), and the next state. The state table is filled out based on the behavior of the control FSM as described in the problem statement.

For example, in state A, the output LDA is set to 1 to load the counter A with the desired value. The next state is B, regardless of the inputs. In state B, the output UA is set to 1 to genera

## 1.5 Esercizio con libro di testo nel prompt

In [55]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## State Table for the CONTROL FSM

**State Variables:** X, Y, Z (We'll use 3 bits to encode 8 states)

**Inputs:** TCA, TCB, TCC

**Outputs:** UA, UB, UC, LDA, LDB, LDC

**State Table:**

| State | XYZ | TCA | TCB | TCC | UA | UB | UC | LDA | LDB | LDC | Next State |
|---|---|---|---|---|---|---|---|---|---|---|---|
| S0 | 000 | - | - | - | 0 | 0 | 0 | 1 | 1 | 1 | S1 |
| S1 | 001 | 0 | - | - | 1 | 0 | 0 | 0 | 0 | 0 | S1 |
| S1 | 001 | 1 | - | - | 1 | 0 | 0 | 0 | 0 | 0 | S2 |
| S2 | 010 | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | S3 |
| S3 | 011 | - | 0 | - | 0 | 1 | 0 | 0 | 0 | 0 | S3 |
| S3 | 011 | - | 1 | - | 0 | 1 | 0 | 0 | 0 | 0 | S4 |
| S4 | 100 | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | S5 |
| S5 | 101 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | 0 | S5 |
| S5 | 101 | - | - | 1 | 0 | 0 | 1 | 0 | 0 | 0 | S0 |

**State Descriptions:**

* **S0 (Initialization):** All outputs are low. Load the values from PA3..PA0, PB3..PB0, and PC3..PC0 into counters A, B, and C respectively.
* 

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [56]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
```
State	W	X	Y	Z	TCA	TCB	TCC	LDA	LDB	LDC	UA	UB	UC
A	0	0	0	0	-	-	-	1	1	1	0	0	0
B	0	0	0	1	1	-	-	0	0	0	1	0	0
C	0	0	1	0	-	1	-	0	0	0	0	0	0
D	0	0	1	1	-	-	-	0	0	0	0	1	0
E	0	1	0	0	-	-	1	0	0	0	0	0	0
F	0	1	0	1	-	-	-	0	0	0	0	0	1
G	0	1	1	0	-	-	-	1	0	0	0	0	1
H	0	1	1	1	-	-	-	1	1	0	0	0	1
I	1	0	0	0	-	-	-	0	1	0	0	0	1
L	1	0	0	1	-	-	-	0	1	1	0	0	1
M	1	0	1	0	-	-	-	0	0	1	0	0	1
N	1	0	1	1	-	-	-	0	0	1	0	0	0
O	1	1	0	0	-	-	-	0	0	1	0	0	0
P	1	1	0	1	-	-	-	1	1	1	0	0	0
```

The state table describes the behavior of the CONTROL FSM, a synchronous Moore machine. The table lists the current state (WXYZ), the inputs (TCA, TCB, TCC), the outputs (LDA, LDB, LDC, UA, UB, UC), and the next state. 

Here's a breakdown of the state table:

**State Variables:**
- W, X, Y, Z: Represent the current state of the FSM.

**Inputs:**
- TCA: Terminal Count signal from Counter A.
- TCB: Terminal Count signal from Counter B.
- TCC: Terminal Count signal from Counter C.

**Outputs:**
- LDA: Load signal for Counte

## 1.7 Esercizio con recupero mediante tool

In [59]:
prompt_text = f"""The goal is to design a programmable multiphase square-wave generator. The system architecture is described below:
{image_description}

This device generates three cyclic signals, UA, UB, and UC, which behave as follows:
- Initially, UA is high for a number of clock cycles equal to TA, while UB and UC remain low.
- After that, UA goes low, UB becomes high for a number of clock cycles equal to TB, while UC remains low.
- Finally, UA stays low, UB goes low, and UC becomes high for a number of clock cycles equal to TC.

The duration of the high time for each signal (TA, TB, TC) can be independently programmed. The system's architecture is described in the figure {image}.
All modules are synchronized with the main clock CK. The finite state machine (FSM) generates the required signals UA, UB, and UC. The three universal 4-bit counters (Cnt4) are responsible for setting the time duration of each output signal and are controlled by the FSM.

Design the CONTROL FSM and create its state table.
"""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

The CONTROL FSM has eight states (S0-S7) and uses three state variables (X, Y, Z). The FSM remains in the initial state (S0) as long as the reset signal is active (-Reset = 1). When the reset is deactivated (-Reset = 0), the FSM transitions to state (S1). The outputs UA, UB, and UC are activated sequentially in states (S1), (S3), and (S5), respectively. The counters are loaded with the values provided at the inputs PA, PB, and PC in states (S0), (S2), (S4), and (S6).

**State Table:**

| State | X | Y | Z | -Reset | TCA | TCB | TCC | UA | UB | UC | LDA | LDB | LDC |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| S0 | 0 | 0 | 0 | 1 | - | - | - | 0 | 0 | 0 | 1 | 1 | 1 |
| S0 | 0 | 0 | 0 | 0 | - | - | - | 0 | 0 | 0 | 1 | 1 | 1 |
| S1 | 0 | 0 | 1 | 0 | 1 | - | - | 1 | 0 | 0 | 0 | 0 | 0 |
| S2 | 0 | 1 | 0 | 0 | 0 | - | - | 1 | 0 | 0 | 1 | 0 | 0 |
| S3 | 0 | 1 | 1 | 0 | - | 1 | - | 0 | 1 | 0 | 0 | 0 | 0 |
| S4 | 1 | 0 | 0 | 0 | - | 0 | - | 0 | 1 | 0 | 0 | 1 | 0 |
| S5 | 1 | 0 | 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "fsm design for programmable multiphase square-wave generator"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "fsm design for programmable multiphase square-wave generator"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [57]:
input = f'{{"System": "{image}", "Task": "{prompt_text_FW}", "System description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
+-------+---+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+
| State | X | Y | Z | TCA| TCB| TCC| UA| UB| UC| LDA| LDB| LDC|    Next State    | iX| iY| iZ|
+-------+---+---+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+
|  S0   | 0 | 0 | 0 |  - |  - |  - | 0 | 0 | 0 |  0 |  0 |  0 |        S1        | 0 | 0 | 1 |
|  S1   | 0 | 0 | 1 |  - |  - |  - | 1 | 0 | 0 |  1 |  0 |  0 |        S2        | 0 | 1 | 0 |
|  S2   | 0 | 1 | 0 |  1 |  - |  - | 1 | 0 | 0 |  0 |  0 |  0 |        S2        | 0 | 1 | 0 |
|  S2   | 0 | 1 | 0 |  0 |  - |  - | 0 | 0 | 0 |  0 |  1 |  0 |        S3        | 1 | 0 | 0 |
|  S3   | 1 | 0 | 0 |  - |  1 |  - | 0 | 1 | 0 |  0 |  0 |  0 |        S3        | 1 | 0 | 0 |
|  S3   | 1 | 0 | 0 |  - |  0 |  - | 0 | 0 | 0 |  0 |  0 |  1 |        S4        | 1 | 0 | 1 |
|  S4   | 1 | 0 | 1 |  - |  - |  1 | 0 | 0 | 1 |  0 |  0 |  0 |        S4        | 1 | 0 | 1 |
|  S4   | 1 | 0 | 1 |  - |  - |  0 | 0 | 0 

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [58]:
input = f'{{"System": "{image}", "Task": "{prompt_text_FW}", "System description": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Example #1
Input: {input1}
Output: {solution1}
2. Example #2
Input: {input2}
Output: {solution2}
3. Example #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Use the following PDF file, which contains useful information on finite state machines, to provide the answer:
{textbook}
To provide the correct answer, also use the solution provided in the video format:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
+-------+---+---+---+---+-----+-----+-----+-----+-----+-----+-----+-----+-----------------+---+---+---+---+
| State | W | Z | Y | X | TCA | TCB | TCC | LDA | LDB | LDC | UA | UB | UC |   Next State    | iW| iZ| iY| iX|
+-------+---+---+---+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------------+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 |  -  |  -  |  -  |  1  |  1  |  1  |  0 |  0 |  0 |        b        | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 1 |  -  |  -  |  -  |  0  |  0  |  0  |  1 |  0 |  0 |        c        | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 1 | 0 |  0  |  -  |  -  |  0  |  0  |  0  |  1 |  0 |  0 |        c        | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 1 | 0 |  1  |  -  |  -  |  0  |  0  |  0  |  1 |  0 |  0 |        d        | 0 | 0 | 1 | 1 |
|   d   | 0 | 0 | 1 | 1 |  -  |  -  |  -  |  0  |  0  |  0  |  0 |  0 |  0 |        e        | 0 | 1 | 0 | 0 |
|   e   | 0 | 1 | 0 | 0 |  -  |  -  |  -  |  0  |  1  |  0  |  0 |  1 |  0 |        f  